# modbus

In [ ]:
!pip install minimalmodbus

In [1]:
import minimalmodbus
import tkinter as tk
from tkinter import ttk
import serial.tools.list_ports
import time
import threading
import csv
import os

SAMPLE_INTERVAL = 1
LOG_FILE = "log.csv"

# Inicjalizacja pliku CSV
if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "port", "slave_id", "register", "value"])

# Funkcja do wykrywania portów COM
def get_com_ports():
    return [port.device for port in serial.tools.list_ports.comports()]

# Główna funkcja odczytu
def read_loop(port, slave_id, register_addr, baudrate, reg_type):
    try:
        instrument = minimalmodbus.Instrument(port, int(slave_id))
        instrument.serial.baudrate = int(baudrate)
        instrument.serial.timeout = 0.2
        instrument.mode = minimalmodbus.MODE_RTU
        instrument.clear_buffers_before_each_transaction = True
    except Exception as e:
        label_status.config(text=f"❌ Błąd inicjalizacji: {e}")
        return

    label_status.config(text=f"✅ Odczyt z {port}, Rejestr {register_addr}")
    
    while not stop_flag.is_set():
        try:
            functioncode = 3 if reg_type == "Holding Register" else 4
            value = instrument.read_register(int(register_addr), 0, functioncode=functioncode)
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            label_value.config(text=f"Wartość: {value}")

            with open(LOG_FILE, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([timestamp, port, slave_id, register_addr, value])
        except Exception as e:
            label_value.config(text=f"Błąd odczytu: {e}")
        time.sleep(SAMPLE_INTERVAL)

# Obsługa startu
def start_reading():
    global active_thread

    port = combo_port.get()
    slave_id = entry_slave.get()
    register_addr = entry_register.get()
    baudrate = combo_baud.get()
    reg_type = combo_regtype.get()

    if not port or not slave_id.isdigit() or not register_addr.isdigit():
        label_status.config(text="❗ Uzupełnij poprawnie wszystkie pola.")
        return

    stop_flag.clear()
    if active_thread and active_thread.is_alive():
        stop_flag.set()
        active_thread.join()

    active_thread = threading.Thread(
        target=read_loop, args=(port, slave_id, register_addr, baudrate, reg_type), daemon=True)
    active_thread.start()

# GUI setup
root = tk.Tk()
root.title("Modbus RTU GUI Reader")
root.geometry("400x350")

frame_top = tk.Frame(root)
frame_top.pack(pady=10)

# Port
tk.Label(frame_top, text="Port COM:").grid(row=0, column=0, sticky="e")
combo_port = ttk.Combobox(frame_top, values=get_com_ports(), width=15)
combo_port.grid(row=0, column=1, padx=5)
combo_port.set(get_com_ports()[0] if get_com_ports() else "")

# Slave ID
tk.Label(frame_top, text="Slave ID:").grid(row=1, column=0, sticky="e")
entry_slave = tk.Entry(frame_top, width=10)
entry_slave.grid(row=1, column=1, padx=5)
entry_slave.insert(0, "1")

# Register
tk.Label(frame_top, text="Adres rejestru:").grid(row=2, column=0, sticky="e")
entry_register = tk.Entry(frame_top, width=10)
entry_register.grid(row=2, column=1, padx=5)
entry_register.insert(0, "100")

# Typ rejestru
tk.Label(frame_top, text="Typ rejestru:").grid(row=3, column=0, sticky="e")
combo_regtype = ttk.Combobox(frame_top, values=["Holding Register", "Input Register"], width=15)
combo_regtype.grid(row=3, column=1, padx=5)
combo_regtype.set("Holding Register")

# Baudrate
tk.Label(frame_top, text="Baudrate:").grid(row=4, column=0, sticky="e")
combo_baud = ttk.Combobox(frame_top, values=["9600", "19200", "38400", "57600", "115200"], width=15)
combo_baud.grid(row=4, column=1, padx=5)
combo_baud.set("9600")

# Przycisk start
btn_start = tk.Button(root, text="▶ Start", font=("Arial", 12), command=start_reading)
btn_start.pack(pady=10)

# Status i wynik
label_status = tk.Label(root, text="Czekam na dane...", font=("Arial", 10), fg="blue")
label_status.pack(pady=5)

label_value = tk.Label(root, text="Wartość: ---", font=("Arial", 16))
label_value.pack(pady=5)

# Zmienna sterująca i pętla GUI
stop_flag = threading.Event()
active_thread = None

root.mainloop()


KeyboardInterrupt: 

In [ ]:
import tkinter as tk
from tkinter import ttk
import subprocess
import threading

def run_mbpoll():
    port = combo_port.get()
    slave_id = entry_slave.get()
    reg = entry_register.get()
    baud = combo_baud.get()
    count = entry_count.get()
    parity = combo_parity.get()
    datatype = combo_type.get()

    if not (port and reg.isdigit() and slave_id.isdigit()):
        label_result.config(text="❗ Wprowadź poprawne dane.")
        return

    cmd = [
        "mbpoll", "-m", "rtu",
        "-a", slave_id,
        "-r", reg,
        "-c", count,
        "-b", baud,
        "-P", parity,
        "-t", datatype,
        port
    ]

    def worker():
        try:
            result = subprocess.check_output(cmd, stderr=subprocess.STDOUT, universal_newlines=True)
            label_result.config(text=result.splitlines()[-1])  # pokaż tylko ostatnią linię (np. [8736]: 8)
        except subprocess.CalledProcessError as e:
            label_result.config(text=f"❌ Błąd: {e.output}")

    threading.Thread(target=worker, daemon=True).start()


# === GUI ===
root = tk.Tk()
root.title("Modbus mbpoll GUI")
root.geometry("400x320")

frame = tk.Frame(root)
frame.pack(pady=10)

# Port
tk.Label(frame, text="Port (np. /dev/ttyUSB0):").grid(row=0, column=0, sticky="e")
combo_port = ttk.Combobox(frame, values=["/dev/ttyUSB0", "/dev/ttyACM0", "/dev/ttyCOM7"], width=20)
combo_port.grid(row=0, column=1)
combo_port.set("/dev/ttyCOM7")

# Slave ID
tk.Label(frame, text="Slave ID:").grid(row=1, column=0, sticky="e")
entry_slave = tk.Entry(frame)
entry_slave.grid(row=1, column=1)
entry_slave.insert(0, "1")

# Adres rejestru
tk.Label(frame, text="Adres rejestru:").grid(row=2, column=0, sticky="e")
entry_register = tk.Entry(frame)
entry_register.grid(row=2, column=1)
entry_register.insert(0, "8736")

# Liczba rejestrów
tk.Label(frame, text="Ilość rejestrów:").grid(row=3, column=0, sticky="e")
entry_count = tk.Entry(frame)
entry_count.grid(row=3, column=1)
entry_count.insert(0, "1")

# Baudrate
tk.Label(frame, text="Baudrate:").grid(row=4, column=0, sticky="e")
combo_baud = ttk.Combobox(frame, values=["9600", "19200", "38400", "115200"], width=20)
combo_baud.grid(row=4, column=1)
combo_baud.set("9600")

# Parzystość
tk.Label(frame, text="Parzystość:").grid(row=5, column=0, sticky="e")
combo_parity = ttk.Combobox(frame, values=["none", "even", "odd"], width=20)
combo_parity.grid(row=5, column=1)
combo_parity.set("none")

# Typ danych
tk.Label(frame, text="Typ danych:").grid(row=6, column=0, sticky="e")
combo_type = ttk.Combobox(frame, values=["4:int", "4:int16", "4:uint16"], width=20)
combo_type.grid(row=6, column=1)
combo_type.set("4:int")

# Przycisk
btn_run = tk.Button(root, text="▶ Odczytaj", command=run_mbpoll, font=("Arial", 12))
btn_run.pack(pady=10)

# Wynik
label_result = tk.Label(root, text="Brak danych", font=("Courier", 12))
label_result.pack(pady=10)

root.mainloop()


In [1]:
from tkinter import Canvas
import random

# Dodajemy animowaną grafikę do głównego okna Tkinter
canvas = Canvas(root, width=380, height=60, bg="#f0f0f0", highlightthickness=0)
canvas.pack(pady=5)

balls = []
colors = ["#4caf50", "#2196f3", "#ff9800", "#e91e63", "#9c27b0"]

for _ in range(8):
    x = random.randint(10, 350)
    y = random.randint(10, 50)
    r = random.randint(8, 16)
    color = random.choice(colors)
    ball = canvas.create_oval(x, y, x + r, y + r, fill=color, outline="")
    balls.append((ball, r, color, random.choice([-2, 2]), random.choice([-2, 2])))

def animate():
    for i, (ball, r, color, dx, dy) in enumerate(balls):
        coords = canvas.coords(ball)
        if coords[0] + dx < 0 or coords[2] + dx > 380:
            dx = -dx
        if coords[1] + dy < 0 or coords[3] + dy > 60:
            dy = -dy
        canvas.move(ball, dx, dy)
        balls[i] = (ball, r, color, dx, dy)
    root.after(30, animate)

animate()

NameError: name 'root' is not defined

In [ ]:
import minimalmodbus
import tkinter as tk
from tkinter import ttk
import serial.tools.list_ports
import threading
import time

# Parametry globalne
SAMPLE_INTERVAL = 1.0  # Sekundy
instrument = None
stop_flag = threading.Event()

def get_serial_ports():
    return [port.device for port in serial.tools.list_ports.comports()]

def start_reading():
    global instrument

    port = combo_port.get()
    slave_id = entry_slave.get()
    register = entry_register.get()
    baudrate = combo_baud.get()
    reg_type = combo_regtype.get()

    # Walidacja danych
    if not (port and slave_id.isdigit() and register.isdigit()):
        label_status.config(text="❗ Uzupełnij wszystkie pola poprawnie.")
        return

    try:
        instrument = minimalmodbus.Instrument(port, int(slave_id))
        instrument.serial.baudrate = int(baudrate)
        instrument.serial.timeout = 0.2
        instrument.mode = minimalmodbus.MODE_RTU
        instrument.clear_buffers_before_each_transaction = True
    except Exception as e:
        label_status.config(text=f"Błąd inicjalizacji: {e}")
        return

    # Uruchom wątek odczytu
    stop_flag.clear()
    thread = threading.Thread(target=read_loop, args=(int(register), reg_type), daemon=True)
    thread.start()
    label_status.config(text=f"✅ Trwa odczyt...")

def read_loop(register, reg_type):
    while not stop_flag.is_set():
        try:
            functioncode = 3 if reg_type == "Holding Register" else 4
            value = instrument.read_register(register, 0, functioncode=functioncode)
            label_value.config(text=f"Wartość: {value}")
        except Exception as e:
            label_value.config(text=f"Błąd: {e}")
        time.sleep(SAMPLE_INTERVAL)

def stop_reading():
    stop_flag.set()
    label_status.config(text="⏹️ Zatrzymano odczyt")

# === GUI ===
root = tk.Tk()
root.title("Modbus RTU (Python + Tkinter)")
root.geometry("400x350")

frame = tk.Frame(root)
frame.pack(pady=10)

# Port COM
tk.Label(frame, text="Port:").grid(row=0, column=0, sticky="e")
combo_port = ttk.Combobox(frame, values=get_serial_ports(), width=20)
combo_port.grid(row=0, column=1)
combo_port.set(get_serial_ports()[0] if get_serial_ports() else "")

# Slave ID
tk.Label(frame, text="Slave ID:").grid(row=1, column=0, sticky="e")
entry_slave = tk.Entry(frame)
entry_slave.grid(row=1, column=1)
entry_slave.insert(0, "1")

# Adres rejestru
tk.Label(frame, text="Adres rejestru:").grid(row=2, column=0, sticky="e")
entry_register = tk.Entry(frame)
entry_register.grid(row=2, column=1)
entry_register.insert(0, "100")

# Typ rejestru
tk.Label(frame, text="Typ rejestru:").grid(row=3, column=0, sticky="e")
combo_regtype = ttk.Combobox(frame, values=["Holding Register", "Input Register"], width=20)
combo_regtype.grid(row=3, column=1)
combo_regtype.set("Holding Register")

# Baudrate
tk.Label(frame, text="Baudrate:").grid(row=4, column=0, sticky="e")
combo_baud = ttk.Combobox(frame, values=["9600", "19200", "38400", "57600", "115200"], width=20)
combo_baud.grid(row=4, column=1)
combo_baud.set("9600")

# Start/Stop
tk.Button(root, text="▶ Start", command=start_reading, font=("Arial", 11)).pack(pady=5)
tk.Button(root, text="⏹️ Stop", command=stop_reading, font=("Arial", 11)).pack()

# Wynik i status
label_value = tk.Label(root, text="Wartość: ---", font=("Arial", 16))
label_value.pack(pady=10)

label_status = tk.Label(root, text="⏳ Oczekiwanie na uruchomienie...", font=("Arial", 10), fg="gray")
label_status.pack(pady=5)

root.mainloop()


: 

In [ ]:
import customtkinter as ctk
import minimalmodbus
import serial.tools.list_ports
import threading
import time

# === GLOBALNE ===
SAMPLE_INTERVAL = 0.5
instrument = None
stop_flag = threading.Event()

# === FUNKCJE ===
def get_ports():
    return [port.device for port in serial.tools.list_ports.comports()]

def start_reading():
    global instrument
    port = port_combo.get()
    slave_id = slave_entry.get()
    register = reg_entry.get()
    baud = baud_combo.get()
    reg_type = type_combo.get()

    if not (port and slave_id.isdigit() and register.isdigit()):
        status_label.configure(text="❗ Nieprawidłowe dane", text_color="red")
        return

    try:
        instrument = minimalmodbus.Instrument(port, int(slave_id))
        instrument.serial.baudrate = int(baud)
        instrument.serial.timeout = 0.3
        instrument.mode = minimalmodbus.MODE_RTU
        instrument.clear_buffers_before_each_transaction = True
    except Exception as e:
        status_label.configure(text=f"Init error: {e}", text_color="red")
        return

    stop_flag.clear()
    threading.Thread(target=read_loop, args=(int(register), reg_type), daemon=True).start()
    status_label.configure(text="✅ Odczyt trwa...", text_color="green")

def stop_reading():
    stop_flag.set()
    status_label.configure(text="⏹️ Zatrzymano", text_color="gray")

def read_loop(register, reg_type):
    while not stop_flag.is_set():
        try:
            fc = 3 if reg_type == "Holding Register" else 4
            value = instrument.read_register(register, 0, functioncode=fc)
            value_label.configure(text=f"Wartość: {value}")
        except Exception as e:
            value_label.configure(text=f"Błąd: {e}")
        time.sleep(SAMPLE_INTERVAL)

# === GUI ===
ctk.set_appearance_mode("dark")  # 'dark' / 'light' / 'system'
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("🧠 Modbus GUI (Python + customtkinter)")
app.geometry("440x420")

frame = ctk.CTkFrame(app)
frame.pack(pady=20, padx=20, fill="both", expand=True)

# Port
port_combo = ctk.CTkComboBox(frame, values=get_ports(), width=180)
ctk.CTkLabel(frame, text="Port COM:").pack(anchor="w", padx=10, pady=(10, 0))
port_combo.pack(padx=10)

# Slave ID
slave_entry = ctk.CTkEntry(frame, width=180)
ctk.CTkLabel(frame, text="Slave ID:").pack(anchor="w", padx=10, pady=(10, 0))
slave_entry.pack(padx=10)
slave_entry.insert(0, "1")

# Rejestr
reg_entry = ctk.CTkEntry(frame, width=180)
ctk.CTkLabel(frame, text="Adres rejestru:").pack(anchor="w", padx=10, pady=(10, 0))
reg_entry.pack(padx=10)
reg_entry.insert(0, "100")

# Typ rejestru
type_combo = ctk.CTkComboBox(frame, values=["Holding Register", "Input Register"], width=180)
ctk.CTkLabel(frame, text="Typ rejestru:").pack(anchor="w", padx=10, pady=(10, 0))
type_combo.pack(padx=10)
type_combo.set("Holding Register")

# Baudrate
baud_combo = ctk.CTkComboBox(frame, values=["9600", "19200", "38400", "115200"], width=180)
ctk.CTkLabel(frame, text="Baudrate:").pack(anchor="w", padx=10, pady=(10, 0))
baud_combo.pack(padx=10)
baud_combo.set("9600")

# Przycisk start/stop
btn_frame = ctk.CTkFrame(frame)
btn_frame.pack(pady=20)
ctk.CTkButton(btn_frame, text="▶ Start", command=start_reading, width=120).pack(side="left", padx=10)
ctk.CTkButton(btn_frame, text="⏹ Stop", command=stop_reading, width=120, fg_color="red").pack(side="left", padx=10)

# Wynik
value_label = ctk.CTkLabel(app, text="Wartość: ---", font=ctk.CTkFont(size=20, weight="bold"))
value_label.pack(pady=10)

# Status
status_label = ctk.CTkLabel(app, text="Czekam na uruchomienie...", font=ctk.CTkFont(size=12))
status_label.pack(pady=5)

app.mainloop()


In [ ]:
!pip install PyQt6

In [1]:
import sys
import minimalmodbus
import serial.tools.list_ports
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout,
    QFormLayout, QComboBox, QLineEdit, QPushButton, QLabel
)
from PyQt6.QtCore import QTimer, Qt

class ModbusApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Modbus RTU Reader (PyQt6)")
        self.setFixedSize(400, 360)

        # === UI Elements ===
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        form = QFormLayout()

        self.port_combo = QComboBox()
        self.port_combo.addItems([port.device for port in serial.tools.list_ports.comports()])
        form.addRow("Port COM:", self.port_combo)

        self.slave_input = QLineEdit("1")
        form.addRow("Slave ID:", self.slave_input)

        self.reg_input = QLineEdit("100")
        form.addRow("Adres rejestru:", self.reg_input)

        self.baud_combo = QComboBox()
        self.baud_combo.addItems(["9600", "19200", "38400", "115200"])
        self.baud_combo.setCurrentText("9600")
        form.addRow("Baudrate:", self.baud_combo)

        self.type_combo = QComboBox()
        self.type_combo.addItems(["Holding Register", "Input Register"])
        form.addRow("Typ rejestru:", self.type_combo)

        self.layout.addLayout(form)

        self.start_btn = QPushButton("▶ Start")
        self.stop_btn = QPushButton("⏹ Stop")
        self.layout.addWidget(self.start_btn)
        self.layout.addWidget(self.stop_btn)

        self.result_label = QLabel("Wartość: ---")
        self.result_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.result_label.setStyleSheet("font-size: 20px; padding: 10px;")
        self.layout.addWidget(self.result_label)

        self.status_label = QLabel("Status: Gotowy")
        self.status_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.layout.addWidget(self.status_label)

        # === Eventy ===
        self.start_btn.clicked.connect(self.start_reading)
        self.stop_btn.clicked.connect(self.stop_reading)

        # === Timer do cyklicznego odczytu ===
        self.timer = QTimer()
        self.timer.setInterval(1000)  # 1s
        self.timer.timeout.connect(self.read_modbus)

        self.instrument = None

    def start_reading(self):
        port = self.port_combo.currentText()
        slave = self.slave_input.text()
        reg = self.reg_input.text()
        baud = self.baud_combo.currentText()

        if not (port and slave.isdigit() and reg.isdigit()):
            self.status_label.setText("❗ Nieprawidłowe dane wejściowe")
            return

        try:
            self.instrument = minimalmodbus.Instrument(port, int(slave))
            self.instrument.serial.baudrate = int(baud)
            self.instrument.serial.timeout = 0.3
            self.instrument.mode = minimalmodbus.MODE_RTU
            self.instrument.clear_buffers_before_each_transaction = True
        except Exception as e:
            self.status_label.setText(f"❌ Init error: {e}")
            return

        self.timer.start()
        self.status_label.setText("✅ Odczyt aktywny...")

    def stop_reading(self):
        self.timer.stop()
        self.result_label.setText("Wartość: ---")
        self.status_label.setText("⏹️ Zatrzymano")

    def read_modbus(self):
        if not self.instrument:
            return
        reg = int(self.reg_input.text())
        reg_type = self.type_combo.currentText()
        function_code = 3 if reg_type == "Holding Register" else 4

        try:
            value = self.instrument.read_register(reg, 0, functioncode=function_code)
            self.result_label.setText(f"Wartość: {value}")
        except Exception as e:
            self.result_label.setText(f"Błąd: {e}")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ModbusApp()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout,
    QLabel, QPushButton, QGraphicsOpacityEffect
)
from PyQt6.QtCore import QPropertyAnimation, Qt, QEasingCurve, QTimer, QRect


class FancyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("💎 Modbus Premium Suite")
        self.setGeometry(300, 200, 480, 300)

        # Styl ciemny + przezroczysty feeling
        self.setStyleSheet("""
            QWidget {
                background-color: #1e1e1e;
                color: #f0f0f0;
                font-family: 'Segoe UI';
                font-size: 15px;
            }
            QPushButton {
                background-color: #2b2b2b;
                color: #00f7ff;
                border: 1px solid #00f7ff;
                border-radius: 10px;
                padding: 8px;
            }
            QPushButton:hover {
                background-color: #003b3b;
            }
        """)

        # Layout
        widget = QWidget()
        layout = QVBoxLayout()
        widget.setLayout(layout)
        self.setCentralWidget(widget)

        # Etykieta z fade-in
        self.label = QLabel("Wartość: ---")
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label.setStyleSheet("font-size: 30px;")
        layout.addWidget(self.label)

        # Fade-in efekt
        self.effect = QGraphicsOpacityEffect()
        self.label.setGraphicsEffect(self.effect)
        self.fade_anim = QPropertyAnimation(self.effect, b"opacity")
        self.fade_anim.setDuration(1000)
        self.fade_anim.setStartValue(0)
        self.fade_anim.setEndValue(1)
        self.fade_anim.setEasingCurve(QEasingCurve.Type.InOutCubic)
        self.fade_anim.start()

        # Przycisk z pulsowaniem
        self.button = QPushButton("▶ Start odczytu")
        layout.addWidget(self.button)

        self.pulse_anim = QPropertyAnimation(self.button, b"geometry")
        self.pulse_anim.setDuration(1000)
        start_geometry = QRect(self.button.geometry())
        end_geometry = QRect(start_geometry.adjusted(-5, -2, 5, 2))
        self.pulse_anim.setStartValue(start_geometry)
        self.pulse_anim.setEndValue(end_geometry)
        self.pulse_anim.setEasingCurve(QEasingCurve.Type.InOutQuad)
        self.pulse_anim.setLoopCount(-1)
        QTimer.singleShot(300, self.pulse_anim.start)

        # Symulacja zmiany wartości co 2 sekundy
        self.val = 42
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_value)
        self.timer.start(2000)

    def update_value(self):
        self.val += 1
        self.label.setText(f"Wartość: {self.val}")
        # Restart fade
        self.fade_anim.stop()
        self.effect.setOpacity(0)
        self.fade_anim.start()


app = QApplication([])
win = FancyWindow()
win.show()
app.exec()


0

In [3]:
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout,
    QLabel, QPushButton, QGraphicsOpacityEffect
)
from PyQt6.QtCore import QPropertyAnimation, Qt, QEasingCurve, QTimer, QRect


class FancyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("💎 Modbus Premium Suite")
        self.setGeometry(300, 200, 480, 360)

        # === Styl GUI ===
        self.setStyleSheet("""
            QWidget {
                background-color: #121212;
                color: #f0f0f0;
                font-family: 'Segoe UI';
                font-size: 15px;
            }
            QPushButton {
                background-color: #1f1f1f;
                color: #00f7ff;
                border: 1px solid #00f7ff;
                border-radius: 12px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #003b3b;
            }
        """)

        # === Layout główny ===
        widget = QWidget()
        self.layout = QVBoxLayout(widget)
        self.layout.setContentsMargins(40, 30, 40, 30)
        self.layout.setSpacing(20)
        self.setCentralWidget(widget)

        # === Label z fade-in ===
        self.label = QLabel("Wartość: ---")
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label.setStyleSheet("font-size: 32px;")
        self.layout.addWidget(self.label)

        self.effect = QGraphicsOpacityEffect()
        self.label.setGraphicsEffect(self.effect)
        self.fade_anim = QPropertyAnimation(self.effect, b"opacity")
        self.fade_anim.setDuration(1000)
        self.fade_anim.setStartValue(0)
        self.fade_anim.setEndValue(1)
        self.fade_anim.setEasingCurve(QEasingCurve.Type.InOutCubic)

        # === Przycisk Start z animacją ===
        self.button = QPushButton("▶ Start odczytu")
        self.layout.addWidget(self.button)
        self.button_animation = None  # dodamy po show()

        # === Timer do symulacji zmiany danych ===
        self.val = 42
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_value)
        self.timer.start(2000)

    def showEvent(self, event):
        super().showEvent(event)
        self.fade_anim.start()

        # Animacja pulsowania przycisku po wyrenderowaniu
        geom = self.button.geometry()
        self.button_animation = QPropertyAnimation(self.button, b"geometry")
        self.button_animation.setDuration(1000)
        self.button_animation.setStartValue(geom)
        self.button_animation.setEndValue(geom.adjusted(-5, -3, 5, 3))
        self.button_animation.setEasingCurve(QEasingCurve.Type.InOutQuad)
        self.button_animation.setLoopCount(-1)
        self.button_animation.start()

    def update_value(self):
        self.val += 1
        self.label.setText(f"Wartość: {self.val}")
        self.fade_anim.stop()
        self.effect.setOpacity(0)
        self.fade_anim.start()


if __name__ == "__main__":
    app = QApplication([])
    window = FancyWindow()
    window.show()
    app.exec()


: 

In [2]:
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QPushButton, QGraphicsOpacityEffect, QComboBox, QLineEdit, QGroupBox
)
from PyQt6.QtCore import QPropertyAnimation, Qt, QEasingCurve, QTimer, QRect


class FancyModbusWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("💎 Modbus Ultra GUI")
        self.setGeometry(300, 200, 600, 480)

        # === Styl klasy premium ===
        self.setStyleSheet("""
            QWidget {
                background-color: #101010;
                color: #eeeeee;
                font-family: 'Segoe UI';
                font-size: 15px;
            }
            QPushButton {
                background-color: #1e1e1e;
                color: #00eaff;
                border: 1px solid #00eaff;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #003d3d;
            }
            QLineEdit, QComboBox {
                background-color: #1e1e1e;
                border: 1px solid #555;
                padding: 6px;
                border-radius: 8px;
                color: #ffffff;
            }
        """)

        # === Layout główny ===
        widget = QWidget()
        self.layout = QVBoxLayout(widget)
        self.layout.setContentsMargins(30, 20, 30, 20)
        self.layout.setSpacing(20)
        self.setCentralWidget(widget)

        # === Grupa: Parametry połączenia ===
        self.conn_group = QGroupBox("⚙️ Parametry połączenia")
        conn_layout = QVBoxLayout()

        self.port_input = QLineEdit("/dev/ttyUSB0")
        self.baudrate_combo = QComboBox()
        self.baudrate_combo.addItems(["9600", "19200", "38400", "57600", "115200"])
        self.parity_combo = QComboBox()
        self.parity_combo.addItems(["None", "Even", "Odd"])
        self.slave_input = QLineEdit("1")

        conn_layout.addWidget(QLabel("Port COM:"))
        conn_layout.addWidget(self.port_input)
        conn_layout.addWidget(QLabel("Baudrate:"))
        conn_layout.addWidget(self.baudrate_combo)
        conn_layout.addWidget(QLabel("Parzystość:"))
        conn_layout.addWidget(self.parity_combo)
        conn_layout.addWidget(QLabel("Slave ID:"))
        conn_layout.addWidget(self.slave_input)

        self.conn_group.setLayout(conn_layout)
        self.layout.addWidget(self.conn_group)

        # === Grupa: Parametry rejestru ===
        self.reg_group = QGroupBox("📍 Rejestr docelowy")
        reg_layout = QVBoxLayout()

        self.register_input = QLineEdit("100")
        self.type_combo = QComboBox()
        self.type_combo.addItems(["Holding Register", "Input Register"])
        self.interval_input = QLineEdit("1000")  # ms

        reg_layout.addWidget(QLabel("Adres rejestru:"))
        reg_layout.addWidget(self.register_input)
        reg_layout.addWidget(QLabel("Typ rejestru:"))
        reg_layout.addWidget(self.type_combo)
        reg_layout.addWidget(QLabel("Odstęp odczytu (ms):"))
        reg_layout.addWidget(self.interval_input)

        self.reg_group.setLayout(reg_layout)
        self.layout.addWidget(self.reg_group)

        # === Label z fade-in ===
        self.label = QLabel("Wartość: ---")
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label.setStyleSheet("font-size: 34px; font-weight: bold;")
        self.layout.addWidget(self.label)

        self.effect = QGraphicsOpacityEffect()
        self.label.setGraphicsEffect(self.effect)
        self.fade_anim = QPropertyAnimation(self.effect, b"opacity")
        self.fade_anim.setDuration(1000)
        self.fade_anim.setStartValue(0)
        self.fade_anim.setEndValue(1)
        self.fade_anim.setEasingCurve(QEasingCurve.Type.InOutCubic)

        # === Przycisk Start + Stop ===
        btn_layout = QHBoxLayout()
        self.start_btn = QPushButton("▶ Start")
        self.stop_btn = QPushButton("⏹ Stop")
        btn_layout.addWidget(self.start_btn)
        btn_layout.addWidget(self.stop_btn)
        self.layout.addLayout(btn_layout)

        # === Symulacja danych ===
        self.val = 42
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_value)

        self.start_btn.clicked.connect(self.start_simulation)
        self.stop_btn.clicked.connect(self.stop_simulation)

    def showEvent(self, event):
        super().showEvent(event)
        # Fade-in na starcie
        self.fade_anim.start()

    def start_simulation(self):
        interval = int(self.interval_input.text())
        self.timer.start(interval)
        self.label.setText("Wartość: ---")
        self.fade_anim.setStartValue(0)
        self.effect.setOpacity(0)
        self.fade_anim.start()

    def stop_simulation(self):
        self.timer.stop()
        self.label.setText("⏹ Zatrzymano")

    def update_value(self):
        self.val += 1
        self.label.setText(f"Wartość: {self.val}")
        self.fade_anim.stop()
        self.effect.setOpacity(0)
        self.fade_anim.start()


app = QApplication([])
window = FancyModbusWindow()
window.show()
app.exec()


0

In [2]:
!pip install pyqtgraph



   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
    --------------------------------------- 0.0/1.9 MB 262.6 kB/s eta 0:00:08
   - -------------------------------------- 0.1/1.9 MB 409.6 kB/s eta 0:00:05
   ----- ---------------------------------- 0.3/1.9 MB 1.3 MB/s eta 0:00:02
   ------------------ --------------------- 0.9/1.9 MB 3.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.2/1.9 MB 4.5 MB/s eta 0:00:01
   ---------------------------------- ----- 1.6/1.9 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 5.3 MB/s eta 0:00:00


In [3]:
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QPushButton, QGraphicsOpacityEffect, QComboBox,
    QLineEdit, QGroupBox
)
from PyQt6.QtCore import QPropertyAnimation, Qt, QEasingCurve, QTimer, QRect
import pyqtgraph as pg
import datetime
import random


class ModbusUltraGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("💎 Modbus Copilot Ultra GUI")
        self.setGeometry(300, 200, 800, 600)

        self.setStyleSheet("""
            QWidget {
                background-color: #101010;
                color: #eeeeee;
                font-family: 'Segoe UI';
                font-size: 15px;
            }
            QPushButton {
                background-color: #1e1e1e;
                color: #00eaff;
                border: 1px solid #00eaff;
                border-radius: 10px;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #003d3d;
            }
            QLineEdit, QComboBox {
                background-color: #1e1e1e;
                border: 1px solid #555;
                padding: 6px;
                border-radius: 8px;
                color: #ffffff;
            }
        """)

        widget = QWidget()
        self.layout = QVBoxLayout(widget)
        self.layout.setContentsMargins(30, 20, 30, 20)
        self.layout.setSpacing(20)
        self.setCentralWidget(widget)

        self.create_connection_section()
        self.create_register_section()
        self.create_display_section()
        self.create_plot()

        self.val = 42
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_value)
        self.start_btn.clicked.connect(self.start_simulation)
        self.stop_btn.clicked.connect(self.stop_simulation)

    def create_connection_section(self):
        conn_group = QGroupBox("⚙️ Parametry połączenia")
        conn_layout = QVBoxLayout()

        self.port_input = QLineEdit("/dev/ttyUSB0")
        self.baudrate_combo = QComboBox()
        self.baudrate_combo.addItems(["9600", "19200", "38400", "57600", "115200"])
        self.parity_combo = QComboBox()
        self.parity_combo.addItems(["None", "Even", "Odd"])
        self.slave_input = QLineEdit("1")

        conn_layout.addWidget(QLabel("Port COM:"))
        conn_layout.addWidget(self.port_input)
        conn_layout.addWidget(QLabel("Baudrate:"))
        conn_layout.addWidget(self.baudrate_combo)
        conn_layout.addWidget(QLabel("Parzystość:"))
        conn_layout.addWidget(self.parity_combo)
        conn_layout.addWidget(QLabel("Slave ID:"))
        conn_layout.addWidget(self.slave_input)

        conn_group.setLayout(conn_layout)
        self.layout.addWidget(conn_group)

    def create_register_section(self):
        reg_group = QGroupBox("📍 Rejestr docelowy")
        reg_layout = QVBoxLayout()

        self.register_input = QLineEdit("100")
        self.type_combo = QComboBox()
        self.type_combo.addItems(["Holding Register", "Input Register"])
        self.interval_input = QLineEdit("1000")

        reg_layout.addWidget(QLabel("Adres rejestru:"))
        reg_layout.addWidget(self.register_input)
        reg_layout.addWidget(QLabel("Typ rejestru:"))
        reg_layout.addWidget(self.type_combo)
        reg_layout.addWidget(QLabel("Odstęp odczytu (ms):"))
        reg_layout.addWidget(self.interval_input)

        reg_group.setLayout(reg_layout)
        self.layout.addWidget(reg_group)

    def create_display_section(self):
        self.label = QLabel("Wartość: ---")
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label.setStyleSheet("font-size: 34px; font-weight: bold;")
        self.layout.addWidget(self.label)

        self.effect = QGraphicsOpacityEffect()
        self.label.setGraphicsEffect(self.effect)
        self.fade_anim = QPropertyAnimation(self.effect, b"opacity")
        self.fade_anim.setDuration(1000)
        self.fade_anim.setStartValue(0)
        self.fade_anim.setEndValue(1)
        self.fade_anim.setEasingCurve(QEasingCurve.Type.InOutCubic)

        btn_layout = QHBoxLayout()
        self.start_btn = QPushButton("▶ Start")
        self.stop_btn = QPushButton("⏹ Stop")
        btn_layout.addWidget(self.start_btn)
        btn_layout.addWidget(self.stop_btn)
        self.layout.addLayout(btn_layout)

    def create_plot(self):
        self.plot_widget = pg.PlotWidget(title="📈 Odczyt w czasie rzeczywistym")
        self.plot_widget.setBackground("#1e1e1e")
        self.plot_widget.showGrid(x=True, y=True)
        self.plot_data = self.plot_widget.plot(pen=pg.mkPen(color="#00f7ff", width=2))
        self.x_data = []
        self.y_data = []
        self.layout.addWidget(self.plot_widget)

    def start_simulation(self):
        interval = int(self.interval_input.text())
        self.timer.start(interval)
        self.label.setText("Wartość: ---")
        self.effect.setOpacity(0)
        self.fade_anim.start()

    def stop_simulation(self):
        self.timer.stop()
        self.label.setText("⏹ Zatrzymano")

    def update_value(self):
        self.val += random.randint(-3, 5)
        now = datetime.datetime.now().strftime("%H:%M:%S")
        self.label.setText(f"Wartość: {self.val}")
        self.effect.setOpacity(0)
        self.fade_anim.start()
        self.x_data.append(now)
        self.y_data.append(self.val)
        if len(self.x_data) > 20:
            self.x_data = self.x_data[-20:]
            self.y_data = self.y_data[-20:]
        self.plot_data.setData(list(range(len(self.y_data))), self.y_data)


app = QApplication([])
window = ModbusUltraGUI()
window.show()
app.exec()


: 

# excel

In [2]:
!pip install openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 330.3 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.9 kB 751.6 kB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.4 MB/s eta 0:00:00


In [1]:
import sys
import pandas as pd
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QFileDialog,
    QLabel, QComboBox, QMessageBox
)
from PyQt6.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure


class TaskVisualizer(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("📅 Zadania osób – Podsumowanie roczne")
        self.setMinimumSize(900, 600)

        self.layout = QVBoxLayout()
        self.setLayout(self.layout)

        self.load_button = QPushButton("📂 Wczytaj plik .xlsx")
        self.load_button.clicked.connect(self.load_excel)
        self.layout.addWidget(self.load_button)

        self.sheet_selector = QComboBox()
        self.sheet_selector.currentIndexChanged.connect(self.plot_summary)
        self.layout.addWidget(QLabel("📅 Wybierz rok:"))
        self.layout.addWidget(self.sheet_selector)

        self.figure = Figure()
        self.canvas = FigureCanvas(self.figure)
        self.layout.addWidget(self.canvas)

        self.sheets_data = {}  # {nazwa_arkusza: DataFrame}

    def load_excel(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Wybierz plik Excel", "", "Excel Files (*.xlsx *.xls)")
        if file_path:
            try:
                xls = pd.ExcelFile(file_path, engine='openpyxl')
                self.sheets_data.clear()
                self.sheet_selector.clear()

                for sheet_name in xls.sheet_names:
                    df = xls.parse(sheet_name)
                    # Sprawdzamy i czyścimy daty
                    for col in df.columns:
                        if "data" in col.lower():
                            df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
                    self.sheets_data[sheet_name] = df
                    self.sheet_selector.addItem(sheet_name)

                QMessageBox.information(self, "Sukces", "✅ Wczytano dane ze wszystkich lat!")
                self.plot_summary()

            except Exception as e:
                QMessageBox.critical(self, "Błąd", f"Nie udało się wczytać pliku:\n{str(e)}")

    def plot_summary(self):
        selected_sheet = self.sheet_selector.currentText()
        if not selected_sheet or selected_sheet not in self.sheets_data:
            return

        df = self.sheets_data[selected_sheet]

        if df.empty or "osoba" not in df.columns.str.lower().tolist():
            QMessageBox.warning(self, "Błąd danych", "Nie znaleziono kolumny 'osoba'.")
            return

        # Normalizacja nazw kolumn
        df.columns = [col.lower() for col in df.columns]

        grouped = df.groupby('osoba')

        summary = {}
        for person, group in grouped:
            completed = group[group['data zakończenia'].notna()].shape[0]
            not_completed = group[group['data zakończenia'].isna()].shape[0]
            summary[person] = {"Ukończone": completed, "Nieukończone": not_completed}

        # Wykres
        self.figure.clear()
        ax = self.figure.add_subplot(111)

        labels = list(summary.keys())
        ukończone = [summary[person]["Ukończone"] for person in labels]
        nieukończone = [summary[person]["Nieukończone"] for person in labels]

        x = range(len(labels))
        ax.bar(x, ukończone, label="Ukończone", color='green')
        ax.bar(x, nieukończone, bottom=ukończone, label="Nieukończone", color='red')

        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=45, ha='right')
        ax.set_title(f"Zadania osób – rok {selected_sheet}")
        ax.set_ylabel("Liczba zadań")
        ax.legend()
        ax.grid(True)

        self.canvas.draw()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = TaskVisualizer()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import sys
import pandas as pd
import numpy as np
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QFileDialog, QMessageBox
)
from PyQt6.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure


class ZadaniaWykresGrupowany(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("📊 Zadania - grupowanie według lat i osób")
        self.setMinimumSize(1200, 800)

        self.layout = QVBoxLayout(self)
        self.setLayout(self.layout)

        self.load_button = QPushButton("📂 Wczytaj plik Excel")
        self.load_button.clicked.connect(self.load_excel)
        self.layout.addWidget(self.load_button)

        self.figure = Figure(figsize=(12, 6))
        self.canvas = FigureCanvas(self.figure)
        self.layout.addWidget(self.canvas)

    def load_excel(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Wybierz plik Excel", "", "Excel Files (*.xlsx *.xls)")
        if not file_path:
            return

        try:
            df = pd.read_excel(file_path, engine='openpyxl')
            df.columns = [str(col).strip().lower() for col in df.columns]

            for date_col in ['data utworzenia', 'data rozpoczęcia', 'data ukończenia']:
                if date_col in df.columns:
                    df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors='coerce')

            if 'przypisane do' not in df.columns or 'data utworzenia' not in df.columns or 'data ukończenia' not in df.columns:
                QMessageBox.warning(self, "Błąd danych", "Brakuje jednej z kolumn: 'Przypisane do', 'Data utworzenia', 'Data ukończenia'")
                return

            df['rok'] = df['data utworzenia'].dt.year

            # Budujemy strukturę: {rok: {osoba: [ukończone, nieukończone]}}
            result = {}
            for (rok, osoba), grupa in df.groupby(['rok', 'przypisane do']):
                ukończone = grupa['data ukończenia'].notna().sum()
                nieukończone = grupa['data ukończenia'].isna().sum()

                if rok not in result:
                    result[rok] = {}
                result[rok][osoba] = [ukończone, nieukończone]

            # Rysujemy jeden wykres
            self.figure.clear()
            ax = self.figure.add_subplot(111)

            bar_width = 0.35
            space_between_persons = 0.3
            space_between_years = 1.0

            x_positions = []
            ukończone_vals = []
            nieukończone_vals = []
            x_labels = []

            xtick_positions = []
            year_labels = []
            current_x = 0

            for rok in sorted(result.keys()):
                osoby = sorted(result[rok].keys())

                for osoba in osoby:
                    ukończone, nieukończone = result[rok][osoba]

                    x1 = current_x
                    x2 = current_x + bar_width

                    x_positions.append((x1, x2))
                    ukończone_vals.append((x1, ukończone))
                    nieukończone_vals.append((x2, nieukończone))
                    x_labels.append(f"{osoba}")

                    current_x += (2 * bar_width + space_between_persons)

                xtick_positions.append(current_x - (space_between_persons + bar_width))
                year_labels.append(str(rok))
                current_x += space_between_years

            # Wykresy słupków
            for x, val in ukończone_vals:
                ax.bar(x, val, width=bar_width, color='green', label='Ukończone' if x == ukończone_vals[0][0] else "")
            for x, val in nieukończone_vals:
                ax.bar(x, val, width=bar_width, color='red', label='Nieukończone' if x == nieukończone_vals[0][0] else "")

            # Oś X: etykiety
            ax.set_xticks([np.mean(pair) for pair in x_positions])
            ax.set_xticklabels(x_labels, rotation=45, ha='right')

            # Druga oś X dla lat
            for xpos, label in zip(xtick_positions, year_labels):
                ax.text(xpos, -max([v for _, v in ukończone_vals + nieukończone_vals]) * 0.1, label,
                        ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transData)

            ax.set_ylabel("Liczba zadań")
            ax.set_title("Zadania ukończone i nieukończone wg osoby i roku")
            ax.legend()
            ax.grid(True, axis='y')

            self.canvas.draw()
            QMessageBox.information(self, "Gotowe", "✅ Wykres wygenerowany automatycznie.")

        except Exception as e:
            QMessageBox.critical(self, "Błąd", f"❌ Nie udało się przetworzyć pliku:\n{str(e)}")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ZadaniaWykresGrupowany()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import pandas as pd
import numpy as np
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QFileDialog, QMessageBox
)
from PyQt6.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure


class ZadaniaWykresGrupowany(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("📊 Zadania - grupowanie według lat i zasobników")
        self.setMinimumSize(1200, 800)

        self.layout = QVBoxLayout(self)
        self.setLayout(self.layout)

        self.load_button = QPushButton("📂 Wczytaj plik Excel")
        self.load_button.clicked.connect(self.load_excel)
        self.layout.addWidget(self.load_button)

        self.figure = Figure(figsize=(12, 6), constrained_layout=True)
        self.canvas = FigureCanvas(self.figure)
        self.layout.addWidget(self.canvas)

    def load_excel(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Wybierz plik Excel", "", "Excel Files (*.xlsx *.xls)")
        if not file_path:
            return

        try:
            df = pd.read_excel(file_path, engine='openpyxl')
            df.columns = [str(col).strip().lower() for col in df.columns]

            for date_col in ['data utworzenia', 'data rozpoczęcia', 'data ukończenia']:
                if date_col in df.columns:
                    df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors='coerce')

            if 'nazwa zasobnika' not in df.columns or 'data utworzenia' not in df.columns or 'data ukończenia' not in df.columns:
                QMessageBox.warning(self, "Błąd danych", "Brakuje jednej z kolumn: 'Nazwa zasobnika', 'Data utworzenia', 'Data ukończenia'")
                return

            # Usuń zasobniki techniczne
            df = df[~df['nazwa zasobnika'].str.upper().isin(['TO DO', 'HOLD', 'INSTRUKCJA'])]

            df['rok'] = df['data utworzenia'].dt.year

            # Budujemy strukturę: {rok: {zasobnik: [ukończone, nieukończone]}}
            result = {}
            for (rok, zasobnik), grupa in df.groupby(['rok', 'nazwa zasobnika']):
                ukończone = grupa['data ukończenia'].notna().sum()
                nieukończone = grupa['data ukończenia'].isna().sum()

                if rok not in result:
                    result[rok] = {}
                result[rok][zasobnik] = [ukończone, nieukończone]

            # Rysujemy wykres
            self.figure.clear()
            ax = self.figure.add_subplot(111)

            bar_width = 0.35
            space_between_items = 0.3
            space_between_years = 1.0

            x_positions = []
            ukończone_vals = []
            nieukończone_vals = []
            x_labels = []

            xtick_positions = []
            year_labels = []
            current_x = 0

            for rok in sorted(result.keys()):
                zasobniki = sorted(result[rok].keys())

                for zasobnik in zasobniki:
                    ukończone, nieukończone = result[rok][zasobnik]

                    x1 = current_x
                    x2 = current_x + bar_width

                    x_positions.append((x1, x2))
                    ukończone_vals.append((x1, ukończone))
                    nieukończone_vals.append((x2, nieukończone))
                    x_labels.append(f"{zasobnik}")

                    current_x += (2 * bar_width + space_between_items)

                xtick_positions.append(current_x - (space_between_items + bar_width))
                year_labels.append(str(rok))
                current_x += space_between_years

            # Rysowanie słupków
            for x, val in ukończone_vals:
                ax.bar(x, val, width=bar_width, color='green', label='Ukończone' if x == ukończone_vals[0][0] else "")
            for x, val in nieukończone_vals:
                ax.bar(x, val, width=bar_width, color='red', label='Nieukończone' if x == nieukończone_vals[0][0] else "")

            # Etykiety X
            ax.set_xticks([np.mean(pair) for pair in x_positions])
            ax.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=10)

            # Napisy z latami pod spodem
            max_height = max([v for _, v in ukończone_vals + nieukończone_vals]) if ukończone_vals else 0
            for xpos, label in zip(xtick_positions, year_labels):
                ax.text(xpos, -max_height * 0.1, label,
                        ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transData)

            ax.set_ylabel("Liczba zadań")
            ax.set_title("Zadania ukończone i nieukończone wg zasobnika i roku")
            ax.legend()
            ax.grid(True, axis='y')

            self.figure.tight_layout()
            self.canvas.draw()

            QMessageBox.information(self, "Gotowe", "✅ Wykres wygenerowany automatycznie.")

        except Exception as e:
            QMessageBox.critical(self, "Błąd", f"❌ Nie udało się przetworzyć pliku:\n{str(e)}")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ZadaniaWykresGrupowany()
    window.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_2136\731900712.py:119: UserWarning: The figure layout has changed to tight
  self.figure.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import sys
import pandas as pd
import numpy as np
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QFileDialog, QMessageBox
)
from PyQt6.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure


class ZadaniaWykresGrupowany(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("📊 Zadania - grupowanie według lat i zasobników")
        self.setMinimumSize(1200, 800)

        self.layout = QVBoxLayout(self)
        self.setLayout(self.layout)

        self.load_button = QPushButton("📂 Wczytaj plik Excel")
        self.load_button.clicked.connect(self.load_excel)
        self.layout.addWidget(self.load_button)

        self.figure = Figure(figsize=(14, 7), constrained_layout=True)
        self.canvas = FigureCanvas(self.figure)
        self.layout.addWidget(self.canvas)

    def load_excel(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Wybierz plik Excel", "", "Excel Files (*.xlsx *.xls)")
        if not file_path:
            return

        try:
            df = pd.read_excel(file_path, engine='openpyxl')
            df.columns = [str(col).strip().lower() for col in df.columns]

            for date_col in ['data utworzenia', 'data rozpoczęcia', 'data ukończenia']:
                if date_col in df.columns:
                    df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors='coerce')

            if 'nazwa zasobnika' not in df.columns or 'data utworzenia' not in df.columns or 'data ukończenia' not in df.columns:
                QMessageBox.warning(self, "Błąd danych", "Brakuje jednej z kolumn: 'Nazwa zasobnika', 'Data utworzenia', 'Data ukończenia'")
                return

            # Filtrowanie zasobników
            df = df[df['nazwa zasobnika'].apply(
    lambda x: str(x).strip().upper() not in ['TO DO', 'HOLD', 'INSTRUKCJA']
)]
            df['rok'] = df['data rozpoczęcia'].dt.year

            # Grupowanie danych: {rok: {zasobnik: [ukończone, nieukończone]}}
            result = {}
            for (rok, zasobnik), grupa in df.groupby(['rok', 'nazwa zasobnika']):
                ukończone = grupa['data ukończenia'].notna().sum()
                nieukończone = grupa['data ukończenia'].isna().sum()

                if rok not in result:
                    result[rok] = {}
                result[rok][zasobnik] = [ukończone, nieukończone]

            # Rysujemy wykres
            self.figure.clear()
            ax = self.figure.add_subplot(111)

            bar_width = 0.35
            space_between_items = 0.3
            space_between_years = 1.0

            x_positions = []
            ukończone_vals = []
            nieukończone_vals = []
            x_labels = []

            xtick_positions = []
            year_labels = []
            current_x = 0

            for rok in sorted(result.keys()):
                zasobniki = sorted(result[rok].keys())

                for zasobnik in zasobniki:
                    ukończone, nieukończone = result[rok][zasobnik]

                    x1 = current_x
                    x2 = current_x + bar_width

                    x_positions.append((x1, x2))
                    ukończone_vals.append((x1, ukończone))
                    nieukończone_vals.append((x2, nieukończone))
                    x_labels.append(f"{zasobnik}\n({rok})")

                    current_x += (2 * bar_width + space_between_items)

                xtick_positions.append(current_x - (space_between_items + bar_width))
                year_labels.append(str(rok))
                current_x += space_between_years

            # Słupki
            for x, val in ukończone_vals:
                ax.bar(x, val, width=bar_width, color='green', label='Ukończone' if x == ukończone_vals[0][0] else "")
            for x, val in nieukończone_vals:
                ax.bar(x, val, width=bar_width, color='red', label='Nieukończone' if x == nieukończone_vals[0][0] else "")

            # Oś X: etykiety z nazwą zasobnika i rokiem
            ax.set_xticks([np.mean(pair) for pair in x_positions])
            ax.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=10)

            # Dodatkowe opisy lat
            max_height = max([v for _, v in ukończone_vals + nieukończone_vals]) if ukończone_vals else 0
            for xpos, label in zip(xtick_positions, year_labels):
                ax.text(xpos, -max_height * 0.1, label,
                        ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transData)

            ax.set_ylabel("Liczba zadań")
            ax.set_title("Zadania ukończone i nieukończone wg zasobnika i roku")
            ax.legend()
            ax.grid(True, axis='y')

            self.canvas.draw()
            QMessageBox.information(self, "Gotowe", "✅ Wykres wygenerowany automatycznie.")

        except Exception as e:
            QMessageBox.critical(self, "Błąd", f"❌ Nie udało się przetworzyć pliku:\n{str(e)}")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ZadaniaWykresGrupowany()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import pandas as pd
import numpy as np
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QFileDialog, QMessageBox
)
from PyQt6.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure


class ZadaniaWykresGrupowany(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("\U0001F4CA Zadania - grupowanie według lat i zasobników")
        self.setMinimumSize(1400, 800)

        self.layout = QVBoxLayout(self)
        self.setLayout(self.layout)

        self.load_button = QPushButton("\U0001F4C2 Wczytaj plik Excel")
        self.load_button.clicked.connect(self.load_excel)
        self.layout.addWidget(self.load_button)

        self.figure = Figure(figsize=(16, 8), constrained_layout=True)
        self.canvas = FigureCanvas(self.figure)
        self.layout.addWidget(self.canvas)

    def load_excel(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Wybierz plik Excel", "", "Excel Files (*.xlsx *.xls)")
        if not file_path:
            return

        try:
            df = pd.read_excel(file_path, engine='openpyxl')
            df.columns = [str(col).strip().lower() for col in df.columns]

            for date_col in ['data utworzenia', 'data rozpoczęcia', 'data ukończenia']:
                if date_col in df.columns:
                    df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors='coerce')

            if 'nazwa zasobnika' not in df.columns or 'data rozpoczęcia' not in df.columns or 'data ukończenia' not in df.columns:
                QMessageBox.warning(self, "Błąd danych", "Brakuje jednej z kolumn: 'Nazwa zasobnika', 'Data rozpoczęcia', 'Data ukończenia'")
                return

            df = df[df['nazwa zasobnika'].apply(lambda x: str(x).strip().upper() not in ['TO DO', 'HOLD', 'INSTRUKCJA'])]
            df['rok'] = df['data rozpoczęcia'].dt.year

            result = {}
            for (rok, zasobnik), grupa in df.groupby(['rok', 'nazwa zasobnika']):
                ukończone = grupa['data ukończenia'].notna().sum()
                nieukończone = grupa['data ukończenia'].isna().sum()
                if rok not in result:
                    result[rok] = {}
                result[rok][zasobnik] = [ukończone, nieukończone]

            self.figure.clear()
            ax = self.figure.add_subplot(111)

            bar_width = 0.35
            space_between_items = 0.3
            space_between_years = 1.0

            current_x = 0
            xtick_positions = []
            xtick_labels = []
            year_labels = []

            for rok in sorted(result.keys()):
                zasobniki = sorted(result[rok].keys())
                group_start_x = current_x

                for zasobnik in zasobniki:
                    ukończone, nieukończone = result[rok][zasobnik]

                    x1 = current_x
                    x2 = current_x + bar_width

                    ax.bar(x1, ukończone, width=bar_width, color='green', label='Ukończone' if x1 == 0 else "")
                    ax.bar(x2, nieukończone, width=bar_width, color='red', label='Nieukończone' if x2 == bar_width else "")

                    middle = (x1 + x2 + bar_width) / 2
                    ax.text(middle, -5, zasobnik, ha='center', va='top', rotation=0, fontsize=10)

                    current_x += 2 * bar_width + space_between_items

                group_middle = (group_start_x + current_x - space_between_items) / 2
                year_labels.append((group_middle, str(rok)))
                current_x += space_between_years

            for xpos, label in year_labels:
                ax.text(xpos, -10, label, ha='center', va='top', fontsize=12, fontweight='bold', transform=ax.transData)

            ax.set_ylabel("Liczba zadań")
            ax.set_title("Zadania ukończone i nieukończone wg zasobnika i roku")
            ax.legend()
            ax.grid(True, axis='y')

            ax.set_xticks([])
            ax.set_xlim(left=0)
            ax.set_ylim(bottom=0)

            self.canvas.draw()
            QMessageBox.information(self, "Gotowe", "✅ Wykres wygenerowany automatycznie.")

        except Exception as e:
            QMessageBox.critical(self, "Błąd", f"❌ Nie udało się przetworzyć pliku:\n{str(e)}")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ZadaniaWykresGrupowany()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 18.08.2025

In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure


# -----------------------------
# Helpers
# -----------------------------

def smart_parse_date(s: pd.Series) -> pd.Series:
    """
    Parse a pandas Series to datetime handling Excel 'Ogólny' strings like 'dd.mm.rrrr'
    and variations with time. Coerce errors to NaT.
    """
    if s is None:
        return pd.Series(pd.NaT, index=[])  # empty
    if not isinstance(s, pd.Series):
        s = pd.Series(s)

    # Already datetime?
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')

    # Try pandas generic with dayfirst
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)

    # For stubborn cases like '12.01.2025 14:55' as string with weird spaces
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        dt2 = pd.to_datetime(s[mask].astype(str).str.strip(), format='%d.%m.%Y', errors='coerce')
        # maybe has time too
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(s[mask][still].astype(str).str.strip(), format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2

    return dt


def to_year_quarter(dt: pd.Series) -> pd.Series:
    dt = pd.to_datetime(dt, errors='coerce')
    year = dt.dt.year
    quarter = dt.dt.quarter
    # Polish label: KWARTAŁ1, KWARTAŁ2, ...
    qlabel = 'KWARTAŁ'
    return year.astype('Int64').astype(str) + ' | ' + qlabel + quarter.astype('Int64').astype(str)


# -----------------------------
# Main Widget
# -----------------------------

class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania: utworzone vs zakończone – osoby × kwartały')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        # Top controls
        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        # Column mapping
        map_group = QGroupBox('Mapowanie kolumn (auto-wykrywanie po polskich nazwach)')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba (np. "Przypisane do")', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        # Filters
        filt_group = QGroupBox('Filtry')
        filt_layout = QHBoxLayout(filt_group)

        # People filter (multi)
        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        # Years filter (multi)
        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        # Options
        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie (zamiast nakładania)')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)
        filt_layout.addWidget(opt_box)

        root.addWidget(filt_group)

        # Plot
        self.fig = Figure(figsize=(14, 6), constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)

        # Status
        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    # ------------------------- UI Actions -------------------------
    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return

        # Normalize column names (strip, keep original separately)
        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)

        # Fill mapping combos
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)

        # Heuristics for default mapping (Polish headers from Trello/Planner-like CSV)
        self.columns_map['person'] = self.pick_col(['Przypisane do', 'Osoba', 'Właściciel', 'Assignee'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia', 'Utworzenia', 'Utworzona', 'Date Created'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia', 'Data ukończenia', 'Ukończone', 'Date Completed'])

        # Set combos to guessed mapping
        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])

        # Populate filters
        self.populate_filters()

        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')

        # People
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)

        # Years from either created or completed
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    # ------------------------- Plotting -------------------------
    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return

        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])

        sel_people = self.get_selected(self.people_list)
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()

        if sel_people:
            df = df[df['__person'].isin(sel_people)]

        # Build buckets per person × quarter, for both created and completed counts
        created = df.dropna(subset=['__created']).copy()
        completed = df.dropna(subset=['__completed']).copy()

        if sel_years:
            created = created[created['__created'].dt.year.isin(sel_years)]
            completed = completed[completed['__completed'].dt.year.isin(sel_years)]

        created['bucket'] = to_year_quarter(created['__created'])
        completed['bucket'] = to_year_quarter(completed['__completed'])

        # Aggregate
        g_created = created.groupby(['__person', 'bucket']).size().rename('Utworzone').reset_index()
        g_completed = completed.groupby(['__person', 'bucket']).size().rename('Zakończone').reset_index()

        # Union of buckets for alignment
        all_keys = pd.MultiIndex.from_frame(pd.concat([
            g_created[['__person', 'bucket']], g_completed[['__person', 'bucket']]
        ], ignore_index=True).drop_duplicates()).tolist()

        if not all_keys:
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych dla wybranych filtrów', ha='center', va='center')
            self.canvas.draw()
            return

        # Build matrix
        mat = []
        persons = sorted(set([k[0] for k in all_keys]))
        # Keep natural quarter order by year|quarter
        buckets = sorted(set([k[1] for k in all_keys]), key=lambda x: (int(x.split('|')[0].strip()), int(x.split('KWARTAŁ')[1].strip())))

        pivot_created = g_created.pivot(index='bucket', columns='__person', values='Utworzone').reindex(buckets).fillna(0)
        pivot_completed = g_completed.pivot(index='bucket', columns='__person', values='Zakończone').reindex(buckets).fillna(0)

        # Plot
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6

        x = np.arange(len(buckets))
        # For each person, offset groups
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2

        for pi, person in enumerate(persons):
            cvals = pivot_created.get(person, pd.Series(0, index=buckets)).values
            fvals = pivot_completed.get(person, pd.Series(0, index=buckets)).values

            if self.chk_side_by_side.isChecked():
                # created and completed side-by-side within person slot
                xpos = x + offset_start + pi*bar_width
                ax.bar(xpos - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                ax.bar(xpos + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
                # annotate
                for xx, v in zip(xpos - bar_width/2, cvals):
                    if v:
                        ax.text(xx, v + 0.3, int(v), ha='center', va='bottom', fontsize=8, rotation=0)
                for xx, v in zip(xpos + bar_width/2, fvals):
                    if v:
                        ax.text(xx, v + 0.3, int(v), ha='center', va='bottom', fontsize=8, rotation=0)
            else:
                xpos = x + offset_start + pi*bar_width
                b1 = ax.bar(xpos, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                b2 = ax.bar(xpos, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
                # annotations on top
                for rect, v in zip(b2, (cvals+fvals)):
                    if v:
                        ax.text(rect.get_x() + rect.get_width()/2, rect.get_height()+rect.get_y()+0.3, int(v), ha='center', va='bottom', fontsize=8)

        ax.set_title('ZADANIE UTWORZONE DO ZAKOŃCZONE – osoby × kwartały')
        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        ax.set_xticklabels(buckets, rotation=45, ha='right')
        ax.legend(loc='upper left', bbox_to_anchor=(1.01, 1.0))
        ax.grid(axis='y', linestyle='--', alpha=0.4)
        self.fig.tight_layout()
        self.canvas.draw()

    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150)
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    # ------------------------- Utils -------------------------
    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        # exact match
        for c in candidates:
            if c in cols:
                return c
        # case-insensitive contains
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        # fallback: choose first datetime-like
        for c in cols:
            if any(k in c.lower() for k in ['zako', 'uko', 'end']) and 'completed' in (" "+c.lower()+" "):
                return c
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        dt2 = pd.to_datetime(s[mask].astype(str).str.strip(), format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(s[mask][still].astype(str).str.strip(), format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def to_year_quarter(dt: pd.Series) -> pd.Series:
    dt = pd.to_datetime(dt, errors='coerce')
    year = dt.dt.year
    quarter = dt.dt.quarter
    return year.astype('Int64').astype(str) + ' | KWARTAŁ' + quarter.astype('Int64').astype(str)


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania: utworzone vs zakończone – osoby × kwartały')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)
        filt_layout.addWidget(opt_box)

        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return
        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)
        self.columns_map['person'] = self.pick_col(['Przypisane do', 'Osoba'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia'])
        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])
        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return
        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])
        sel_people = self.get_selected(self.people_list)
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_people:
            df = df[df['__person'].isin(sel_people)]
        created = df.dropna(subset=['__created']).copy()
        completed = df.dropna(subset=['__completed']).copy()
        if sel_years:
            created = created[created['__created'].dt.year.isin(sel_years)]
            completed = completed[completed['__completed'].dt.year.isin(sel_years)]
        created['bucket'] = to_year_quarter(created['__created'])
        completed['bucket'] = to_year_quarter(completed['__completed'])
        g_created = created.groupby(['__person', 'bucket']).size().rename('Utworzone').reset_index()
        g_completed = completed.groupby(['__person', 'bucket']).size().rename('Zakończone').reset_index()
        all_keys = pd.MultiIndex.from_frame(pd.concat([
            g_created[['__person', 'bucket']], g_completed[['__person', 'bucket']]
        ], ignore_index=True).drop_duplicates()).tolist()
        if not all_keys:
            self.fig.clear()
            ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw()
            return
        persons = sorted(set([k[0] for k in all_keys]))
        buckets = sorted(set([k[1] for k in all_keys]), key=lambda x: (int(x.split('|')[0].strip()), int(x.split('KWARTAŁ')[1].strip())))
        pivot_created = g_created.pivot(index='bucket', columns='__person', values='Utworzone').reindex(buckets).fillna(0)
        pivot_completed = g_completed.pivot(index='bucket', columns='__person', values='Zakończone').reindex(buckets).fillna(0)
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2
        for pi, person in enumerate(persons):
            cvals = pivot_created.get(person, pd.Series(0, index=buckets)).values
            fvals = pivot_completed.get(person, pd.Series(0, index=buckets)).values
            if self.chk_side_by_side.isChecked():
                xpos = x + offset_start + pi*bar_width
                ax.bar(xpos - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                ax.bar(xpos + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
            else:
                xpos = x + offset_start + pi*bar_width
                ax.bar(xpos, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                ax.bar(xpos, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
        ax.set_title('ZADANIE UTWORZONE DO ZAKOŃCZONE')
        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)
        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            bottom_pad = 0.10 + 0.03 * max(rows - 1, 0)
            self.fig.subplots_adjust(bottom=bottom_pad)
        self.fig.tight_layout()
        self.canvas.draw()

    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_29988\681617132.py:280: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=bottom_pad)
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_29988\681617132.py:281: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        dt2 = pd.to_datetime(s[mask].astype(str).str.strip(), format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(s[mask][still].astype(str).str.strip(), format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def to_year_quarter(dt: pd.Series) -> pd.Series:
    dt = pd.to_datetime(dt, errors='coerce')
    year = dt.dt.year
    quarter = dt.dt.quarter
    return year.astype('Int64').astype(str) + ' | KWARTAŁ' + quarter.astype('Int64').astype(str)


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania: utworzone vs zakończone – osoby × kwartały')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)
        filt_layout.addWidget(opt_box)

        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return
        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)
        self.columns_map['person'] = self.pick_col(['Przypisane do', 'Osoba'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia'])
        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])
        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return
        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])
        sel_people = self.get_selected(self.people_list)
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_people:
            df = df[df['__person'].isin(sel_people)]
        created = df.dropna(subset=['__created']).copy()
        completed = df.dropna(subset=['__completed']).copy()
        if sel_years:
            created = created[created['__created'].dt.year.isin(sel_years)]
            completed = completed[completed['__completed'].dt.year.isin(sel_years)]
        created['bucket'] = to_year_quarter(created['__created'])
        completed['bucket'] = to_year_quarter(completed['__completed'])
        g_created = created.groupby(['__person', 'bucket']).size().rename('Utworzone').reset_index()
        g_completed = completed.groupby(['__person', 'bucket']).size().rename('Zakończone').reset_index()
        all_keys = pd.MultiIndex.from_frame(pd.concat([
            g_created[['__person', 'bucket']], g_completed[['__person', 'bucket']]
        ], ignore_index=True).drop_duplicates()).tolist()
        if not all_keys:
            self.fig.clear()
            ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw()
            return
        persons = sorted(set([k[0] for k in all_keys]))
        buckets = sorted(set([k[1] for k in all_keys]), key=lambda x: (int(x.split('|')[0].strip()), int(x.split('KWARTAŁ')[1].strip())))
        pivot_created = g_created.pivot(index='bucket', columns='__person', values='Utworzone').reindex(buckets).fillna(0)
        pivot_completed = g_completed.pivot(index='bucket', columns='__person', values='Zakończone').reindex(buckets).fillna(0)
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2
        for pi, person in enumerate(persons):
            cvals = pivot_created.get(person, pd.Series(0, index=buckets)).values
            fvals = pivot_completed.get(person, pd.Series(0, index=buckets)).values
            if self.chk_side_by_side.isChecked():
                xpos = x + offset_start + pi*bar_width
                bars1 = ax.bar(xpos - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                bars2 = ax.bar(xpos + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
            else:
                xpos = x + offset_start + pi*bar_width
                bars1 = ax.bar(xpos, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                bars2 = ax.bar(xpos, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
            # add person label on top group center
            mid_x = np.mean([b.get_x() + b.get_width()/2 for b in bars1] + [b.get_x() + b.get_width()/2 for b in bars2])
            ax.text(mid_x, ax.get_ylim()[1]*1.02, person, ha='center', va='bottom', fontsize=10, rotation=0)
        ax.set_title('ZADANIE UTWORZONE DO ZAKOŃCZONE')
        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)
        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            bottom_pad = 0.10 + 0.03 * max(rows - 1, 0)
            self.fig.subplots_adjust(bottom=bottom_pad)
        self.fig.tight_layout()
        self.canvas.draw()

    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_29988\3462022564.py:283: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=bottom_pad)
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_29988\3462022564.py:284: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        dt2 = pd.to_datetime(s[mask].astype(str).str.strip(), format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(s[mask][still].astype(str).str.strip(), format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def to_year_quarter(dt: pd.Series) -> pd.Series:
    dt = pd.to_datetime(dt, errors='coerce')
    year = dt.dt.year
    quarter = dt.dt.quarter
    return year.astype('Int64').astype(str) + ' | KWARTAŁ' + quarter.astype('Int64').astype(str)


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania: utworzone vs zakończone – osoby × kwartały')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)
        filt_layout.addWidget(opt_box)

        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return
        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)
        self.columns_map['person'] = self.pick_col(['Przypisane do', 'Osoba'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia'])
        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])
        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return
        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])
        sel_people = self.get_selected(self.people_list)
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_people:
            df = df[df['__person'].isin(sel_people)]
        created = df.dropna(subset=['__created']).copy()
        completed = df.dropna(subset=['__completed']).copy()
        if sel_years:
            created = created[created['__created'].dt.year.isin(sel_years)]
            completed = completed[completed['__completed'].dt.year.isin(sel_years)]
        created['bucket'] = to_year_quarter(created['__created'])
        completed['bucket'] = to_year_quarter(completed['__completed'])
        g_created = created.groupby(['__person', 'bucket']).size().rename('Utworzone').reset_index()
        g_completed = completed.groupby(['__person', 'bucket']).size().rename('Zakończone').reset_index()
        all_keys = pd.MultiIndex.from_frame(pd.concat([
            g_created[['__person', 'bucket']], g_completed[['__person', 'bucket']]
        ], ignore_index=True).drop_duplicates()).tolist()
        if not all_keys:
            self.fig.clear()
            ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw()
            return
        persons = sorted(set([k[0] for k in all_keys]))
        buckets = sorted(set([k[1] for k in all_keys]), key=lambda x: (int(x.split('|')[0].strip()), int(x.split('KWARTAŁ')[1].strip())))
        pivot_created = g_created.pivot(index='bucket', columns='__person', values='Utworzone').reindex(buckets).fillna(0)
        pivot_completed = g_completed.pivot(index='bucket', columns='__person', values='Zakończone').reindex(buckets).fillna(0)
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2
        for pi, person in enumerate(persons):
            cvals = pivot_created.get(person, pd.Series(0, index=buckets)).values
            fvals = pivot_completed.get(person, pd.Series(0, index=buckets)).values
            if self.chk_side_by_side.isChecked():
                xpos = x + offset_start + pi*bar_width
                bars1 = ax.bar(xpos - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                bars2 = ax.bar(xpos + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
            else:
                xpos = x + offset_start + pi*bar_width
                bars1 = ax.bar(xpos, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                bars2 = ax.bar(xpos, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
            # add person label above each individual bar group
            for bar in list(bars1) + list(bars2):
                bx = bar.get_x() + bar.get_width()/2
                ax.text(bx, bar.get_height() + bar.get_y() + 0.5, person,
                        ha='center', va='bottom', fontsize=9, rotation=90)
        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)
        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            bottom_pad = 0.10 + 0.03 * max(rows - 1, 0)
            self.fig.subplots_adjust(bottom=bottom_pad)
        self.fig.tight_layout()
        self.canvas.draw()

    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_19100\3697020877.py:284: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=bottom_pad)
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_19100\3697020877.py:285: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        dt2 = pd.to_datetime(s[mask].astype(str).str.strip(), format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(s[mask][still].astype(str).str.strip(), format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def to_year_quarter(dt: pd.Series) -> pd.Series:
    dt = pd.to_datetime(dt, errors='coerce')
    year = dt.dt.year
    quarter = dt.dt.quarter
    return year.astype('Int64').astype(str) + ' | KWARTAŁ' + quarter.astype('Int64').astype(str)


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania: utworzone vs zakończone – osoby × kwartały')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)
        filt_layout.addWidget(opt_box)

        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return
        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)
        self.columns_map['person'] = self.pick_col(['Przypisane do', 'Osoba'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia'])
        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])
        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return
        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])
        sel_people = self.get_selected(self.people_list)
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_people:
            df = df[df['__person'].isin(sel_people)]
        created = df.dropna(subset=['__created']).copy()
        completed = df.dropna(subset=['__completed']).copy()
        if sel_years:
            created = created[created['__created'].dt.year.isin(sel_years)]
            completed = completed[completed['__completed'].dt.year.isin(sel_years)]
        created['bucket'] = to_year_quarter(created['__created'])
        completed['bucket'] = to_year_quarter(completed['__completed'])
        g_created = created.groupby(['__person', 'bucket']).size().rename('Utworzone').reset_index()
        g_completed = completed.groupby(['__person', 'bucket']).size().rename('Zakończone').reset_index()
        all_keys = pd.MultiIndex.from_frame(pd.concat([
            g_created[['__person', 'bucket']], g_completed[['__person', 'bucket']]
        ], ignore_index=True).drop_duplicates()).tolist()
        if not all_keys:
            self.fig.clear()
            ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw()
            return
        persons = sorted(set([k[0] for k in all_keys]))
        buckets = sorted(set([k[1] for k in all_keys]), key=lambda x: (int(x.split('|')[0].strip()), int(x.split('KWARTAŁ')[1].strip())))
        pivot_created = g_created.pivot(index='bucket', columns='__person', values='Utworzone').reindex(buckets).fillna(0)
        pivot_completed = g_completed.pivot(index='bucket', columns='__person', values='Zakończone').reindex(buckets).fillna(0)
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2
        for pi, person in enumerate(persons):
            cvals = pivot_created.get(person, pd.Series(0, index=buckets)).values
            fvals = pivot_completed.get(person, pd.Series(0, index=buckets)).values
            if self.chk_side_by_side.isChecked():
                xpos = x + offset_start + pi*bar_width
                bars1 = ax.bar(xpos - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                bars2 = ax.bar(xpos + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
                group_bars = list(bars1) + list(bars2)
            else:
                xpos = x + offset_start + pi*bar_width
                bars1 = ax.bar(xpos, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                bars2 = ax.bar(xpos, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
                group_bars = list(bars1)
            # add single person label once above group center
            if group_bars:
                mid_x = np.mean([b.get_x() + b.get_width()/2 for b in group_bars])
                max_h = max([b.get_height() + b.get_y() for b in group_bars])
                ax.text(mid_x, max_h + 0.5, person,
                        ha='center', va='bottom', fontsize=9, rotation=0, fontweight='bold')
        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)
        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            bottom_pad = 0.10 + 0.03 * max(rows - 1, 0)
            self.fig.subplots_adjust(bottom=bottom_pad)
        self.fig.tight_layout()
        self.canvas.draw()

    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_19100\3391229591.py:287: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=bottom_pad)
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_19100\3391229591.py:288: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        dt2 = pd.to_datetime(s[mask].astype(str).str.strip(), format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(s[mask][still].astype(str).str.strip(), format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def to_year_quarter(dt: pd.Series) -> pd.Series:
    dt = pd.to_datetime(dt, errors='coerce')
    year = dt.dt.year
    quarter = dt.dt.quarter
    return year.astype('Int64').astype(str) + ' | KWARTAŁ' + quarter.astype('Int64').astype(str)


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania: utworzone vs zakończone – osoby × kwartały')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)
        
                # Etykiety nad kolumnami – mogą spowalniać przy wielu słupkach
        self.chk_labels = QCheckBox('Etykiety nad kolumnami (wolniej)')
        self.chk_labels.setChecked(False)
        self.chk_labels.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_labels)

        
        
        filt_layout.addWidget(opt_box)

        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return
        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)
        self.columns_map['person'] = self.pick_col(['Przypisane do', 'Osoba'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia'])
        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])
        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return

        # --- Szybkie przygotowanie danych ---
        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        # wiele osób w jednej komórce -> rozbij na osobne rekordy (bez duplikowania liczeń w czasie)
        df = df.assign(__person=df['__person'].str.split(r'[;,]')).explode('__person')
        df['__person'] = df['__person'].str.strip()

        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])

        # Filtr po osobach
        sel_people = self.get_selected(self.people_list)
        if sel_people:
            df = df[df['__person'].isin(sel_people)]

        # Zbuduj zakres kwartałów (bucketów)
        min_dt = pd.to_datetime(df['__created'].min())
        max_dt = pd.to_datetime(pd.concat([df['__created'], df['__completed']]).max())
        if pd.isna(min_dt) or pd.isna(max_dt):
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw(); return

        buckets_periods = pd.period_range(min_dt.to_period('Q'), max_dt.to_period('Q'), freq='Q')
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_years:
            buckets_periods = pd.PeriodIndex([p for p in buckets_periods if p.year in sel_years], freq='Q')
        if len(buckets_periods) == 0:
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych po filtrach', ha='center', va='center')
            self.canvas.draw(); return

        buckets = [f"{p.year} | KWARTAŁ{p.quarter}" for p in buckets_periods]
        persons = sorted(df['__person'].dropna().unique().tolist())

        # Macierze wyników: wiersze=bucket, kolumny=osoby
        created_counts   = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)  # NIEUKOŃCZONE (otwarte)
        completed_counts = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)  # ZAKOŃCZONE

        # Zakończone – szybkie grupowanie po kwartale zakończenia
        completed_p = df['__completed'].dt.to_period('Q')
        mask_done = ~completed_p.isna()
        if mask_done.any():
            tmp = df.loc[mask_done, ['__person']].copy()
            tmp['bucket'] = completed_p[mask_done]
            tmp['bucket'] = tmp['bucket'].astype(str).str.replace('Q','KWARTAŁ', regex=False).str.replace(' ', ' | ', regex=False)
            done_counts = tmp.groupby(['bucket','__person']).size()
            for (b,p), v in done_counts.items():
                if b in completed_counts.index and p in completed_counts.columns:
                    completed_counts.loc[b, p] = int(v)

        # Nieukończone – licz wektorowo per bucket: created<=bucket & (completed NaT lub > bucket)
        created_p   = df['__created'].dt.to_period('Q').values
        completed_p_arr = df['__completed'].dt.to_period('Q').values
        people_arr  = df['__person'].values

        for bidx, bper in enumerate(buckets_periods):
            le_mask   = (created_p <= bper)
            comp_after = np.array([ (pd.isna(d) or d > bper) for d in completed_p_arr ], dtype=bool)
            open_mask = le_mask & comp_after
            if not open_mask.any():
                continue
            counts = pd.Series(people_arr[open_mask]).value_counts()
            for p, v in counts.items():
                if p in created_counts.columns:
                    created_counts.iloc[bidx, created_counts.columns.get_loc(p)] = int(v)

        # --- Rysowanie ---
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2

        for pi, person in enumerate(persons):
            cvals = created_counts[person].values
            fvals = completed_counts[person].values
            center = x + offset_start + pi*bar_width

            if self.chk_side_by_side.isChecked():
                ax.bar(center - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                ax.bar(center + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
                tops = np.maximum(cvals, fvals)
                centers_for_labels = (center - bar_width/2 + center + bar_width/2)/2
            else:
                ax.bar(center, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                ax.bar(center, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
                tops = cvals + fvals
                centers_for_labels = center

            # Etykiety nad słupkami – tylko jeśli włączone (oszczędza czas)
            if self.chk_labels.isChecked():
                for j in range(len(buckets)):
                    if tops[j] > 0:
                        ax.text(centers_for_labels[j], tops[j] + 0.6, person, ha='center', va='bottom', fontsize=9)

        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)

        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                    loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                    ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))

        self.fig.tight_layout()
        self.canvas.draw()


    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_25644\3528001107.py:339: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_25644\3528001107.py:341: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


# -----------------------------
# Parsowanie dat i etykiety kwartałów
# -----------------------------

def smart_parse_date(s: pd.Series) -> pd.Series:
    """Próbuje sparsować ciągi typu "dd.mm.rrrr" (czasem z godziną) do datetime.
    Błędy -> NaT. Działa też gdy kolumna jest już datetime.
    """
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')

    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        vals = s[mask].astype(str).str.strip()
        dt2 = pd.to_datetime(vals, format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(vals[still], format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def quarter_label(p: pd.Period) -> str:
    return f"{p.year} | KWARTAŁ{p.quarter}"


# -----------------------------
# Główne GUI
# -----------------------------

class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania – osoby × kwartały: Nieukończone vs Zakończone')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',    # Przypisane do
            'created': '',   # Data utworzenia
            'completed': ''  # Data zakończenia
        }

        root = QVBoxLayout(self)

        # Górny pasek
        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        # Mapowanie kolumn
        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba (Przypisane do)', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        # Filtry i opcje
        filt_group = QGroupBox('Filtry i opcje')
        filt_layout = QHBoxLayout(filt_group)

        # Osoby
        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        # Lata
        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        # Opcje
        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie (vs łączone)')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)

        self.chk_labels = QCheckBox('Etykiety z nazwą osoby nad kolumną (wolniej)')
        self.chk_labels.setChecked(False)
        self.chk_labels.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_labels)

        filt_layout.addWidget(opt_box)
        root.addWidget(filt_group)

        # Wykres + toolbar
        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    # ------------------------- I/O -------------------------
    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return

        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)

        # Heurystyki nazewnictwa PL
        self.columns_map['person'] = self.pick_col(['Przypisane do','Osoba','Assignee'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia','Utworzenia','Utworzona'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia','Data ukończenia','Ukończone'])

        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])

        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')

        # Osoby
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)

        # Lata — z utworzenia i zakończenia
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    # ------------------------- AGREGACJA + WYKRES -------------------------
    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return

        # --- Szybkie przygotowanie danych ---
        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        # Wiele osób w jednej komórce: "Ala, Ola" → dwa rekordy
        df = df.assign(__person=df['__person'].str.split(r'[;,]')).explode('__person')
        df['__person'] = df['__person'].str.strip()

        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])

        # Filtr po osobach
        sel_people = self.get_selected(self.people_list)
        if sel_people:
            df = df[df['__person'].isin(sel_people)]

        # Zakres kwartałów (globalnie), potem filtr po latach
        min_dt = pd.to_datetime(df['__created'].min())
        max_dt = pd.to_datetime(pd.concat([df['__created'], df['__completed']]).max())
        if pd.isna(min_dt) or pd.isna(max_dt):
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw(); return

        buckets_periods = pd.period_range(min_dt.to_period('Q'), max_dt.to_period('Q'), freq='Q')
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_years:
            buckets_periods = pd.PeriodIndex([p for p in buckets_periods if p.year in sel_years], freq='Q')
        if len(buckets_periods) == 0:
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych po filtrach', ha='center', va='center')
            self.canvas.draw(); return

        buckets = [quarter_label(p) for p in buckets_periods]
        persons = sorted(df['__person'].dropna().unique().tolist())

        # Macierze wyników: wiersze=bucket, kolumny=osoby
        open_counts   = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)  # NIEUKOŃCZONE (otwarte)
        done_counts   = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)  # ZAKOŃCZONE

        # Zakończone – szybkie grupowanie po kwartale zakończenia
        completed_p = df['__completed'].dt.to_period('Q')
        mask_done = ~completed_p.isna()
        if mask_done.any():
            tmp = df.loc[mask_done, ['__person']].copy()
            tmp['bucket'] = completed_p[mask_done]
            tmp['bucket'] = tmp['bucket'].astype(str).str.replace('Q','KWARTAŁ', regex=False).str.replace(' ', ' | ', regex=False)
            grp = tmp.groupby(['bucket','__person']).size()
            for (b,p), v in grp.items():
                if b in done_counts.index and p in done_counts.columns:
                    done_counts.loc[b, p] = int(v)

        # Nieukończone – per bucket: created≤bucket i (completed NaT lub > bucket)
        created_p = df['__created'].dt.to_period('Q').values
        completed_p_arr = df['__completed'].dt.to_period('Q').values
        people_arr = df['__person'].values
        for bidx, bper in enumerate(buckets_periods):
            le_mask = (created_p <= bper)
            comp_after = np.array([(pd.isna(d) or d > bper) for d in completed_p_arr], dtype=bool)
            open_mask = le_mask & comp_after
            if open_mask.any():
                counts = pd.Series(people_arr[open_mask]).value_counts()
                for p, v in counts.items():
                    if p in open_counts.columns:
                        open_counts.iloc[bidx, open_counts.columns.get_loc(p)] = int(v)

        # --- Rysowanie ---
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2

        for pi, person in enumerate(persons):
            cvals = open_counts[person].values
            fvals = done_counts[person].values
            center = x + offset_start + pi*bar_width

            if self.chk_side_by_side.isChecked():
                ax.bar(center - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Nieukończone")
                ax.bar(center + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
                tops = np.maximum(cvals, fvals)
                centers_for_labels = (center - bar_width/2 + center + bar_width/2)/2
            else:
                ax.bar(center, cvals, bar_width*0.95, label=f"{person} – Nieukończone")
                ax.bar(center, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
                tops = cvals + fvals
                centers_for_labels = center

            if self.chk_labels.isChecked():
                for j in range(len(buckets)):
                    if tops[j] > 0:
                        ax.text(centers_for_labels[j], tops[j] + 0.6, person, ha='center', va='bottom', fontsize=9)

        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)

        # Legenda responsywna (na dole)
        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))

        self.fig.tight_layout()
        self.canvas.draw()

    # ------------------------- Eksport -------------------------
    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    # ------------------------- Pomocnicze -------------------------
    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_31620\28498709.py:360: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_31620\28498709.py:362: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')

    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        vals = s[mask].astype(str).str.strip()
        dt2 = pd.to_datetime(vals, format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(vals[still], format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def quarter_label(p: pd.Period) -> str:
    return f"{p.year} | KWARTAŁ{p.quarter}"


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania – osoby × kwartały: Nieukończone vs Zakończone')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba (Przypisane do)', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry i opcje')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie (vs łączone)')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)

        self.chk_labels = QCheckBox('Etykiety z nazwą osoby nad kolumną (wolniej)')
        self.chk_labels.setChecked(False)
        self.chk_labels.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_labels)

        filt_layout.addWidget(opt_box)
        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return

        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)

        self.columns_map['person'] = self.pick_col(['Przypisane do','Osoba','Assignee'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia','Utworzenia','Utworzona'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia','Data ukończenia','Ukończone'])

        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])

        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')

        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)

        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return

        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)

        # obsługa wielu osób – rozdziel też "i" jako łącznik
        pat = r'[;,/|&+]|\bi\b'
        df = df.assign(
            __person=df['__person']
                .astype(str)
                .str.replace(r'\s+', ' ', regex=True)
                .str.split(pat)
        ).explode('__person')
        df['__person'] = df['__person'].str.strip().replace('', np.nan)
        df = df.dropna(subset=['__person'])

        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])

        sel_people = self.get_selected(self.people_list)
        if sel_people:
            df = df[df['__person'].isin(sel_people)]

        min_dt = pd.to_datetime(df[['__created','__completed']].min().min())
        max_dt = pd.to_datetime(df[['__created','__completed']].max().max())
        if pd.isna(min_dt) or pd.isna(max_dt):
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw(); return

        buckets_periods = pd.period_range(min_dt.to_period('Q'), max_dt.to_period('Q'), freq='Q')
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_years:
            buckets_periods = pd.PeriodIndex([p for p in buckets_periods if p.year in sel_years], freq='Q')
        if len(buckets_periods) == 0:
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych po filtrach', ha='center', va='center')
            self.canvas.draw(); return

        buckets = [quarter_label(p) for p in buckets_periods]
        persons = sorted(df['__person'].dropna().unique().tolist())

        open_counts = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)
        done_counts = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)

        # zliczanie zakończonych
        completed_p = df['__completed'].dt.to_period('Q')
        mask_done = ~completed_p.isna()
        if mask_done.any():
            tmp = df.loc[mask_done, ['__person']].copy()
            tmp['bucket'] = completed_p[mask_done].apply(quarter_label)
            grp = tmp.groupby(['bucket','__person']).size()
            for (b,p), v in grp.items():
                if b in done_counts.index and p in done_counts.columns:
                    done_counts.loc[b, p] += int(v)

        # zliczanie nieukończonych (od utworzenia do kwartału zakończenia)
        created_p = df['__created'].dt.to_period('Q').values
        completed_p_arr = df['__completed'].dt.to_period('Q').values
        people_arr = df['__person'].values
        for bidx, bper in enumerate(buckets_periods):
            for ci, cper in enumerate(created_p):
                if pd.isna(cper):
                    continue
                comp_per = completed_p_arr[ci]
                if cper <= bper and (pd.isna(comp_per) or comp_per > bper):
                    person = people_arr[ci]
                    if person in open_counts.columns:
                        open_counts.loc[quarter_label(bper), person] += 1

        # rysowanie wykresu
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2

        for pi, person in enumerate(persons):
            cvals = open_counts[person].values
            fvals = done_counts[person].values
            center = x + offset_start + pi*bar_width

            if self.chk_side_by_side.isChecked():
                ax.bar(center - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Nieukończone")
                ax.bar(center + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
                tops = np.maximum(cvals, fvals)
                centers_for_labels = (center - bar_width/2 + center + bar_width/2)/2
            else:
                ax.bar(center, cvals, bar_width*0.95, label=f"{person} – Nieukończone")
                ax.bar(center, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
                tops = cvals + fvals
                centers_for_labels = center

            if self.chk_labels.isChecked():
                for j in range(len(buckets)):
                    if tops[j] > 0:
                        ax.text(centers_for_labels[j], tops[j] + 0.6, person, ha='center', va='bottom', fontsize=9)

        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)

        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))

        try:
            total_rows = len(df)
            sum_open = int(open_counts.to_numpy().sum())
            sum_done = int(done_counts.to_numpy().sum())
            self.status.setText(
                f"Wiersze po filtrach: {total_rows} | Osoby: {len(persons)} | "
                f"Kwartały: {len(buckets)} | Suma nieukończonych: {sum_open} | Zakończone: {sum_done}"
            )
        except Exception:
            pass

        self.fig.tight_layout()
        self.canvas.draw()

    def export_png(self):
        if self.df is None:
            QMessageBox.warning(self, 'Brak danych', 'Wczytaj najpierw plik XLSX.')
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_25644\3002864604.py:339: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_25644\3002864604.py:352: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## działa 

In [ ]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox
)

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        dt2 = pd.to_datetime(s[mask].astype(str).str.strip(), format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(s[mask][still].astype(str).str.strip(), format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def to_year_quarter(dt: pd.Series) -> pd.Series:
    dt = pd.to_datetime(dt, errors='coerce')
    year = dt.dt.year
    quarter = dt.dt.quarter
    return year.astype('Int64').astype(str) + ' | KWARTAŁ' + quarter.astype('Int64').astype(str)


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania: utworzone vs zakończone – osoby × kwartały')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)
        filt_layout.addWidget(opt_box)

        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return
        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)
        self.columns_map['person'] = self.pick_col(['Przypisane do', 'Osoba'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia'])
        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])
        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        people = []
        if person_col in self.df.columns:
            people = sorted(pd.Series(self.df[person_col].dropna().astype(str).unique()).tolist())
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)
        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return
        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)
        # Wsparcie dla wielu osób w polu ("Ala, Ola" -> dwie kopie wiersza):
        df = df.assign(__person=df['__person'].str.split(r'[;,]')).explode('__person')
        df['__person'] = df['__person'].str.strip()

        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])
        sel_people = self.get_selected(self.people_list)
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_people:
            df = df[df['__person'].isin(sel_people)]
        created = df.dropna(subset=['__created']).copy()
        completed = df.dropna(subset=['__completed']).copy()
        if sel_years:
            created = created[created['__created'].dt.year.isin(sel_years)]
            completed = completed[completed['__completed'].dt.year.isin(sel_years)]
        created['bucket'] = to_year_quarter(created['__created'])
        completed['bucket'] = to_year_quarter(completed['__completed'])
        g_created = created.groupby(['__person', 'bucket']).size().rename('Utworzone').reset_index()
        g_completed = completed.groupby(['__person', 'bucket']).size().rename('Zakończone').reset_index()
        all_keys = pd.MultiIndex.from_frame(pd.concat([
            g_created[['__person', 'bucket']], g_completed[['__person', 'bucket']]
        ], ignore_index=True).drop_duplicates()).tolist()
        if not all_keys:
            self.fig.clear()
            ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw()
            return
        persons = sorted(set([k[0] for k in all_keys]))
        buckets = sorted(set([k[1] for k in all_keys]), key=lambda x: (int(x.split('|')[0].strip()), int(x.split('KWARTAŁ')[1].strip())))
        pivot_created = g_created.pivot(index='bucket', columns='__person', values='Utworzone').reindex(buckets).fillna(0)
        pivot_completed = g_completed.pivot(index='bucket', columns='__person', values='Zakończone').reindex(buckets).fillna(0)
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2

        # Rysowanie + etykiety bez duplikacji: 1 etykieta na kwartal/osoba
        for pi, person in enumerate(persons):
            cvals = pivot_created.get(person, pd.Series(0, index=buckets)).values
            fvals = pivot_completed.get(person, pd.Series(0, index=buckets)).values
            center = x + offset_start + pi*bar_width

            if self.chk_side_by_side.isChecked():
                ax.bar(center - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                ax.bar(center + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
                tops = np.maximum(cvals, fvals)
            else:
                ax.bar(center, cvals, bar_width*0.95, label=f"{person} – Utworzone")
                ax.bar(center, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
                tops = cvals + fvals

            for j in range(len(buckets)):
                if tops[j] > 0:
                    ax.text(center[j], tops[j] + 0.6, person, ha='center', va='bottom', fontsize=9)

        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)
        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            bottom_pad = 0.10 + 0.03 * max(rows - 1, 0)
            self.fig.subplots_adjust(bottom=bottom_pad)
        self.fig.tight_layout()
        self.canvas.draw()

    def export_png(self):
        if self.df is None:
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_21872\1213875219.py:292: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=bottom_pad)
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_21872\1213875219.py:293: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List

from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QPushButton,
    QFileDialog, QMessageBox, QComboBox, QListWidget, QListWidgetItem,
    QGroupBox, QFormLayout, QCheckBox, QPlainTextEdit
)


from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    if np.issubdtype(s.dtype, np.datetime64):
        return pd.to_datetime(s, errors='coerce')

    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        vals = s[mask].astype(str).str.strip()
        dt2 = pd.to_datetime(vals, format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(vals[still], format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt


def quarter_label(p: pd.Period) -> str:
    return f"{p.year} | KWARTAŁ{p.quarter}"


class TasksDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('📊 Zadania – osoby × kwartały: Nieukończone vs Zakończone')
        self.setMinimumSize(1280, 800)

        self.df: Optional[pd.DataFrame] = None
        self.columns_map: Dict[str, str] = {
            'person': '',
            'created': '',
            'completed': ''
        }

        root = QVBoxLayout(self)

        top = QHBoxLayout()
        self.open_btn = QPushButton('📂 Wczytaj XLSX')
        self.open_btn.clicked.connect(self.on_open)
        top.addWidget(self.open_btn)

        self.refresh_btn = QPushButton('🔄 Odśwież wykres')
        self.refresh_btn.clicked.connect(self.update_plot)
        self.refresh_btn.setEnabled(False)
        top.addWidget(self.refresh_btn)

        self.export_btn = QPushButton('💾 Eksportuj wykres (PNG)')
        self.export_btn.clicked.connect(self.export_png)
        self.export_btn.setEnabled(False)
        top.addWidget(self.export_btn)

        top.addStretch(1)
        root.addLayout(top)

        map_group = QGroupBox('Mapowanie kolumn')
        map_form = QFormLayout(map_group)
        self.cmb_person = QComboBox(); self.cmb_created = QComboBox(); self.cmb_completed = QComboBox()
        self.cmb_person.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_created.currentIndexChanged.connect(self.on_mapping_changed)
        self.cmb_completed.currentIndexChanged.connect(self.on_mapping_changed)
        map_form.addRow('👤 Osoba (Przypisane do)', self.cmb_person)
        map_form.addRow('📅 Data utworzenia', self.cmb_created)
        map_form.addRow('✅ Data zakończenia', self.cmb_completed)
        root.addWidget(map_group)

        filt_group = QGroupBox('Filtry i opcje')
        filt_layout = QHBoxLayout(filt_group)

        self.people_list = QListWidget()
        self.people_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.people_list.itemChanged.connect(self.update_plot)
        ppl_box = QGroupBox('Osoby')
        ppl_layout = QVBoxLayout(ppl_box)
        ppl_layout.addWidget(self.people_list)
        filt_layout.addWidget(ppl_box)

        self.years_list = QListWidget()
        self.years_list.setSelectionMode(QListWidget.SelectionMode.MultiSelection)
        self.years_list.itemChanged.connect(self.update_plot)
        year_box = QGroupBox('Lata')
        year_layout = QVBoxLayout(year_box)
        year_layout.addWidget(self.years_list)
        filt_layout.addWidget(year_box)

        opt_box = QGroupBox('Opcje')
        opt_form = QFormLayout(opt_box)
        
        # === DIAGNOSTYKA ===
        self.btn_diag = QPushButton('🔍 Pokaż diagnostykę danych')
        self.btn_diag.clicked.connect(self.show_diagnostics)
        opt_form.addRow(self.btn_diag)
        self.chk_side_by_side = QCheckBox('Słupki obok siebie (vs łączone)')
        self.chk_side_by_side.setChecked(True)
        self.chk_side_by_side.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_side_by_side)

        self.chk_labels = QCheckBox('Etykiety z nazwą osoby nad kolumną (wolniej)')
        self.chk_labels.setChecked(False)
        self.chk_labels.stateChanged.connect(self.update_plot)
        opt_form.addRow(self.chk_labels)

        filt_layout.addWidget(opt_box)
        root.addWidget(filt_group)

        self.fig = Figure(constrained_layout=True)
        self.canvas = FigureCanvas(self.fig)
        root.addWidget(self.canvas, stretch=1)
        self.toolbar = NavigationToolbar(self.canvas, self)
        root.addWidget(self.toolbar)

        self.status = QLabel('Wczytaj plik XLSX…')
        root.addWidget(self.status)

    def on_open(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Wybierz plik Excel', filter='Pliki Excel (*.xlsx *.xls)')
        if not path:
            return
        try:
            self.df = pd.read_excel(path)
        except Exception as e:
            QMessageBox.critical(self, 'Błąd', f'Nie udało się wczytać pliku.\n{e}')
            return

        self.df.columns = [str(c).strip() for c in self.df.columns]
        cols = list(self.df.columns)
        for cmb in (self.cmb_person, self.cmb_created, self.cmb_completed):
            cmb.blockSignals(True)
            cmb.clear(); cmb.addItems(cols)
            cmb.blockSignals(False)

        self.columns_map['person'] = self.pick_col(['Przypisane do','Osoba','Assignee'])
        self.columns_map['created'] = self.pick_col(['Data utworzenia','Utworzenia','Utworzona'])
        self.columns_map['completed'] = self.pick_col(['Data zakończenia','Data ukończenia','Ukończone'])

        def set_cmb(cmb: QComboBox, col: str):
            if col in cols:
                cmb.setCurrentIndex(cols.index(col))
        set_cmb(self.cmb_person, self.columns_map['person'])
        set_cmb(self.cmb_created, self.columns_map['created'])
        set_cmb(self.cmb_completed, self.columns_map['completed'])

        self.populate_filters()
        self.refresh_btn.setEnabled(True)
        self.export_btn.setEnabled(True)
        self.status.setText(f'Wczytano: {path}  |  {len(self.df)} wierszy')
        self.update_plot()

    def on_mapping_changed(self):
        if self.df is None:
            return
        self.columns_map['person'] = self.cmb_person.currentText()
        self.columns_map['created'] = self.cmb_created.currentText()
        self.columns_map['completed'] = self.cmb_completed.currentText()
        self.populate_filters()
        self.update_plot()

    def populate_filters(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')

        # Osoby: rozbij tak samo jak w update_plot (wspólna logika)
        people = []
        if person_col in self.df.columns:
            ser = (self.df[person_col].dropna().astype(str)
                   .str.replace(r'\s+', ' ', regex=True))
            pat = r'[;,/|&+]|\bi\b'
            ser = ser.str.split(pat).explode().str.strip()
            people = sorted([p for p in ser.unique().tolist() if p])
        self.people_list.blockSignals(True)
        self.people_list.clear()
        for p in people:
            it = QListWidgetItem(p)
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.people_list.addItem(it)
        self.people_list.blockSignals(False)

        years = set()
        for col in [created_col, completed_col]:
            if col in self.df.columns:
                dt = smart_parse_date(self.df[col])
                years |= set(dt.dt.year.dropna().astype(int).unique().tolist())
        years = sorted(list(years))
        self.years_list.blockSignals(True)
        self.years_list.clear()
        for y in years:
            it = QListWidgetItem(str(y))
            it.setFlags(it.flags() | Qt.ItemFlag.ItemIsUserCheckable)
            it.setCheckState(Qt.CheckState.Checked)
            self.years_list.addItem(it)
        self.years_list.blockSignals(False)

    def get_selected(self, lw: QListWidget) -> List[str]:
        vals = []
        for i in range(lw.count()):
            it = lw.item(i)
            if it.checkState() == Qt.CheckState.Checked:
                vals.append(it.text())
        return vals

    def update_plot(self):
        if self.df is None:
            return
        person_col = self.columns_map.get('person')
        created_col = self.columns_map.get('created')
        completed_col = self.columns_map.get('completed')
        if any(col not in self.df.columns for col in [person_col, created_col, completed_col]):
            return

        df = self.df.copy()
        df['__person'] = df[person_col].astype(str)

        # obsługa wielu osób – rozdziel też "i" jako łącznik
        pat = r'[;,/|&+]|\bi\b'
        df = df.assign(
            __person=df['__person']
                .astype(str)
                .str.replace(r'\s+', ' ', regex=True)
                .str.split(pat)
        ).explode('__person')
        df['__person'] = df['__person'].str.strip().replace('', np.nan)
        df = df.dropna(subset=['__person'])

        df['__created'] = smart_parse_date(df[created_col])
        df['__completed'] = smart_parse_date(df[completed_col])

        sel_people = self.get_selected(self.people_list)
        if sel_people:
            df = df[df['__person'].isin(sel_people)]

        min_dt = pd.to_datetime(df[['__created','__completed']].min().min())
        max_dt = pd.to_datetime(df[['__created','__completed']].max().max())
        if pd.isna(min_dt) or pd.isna(max_dt):
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center')
            self.canvas.draw(); return

        buckets_periods = pd.period_range(min_dt.to_period('Q'), max_dt.to_period('Q'), freq='Q')
        sel_years = set(map(int, self.get_selected(self.years_list))) if self.years_list.count() else set()
        if sel_years:
            buckets_periods = pd.PeriodIndex([p for p in buckets_periods if p.year in sel_years], freq='Q')
        if len(buckets_periods) == 0:
            self.fig.clear(); ax = self.fig.add_subplot(111)
            ax.text(0.5, 0.5, 'Brak danych po filtrach', ha='center', va='center')
            self.canvas.draw(); return

        buckets = [quarter_label(p) for p in buckets_periods]
        persons = sorted(df['__person'].dropna().unique().tolist())

        open_counts = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)
        done_counts = pd.DataFrame(0, index=buckets, columns=persons, dtype=int)

        # zliczanie zakończonych
        completed_p = df['__completed'].dt.to_period('Q')
        mask_done = ~completed_p.isna()
        if mask_done.any():
            tmp = df.loc[mask_done, ['__person']].copy()
            tmp['bucket'] = completed_p[mask_done].apply(quarter_label)
            grp = tmp.groupby(['bucket','__person']).size()
            for (b,p), v in grp.items():
                if b in done_counts.index and p in done_counts.columns:
                    done_counts.loc[b, p] += int(v)

        # zliczanie nieukończonych (od utworzenia do kwartału zakończenia)
        created_p = df['__created'].dt.to_period('Q').values
        completed_p_arr = df['__completed'].dt.to_period('Q').values
        people_arr = df['__person'].values
        for bidx, bper in enumerate(buckets_periods):
            for ci, cper in enumerate(created_p):
                if pd.isna(cper):
                    continue
                comp_per = completed_p_arr[ci]
                if cper <= bper and (pd.isna(comp_per) or comp_per > bper):
                    person = people_arr[ci]
                    if person in open_counts.columns:
                        open_counts.loc[quarter_label(bper), person] += 1

        # rysowanie wykresu
        self.fig.clear()
        ax = self.fig.add_subplot(111)
        width = 0.35 if self.chk_side_by_side.isChecked() else 0.6
        x = np.arange(len(buckets))
        n_persons = max(1, len(persons))
        bar_width = width / n_persons
        offset_start = -width/2

        for pi, person in enumerate(persons):
            cvals = open_counts[person].values
            fvals = done_counts[person].values
            center = x + offset_start + pi*bar_width

            if self.chk_side_by_side.isChecked():
                ax.bar(center - bar_width/2, cvals, bar_width*0.95, label=f"{person} – Nieukończone")
                ax.bar(center + bar_width/2, fvals, bar_width*0.95, label=f"{person} – Zakończone")
                tops = np.maximum(cvals, fvals)
                centers_for_labels = (center - bar_width/2 + center + bar_width/2)/2
            else:
                ax.bar(center, cvals, bar_width*0.95, label=f"{person} – Nieukończone")
                ax.bar(center, fvals, bar_width*0.95, bottom=cvals, label=f"{person} – Zakończone")
                tops = cvals + fvals
                centers_for_labels = center

            if self.chk_labels.isChecked():
                for j in range(len(buckets)):
                    if tops[j] > 0:
                        ax.text(centers_for_labels[j], tops[j] + 0.6, person, ha='center', va='bottom', fontsize=9)

        ax.set_xlabel('Rok | Kwartał')
        ax.set_ylabel('Liczba zadań')
        ax.set_xticks(x)
        tick_fs = 10 if len(buckets) <= 12 else (9 if len(buckets) <= 20 else 8)
        ax.set_xticklabels(buckets, rotation=45, ha='right', fontsize=tick_fs)
        ax.grid(axis='y', linestyle='--', alpha=0.4)

        handles, labels = ax.get_legend_handles_labels()
        if labels:
            n = len(labels)
            ncol = min(max(3, int(self.fig.get_size_inches()[0] // 2)), n)
            leg_fs = 10 if n <= 8 else (9 if n <= 14 else 8)
            legend = self.fig.legend(handles, labels,
                                     loc='lower center', bbox_to_anchor=(0.5, -0.02),
                                     ncol=ncol, frameon=False, fontsize=leg_fs)
            legend.set_draggable(True)
            rows = int(np.ceil(n / max(ncol,1)))
            self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))

        try:
            total_rows = len(df)
            sum_open = int(open_counts.to_numpy().sum())
            sum_done = int(done_counts.to_numpy().sum())
            self.status.setText(
                f"Wiersze po filtrach: {total_rows} | Osoby: {len(persons)} | "
                f"Kwartały: {len(buckets)} | Suma nieukończonych: {sum_open} | Zakończone: {sum_done}"
            )
        except Exception:
            pass

        self.fig.tight_layout()
        self.canvas.draw()



    def show_diagnostics(self):
        """Proste okno diagnostyczne z podglądem kolumn i pierwszych wierszy"""
        if self.df is None:
            QMessageBox.information(self, "Diagnostyka", "Brak danych – wczytaj plik.")
            return

        try:
            txt = "=== Kolumny ===\n"
            txt += ", ".join(self.df.columns) + "\n\n"

            txt += "=== Pierwsze 5 wierszy ===\n"
            txt += str(self.df.head().to_string()) + "\n\n"

            txt += "=== Liczba wierszy ===\n"
            txt += str(len(self.df))

            dlg = QMessageBox(self)
            dlg.setWindowTitle("Diagnostyka danych")
            dlg.setTextInteractionFlags(Qt.TextInteractionFlag.TextSelectableByMouse)
            dlg.setIcon(QMessageBox.Icon.Information)
            dlg.setText(txt)
            dlg.exec()
        except Exception as e:
            QMessageBox.critical(self, "Błąd diagnostyki", str(e))



    def export_png(self):
        if self.df is None:
            QMessageBox.warning(self, 'Brak danych', 'Wczytaj najpierw plik XLSX.')
            return
        path, _ = QFileDialog.getSaveFileName(self, 'Zapisz wykres jako PNG', filter='PNG (*.png)')
        if not path:
            return
        try:
            self.fig.savefig(path, dpi=150, bbox_inches='tight')
            QMessageBox.information(self, 'Zapisano', f'Wykres zapisany do:\n{path}')
        except Exception as e:
            QMessageBox.critical(self, 'Błąd zapisu', str(e))

    def pick_col(self, candidates: List[str]) -> str:
        if self.df is None:
            return ''
        cols = list(self.df.columns)
        for c in candidates:
            if c in cols:
                return c
        lower_cols = {c.lower(): c for c in cols}
        for c in candidates:
            for lc, orig in lower_cols.items():
                if c.lower() in lc:
                    return orig
        return cols[0] if cols else ''


if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = TasksDashboard()
    w.show()
    sys.exit(app.exec())


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_3164\1748613853.py:350: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  self.fig.subplots_adjust(bottom=0.10 + 0.03 * max(rows - 1, 0))
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_3164\1748613853.py:363: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


SystemExit: 0

c:\Users\mzasiadczuk\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import pandas as pd

APP_TITLE = "Konwerter zadań: Excel → Excel (rok/kwartał/zasobnik)"
APP_GEOMETRY = "720x520"


def best_guess_column(columns, candidates):
    """
    Zwraca najlepsze dopasowanie kolumny na podstawie listy słów-kluczy.
    Dopasowanie ignoruje wielkość liter, polskie znaki, myślniki i spacje.
    """
    import unicodedata

    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s

    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]

    # pełne dopasowanie
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col

    # dopasowanie cząstkowe (najdłuższe trafienie)
    best = None
    best_len = 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best = col
                best_len = len(c)
    return best or (columns[0] if columns else None)


def detect_sheet(path):
    """Zwraca nazwę (pierwszego) arkusza. Jeśli xls/xlsx ma wiele arkuszy, wybiera pierwszy (preferuje nazwy typu: data/dane/zadania)."""
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names if str(s).strip().lower() in {'data', 'dane', 'tasks', 'zadania', 'sheet1', 'arkusz1'}]
        if preferred:
            return preferred[0]
        return xls.sheet_names[0]
    except Exception:
        return None


def aggregate(df, col_person, col_created, col_completed):
    """Buduje tabelę: rok | kwartał | zasobnik | zadania_ukończone | zadania_nieukończone"""
    df = df.copy()

    created = pd.to_datetime(df[col_created], errors='coerce')
    completed = pd.to_datetime(df[col_completed], errors='coerce')

    # osoba jako string; puste -> '—'
    person = df[col_person].astype(str).replace({'nan': '', 'None': ''}).str.strip().replace('', '—')

    # Ukończone: liczymy wg daty ukończenia (rok/kwartał z completed)
    done = (
        pd.DataFrame({
            'rok': completed.dt.year,
            'kwartał': completed.dt.quarter,
            'zasobnik': person,
        })
        .dropna(subset=['rok', 'kwartał'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False)
        .size()
        .rename('zadania_ukończone')
        .reset_index()
    )

    # Nieukończone: brak daty ukończenia; liczymy wg daty utworzenia
    not_done_mask = completed.isna()
    not_done_created = created.where(not_done_mask)

    not_done = (
        pd.DataFrame({
            'rok': not_done_created.dt.year,
            'kwartał': not_done_created.dt.quarter,
            'zasobnik': person.where(not_done_mask),
        })
        .dropna(subset=['rok', 'kwartał', 'zasobnik'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False)
        .size()
        .rename('zadania_nieukończone')
        .reset_index()
    )

    # Łączenie i porządkowanie
    out = pd.merge(done, not_done, on=['rok', 'kwartał', 'zasobnik'], how='outer')
    out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
    out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)
    out = out.sort_values(by=['rok', 'kwartał', 'zasobnik']).reset_index(drop=True)
    return out


class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self.create_widgets()

    def create_widgets(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self)
        frm.pack(fill='both', expand=True)

        # Wybór pliku
        row = ttk.Frame(frm); row.pack(fill='x', **pad)
        ttk.Label(row, text="Plik Excel:").pack(side='left')
        ttk.Entry(row, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row, text="Wybierz...", command=self.pick_file).pack(side='left')

        # Arkusz
        row2 = ttk.Frame(frm); row2.pack(fill='x', **pad)
        ttk.Label(row2, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(row2, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row2, text="Wczytaj", command=self.load_sheet).pack(side='left')

        # Mapowanie kolumn
        box = ttk.LabelFrame(frm, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        r1 = ttk.Frame(box); r1.pack(fill='x', **pad)
        ttk.Label(r1, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r1, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r2 = ttk.Frame(box); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r2, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r3 = ttk.Frame(box); r3.pack(fill='x', **pad)
        ttk.Label(r3, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r3, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # Akcje
        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz...", command=self.convert_and_save).pack(side='right')

        # Status/log
        self.status = tk.Text(frm, height=10)
        self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')

        self.log("Witaj! Wybierz plik Excel i arkusz, potem sprawdź mapowanie kolumn.")

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path:
            return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_var.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self.log(f"Błąd odczytu pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self.log(f"Wczytano arkusz. Kolumny: {self.columns}")

            # comboboksy
            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns

            # auto-detekcja
            person_guess = best_guess_column(self.columns, [
                'zasobnik', 'osoba', 'assignee', 'owner', 'wykonawca', 'przypisane do', 'assigned to', 'user'
            ])
            created_guess = best_guess_column(self.columns, [
                'data utworzenia', 'utworzenia', 'created', 'creation date', 'created at', 'created time', 'start date', 'start'
            ])
            completed_guess = best_guess_column(self.columns, [
                'data ukończenia', 'zakonczenia', 'ukończenia', 'completed', 'done', 'closed', 'finished', 'end date', 'resolution date'
            ])

            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self.log("Sprawdź automatyczne dopasowanie kolumn i w razie potrzeby zmień ręcznie.")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self.log(f"Błąd wczytywania arkusza: {e}")

    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            out = aggregate(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            top = out.head(50)
            self.log(f"Podgląd (pierwsze {len(top)} wierszy):\n{top.to_string(index=False)}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się przygotować podglądu:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            out = aggregate(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            in_path = self.path_var.get().strip()
            base, _ = os.path.splitext(in_path)
            default_out = base + "_AGREGAT_kwartały.xlsx"

            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not path:
                self.log("Anulowano zapis.")
                return

            with pd.ExcelWriter(path, engine="openpyxl") as writer:
                out.to_excel(writer, index=False, sheet_name="Agregat")
            self.log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
            self.log(f"Błąd zapisu: {e}")


def main():
    # DPI-aware na Windows (jeśli dostępne)
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()


if __name__ == "__main__":
    main()


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_3164\1814786085.py:61: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  created = pd.to_datetime(df[col_created], errors='coerce')
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_3164\1814786085.py:62: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  completed = pd.to_datetime(df[col_completed], errors='coerce')


In [6]:
!pip install xlsxwriter

   ---------------------------------------- 0.0/172.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/172.3 kB ? eta -:--:--
   ------ -------------------------------- 30.7/172.3 kB 435.7 kB/s eta 0:00:01
   ------------- ------------------------- 61.4/172.3 kB 550.5 kB/s eta 0:00:01
   ------------------------------- ------ 143.4/172.3 kB 950.9 kB/s eta 0:00:01
   ---------------------------------------- 172.3/172.3 kB 1.0 MB/s eta 0:00:00


In [2]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import xlsxwriter  # opcjonalnie

import pandas as pd

APP_TITLE = "Konwerter zadań: Excel → Excel (rok/kwartał/zasobnik)"
APP_GEOMETRY = "720x520"


def best_guess_column(columns, candidates):
    """
    Zwraca najlepsze dopasowanie kolumny na podstawie listy słów-kluczy.
    Dopasowanie ignoruje wielkość liter, polskie znaki, myślniki i spacje.
    """
    import unicodedata

    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s

    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]

    # pełne dopasowanie
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col

    # dopasowanie cząstkowe (najdłuższe trafienie)
    best = None
    best_len = 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best = col
                best_len = len(c)
    return best or (columns[0] if columns else None)


def detect_sheet(path):
    """Zwraca nazwę (pierwszego) arkusza. Jeśli xls/xlsx ma wiele arkuszy, wybiera pierwszy (preferuje nazwy typu: data/dane/zadania)."""
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names if str(s).strip().lower() in {'data', 'dane', 'tasks', 'zadania', 'sheet1', 'arkusz1'}]
        if preferred:
            return preferred[0]
        return xls.sheet_names[0]
    except Exception:
        return None

def _base_frame_for_chart(df, col_person, col_created, col_completed):
    created = pd.to_datetime(df[col_created], errors='coerce')
    completed = pd.to_datetime(df[col_completed], errors='coerce')
    person = df[col_person].astype(str).replace({'nan': '', 'None': ''}).str.strip().replace('', '—')
    # filtruj zasobniki niebrane pod uwagę
    keep = ~person.str.upper().isin(["TO DO", "INSTRUKCJA", "HOLD"])
    return person[keep], created[keep], completed[keep]


def make_chart_df(df, col_person, col_created, col_completed):
    """Dane pod wykres: Osoba/Rok/Kwartał + 2 serie:
       'Liczba NIEUKOŃCZONYCH' (brak daty zakończenia; liczone wg daty utworzenia)
       'Liczba ZAKOŃCZONYCH'  (liczone wg daty zakończenia)"""
    person, created, completed = _base_frame_for_chart(df, col_person, col_created, col_completed)

    # NIEUKOŃCZONE: completed isna() → grupujemy po kwartale z daty utworzenia
    nd_mask = completed.isna()
    nd = (pd.DataFrame({
            'osoba': person.where(nd_mask),
            'rok': created.where(nd_mask).dt.year,
            'kwartał': created.where(nd_mask).dt.quarter
        })
        .dropna(subset=['osoba', 'rok', 'kwartał'])
        .groupby(['osoba', 'rok', 'kwartał']).size()
        .rename('Liczba NIEUKOŃCZONYCH').reset_index()
    )

    # UKOŃCZONE: liczymy wg daty zakończenia
    done = (pd.DataFrame({
            'osoba': person,
            'rok': completed.dt.year,
            'kwartał': completed.dt.quarter
        })
        .dropna(subset=['rok', 'kwartał'])
        .groupby(['osoba', 'rok', 'kwartał']).size()
        .rename('Liczba ZAKOŃCZONYCH').reset_index()
    )

    pivot = pd.merge(nd, done, on=['osoba', 'rok', 'kwartał'], how='outer')
    pivot['Liczba NIEUKOŃCZONYCH'] = pivot['Liczba NIEUKOŃCZONYCH'].fillna(0).astype(int)
    pivot['Liczba ZAKOŃCZONYCH']  = pivot['Liczba ZAKOŃCZONYCH'].fillna(0).astype(int)

    pivot = pivot.sort_values(
        by=['osoba', 'rok', 'kwartał'],
        key=lambda s: s.str.casefold() if s.name == 'osoba' else s
    ).reset_index(drop=True)
    return pivot



def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None, title="ZADANIE UTWORZONE DO ZAKOŃCZONE"):
    """Tworzy arkusze: Agregat (opcjonalnie), Szczegóły (opcjonalnie), Pivot oraz Wykres z 3-poziomową osią."""
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        # (opcjonalnie) Twoje dotychczasowe tabele:
        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        # Pivot pod wykres
        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot
        cat_person = pivot_df['osoba'].tolist()
        cat_year   = pivot_df['rok'].astype(int).tolist()
        cat_quart  = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)",   *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba NIEUKOŃCZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba NIEUKOŃCZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        # Arkusz z wykresem
        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column','subtype': 'stacked'})

        # multi-level categories: 3 rzędy (0..2), kolumny 1..N
        last_col = len(pivot_df)
        chart.add_series({
            'name':       "='Pivot'!$A$5",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#F9A825'},  # (opcjonalnie) pomarańczowy dla NIEUKOŃCZONYCH
            'border': {'color': '#F9A825'},
        })
        chart.add_series({
            'name':       "='Pivot'!$A$6",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#1E88E5'},  # niebieski dla ZAKOŃCZONYCH
            'border': {'color': '#1E88E5'},
        })
        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})

        ws_chart.insert_chart('B2', chart, {'x_scale': 2.0, 'y_scale': 1.6})


def _sort_key_person(series):
    # sortowanie osób bez rozróżniania wielkości liter i z zachowaniem oryginalnej postaci
    return series.astype(str).fillna('').str.casefold()


def aggregate_and_details(df, col_person, col_created, col_completed):
    """
    Zwraca:
      - out (agregat): rok | kwartał | zasobnik | zadania_ukończone | zadania_nieukończone
      - details (szczegóły zadań): osoba | data_utworzenia | data_ukonczenia | status | rok | kwartał
        (rok/kwartał wg ukończenia dla ukończonych, a wg utworzenia dla nieukończonych)
    oraz odpowiednie sortowania.
    """
    df = df.copy()

    created = pd.to_datetime(df[col_created], errors='coerce')
    completed = pd.to_datetime(df[col_completed], errors='coerce')

    person_raw = df[col_person].astype(str).replace({'nan': '', 'None': ''}).str.strip()
    person = person_raw.replace('', '—')


# --- FILTR: pomiń zasobniki "TO DO", "INSTRUKCJA", "HOLD"
    mask = ~person.str.upper().isin(["TO DO", "INSTRUKCJA", "HOLD"])
    df = df.loc[mask].copy()
    created = created.loc[mask]
    completed = completed.loc[mask]
    person = person.loc[mask]
    # ---

    # === AGREGAT ===
    # Ukończone wg daty ukończenia
    done = (
        pd.DataFrame({
            'rok': completed.dt.year,
            'kwartał': completed.dt.quarter,
            'zasobnik': person,
        })
        .dropna(subset=['rok', 'kwartał'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False)
        .size()
        .rename('zadania_ukończone')
        .reset_index()
    )

    # Nieukończone wg daty utworzenia
    not_done_mask = completed.isna()
    not_done_created = created.where(not_done_mask)

    not_done = (
        pd.DataFrame({
            'rok': not_done_created.dt.year,
            'kwartał': not_done_created.dt.quarter,
            'zasobnik': person.where(not_done_mask),
        })
        .dropna(subset=['rok', 'kwartał', 'zasobnik'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False)
        .size()
        .rename('zadania_nieukończone')
        .reset_index()
    )

    out = pd.merge(done, not_done, on=['rok', 'kwartał', 'zasobnik'], how='outer')
    out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
    out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)

    # Sort: osoba → rok → kwartał
    out.sort_values(
        by=['zasobnik', 'rok', 'kwartał'],
        key=lambda col: _sort_key_person(col) if col.name == 'zasobnik' else col,
        inplace=True
    )
    out.reset_index(drop=True, inplace=True)

    # === SZCZEGÓŁY (chronologicznie) ===
    # Budujemy dwa zestawy: ukończone i nieukończone, z odpowiednimi "rok/kwartał"
    det_done = pd.DataFrame({
        'osoba': person,
        'data_utworzenia': created,
        'data_ukonczenia': completed,
        'status': pd.Series(['ukończone'] * len(df)),
        'rok': completed.dt.year,
        'kwartał': completed.dt.quarter,
    }).dropna(subset=['rok', 'kwartał'])  # tylko te, które faktycznie są ukończone

    det_not_done = pd.DataFrame({
        'osoba': person.where(not_done_mask),
        'data_utworzenia': created.where(not_done_mask),
        'data_ukonczenia': completed.where(not_done_mask),
        'status': pd.Series(['nieukończone'] * len(df)).where(not_done_mask),
        'rok': not_done_created.dt.year,
        'kwartał': not_done_created.dt.quarter,
    }).dropna(subset=['osoba', 'rok', 'kwartał'])

    details = pd.concat([det_done, det_not_done], ignore_index=True)

    # Sort: osoba → rok → kwartał → data_utworzenia → data_ukonczenia
    details.sort_values(
        by=['osoba', 'rok', 'kwartał', 'data_utworzenia', 'data_ukonczenia'],
        key=lambda col: _sort_key_person(col) if col.name == 'osoba' else col,
        inplace=True
    )
    details.reset_index(drop=True, inplace=True)

    return out, details


class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self.create_widgets()

    def create_widgets(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self)
        frm.pack(fill='both', expand=True)

        # Wybór pliku
        row = ttk.Frame(frm); row.pack(fill='x', **pad)
        ttk.Label(row, text="Plik Excel:").pack(side='left')
        ttk.Entry(row, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row, text="Wybierz...", command=self.pick_file).pack(side='left')

        # Arkusz
        row2 = ttk.Frame(frm); row2.pack(fill='x', **pad)
        ttk.Label(row2, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(row2, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row2, text="Wczytaj", command=self.load_sheet).pack(side='left')

        # Mapowanie kolumn
        box = ttk.LabelFrame(frm, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        r1 = ttk.Frame(box); r1.pack(fill='x', **pad)
        ttk.Label(r1, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r1, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r2 = ttk.Frame(box); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r2, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r3 = ttk.Frame(box); r3.pack(fill='x', **pad)
        ttk.Label(r3, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r3, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # Akcje
        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz...", command=self.convert_and_save).pack(side='right')

        # Status/log
        self.status = tk.Text(frm, height=10)
        self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')

        self.log("Wybierz plik i arkusz, potem sprawdź mapowanie kolumn. Sort: osoba → rok → kwartał (1–4).")

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path:
            return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_var.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self.log(f"Błąd odczytu pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self.log(f"Wczytano arkusz. Kolumny: {self.columns}")

            # comboboksy
            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns

            # auto-detekcja
            person_guess = best_guess_column(self.columns, [
                'zasobnik', 'osoba', 'assignee', 'owner', 'wykonawca', 'przypisane do', 'assigned to', 'user'
            ])
            created_guess = best_guess_column(self.columns, [
                'data utworzenia', 'utworzenia', 'created', 'creation date', 'created at', 'created time', 'start date', 'start'
            ])
            completed_guess = best_guess_column(self.columns, [
                'data ukończenia', 'zakonczenia', 'ukończenia', 'completed', 'done', 'closed', 'finished', 'end date', 'resolution date'
            ])

            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self.log("Sprawdź automatyczne dopasowanie kolumn i w razie potrzeby zmień ręcznie.")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self.log(f"Błąd wczytywania arkusza: {e}")

    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            out, details = aggregate_and_details(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            top = out.head(50)
            self.log(f"AGREGAT (pierwsze {len(top)} wierszy):\n{top.to_string(index=False)}")
            self.log("Szczegóły zadań (posortowane): osoba → rok → kwartał → daty (podgląd 10 pierwszych):")
            self.log(details.head(10).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się przygotować podglądu:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
            if self.df is None:
                messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
                return
            try:
                # Twoje dotychczasowe zestawienia
                out, details = aggregate_and_details(
                    self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
                )
                # Dane pod wykres (zielone=utworzone, niebieskie=ukończone)
                pivot = make_chart_df(
                    self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
                )

                in_path = self.path_var.get().strip()
                base, _ = os.path.splitext(in_path)
                default_out = base + "_WYNIK_z_wykresem.xlsx"

                path = filedialog.asksaveasfilename(
                    title="Zapisz wynik jako",
                    defaultextension=".xlsx",
                    initialfile=os.path.basename(default_out),
                    filetypes=[("Excel files", "*.xlsx")]
                )
                if not path:
                    self.log("Anulowano zapis.")
                    return

                write_excel_with_chart(
                    path,
                    pivot_df=pivot,
                    aggr_df=out,
                    details_df=details,
                    title="ZADANIE UTWORZONE DO ZAKOŃCZONE"
                )
                self.log(f"Zapisano wynik do: {path}")
                messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
            except Exception as e:
                messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
                self.log(f"Błąd zapisu: {e}")



def main():
    # DPI-aware na Windows (jeśli dostępne)
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()


if __name__ == "__main__":
    main()


C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_3164\4176094109.py:46: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  created = pd.to_datetime(df[col_created], errors='coerce')
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_3164\4176094109.py:47: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  completed = pd.to_datetime(df[col_completed], errors='coerce')
C:\Users\mzasiadczuk\AppData\Local\Temp\ipykernel_3164\4176094109.py:46: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  created = pd.to_datetime(df[col_created], errors='coerce')
C:\Users\mzasiadczuk\App

# 22.08.2025


In [10]:
import os
import re
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd


APP_TITLE = "Konwerter zadań: Excel → Excel (rok/kwartał/zasobnik)"
APP_GEOMETRY = "720x520"


# ---------- UTYLITY ----------

def best_guess_column(columns, candidates):
    """
    Zwraca najlepsze dopasowanie kolumny na podstawie listy słów-kluczy.
    Dopasowanie ignoruje wielkość liter, polskie znaki i separatory.
    """
    import unicodedata

    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s

    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]

    # pełne dopasowanie
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col

    # dopasowanie cząstkowe (najdłuższe trafienie)
    best = None
    best_len = 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best = col
                best_len = len(c)
    return best or (columns[0] if columns else None)


def detect_sheet(path):
    """Zwraca pierwszy pasujący arkusz (preferuje: data/dane/zadania), albo pierwszy z listy."""
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names
                     if str(s).strip().lower() in {'data', 'dane', 'tasks', 'zadania', 'sheet1', 'arkusz1'}]
        return preferred[0] if preferred else xls.sheet_names[0]
    except Exception:
        return None


def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)

    # próba standardowa
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)

    # poprawki dla dd.mm.rrrr i dd.mm.rrrr HH:MM
    mask = dt.isna() & s.astype(str).str.match(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        vals = s[mask].astype(str).str.strip()
        dt2 = pd.to_datetime(vals, format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(vals[still], format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2

    # poprawki dla ISO (rrrr-mm-dd)
    mask_iso = dt.isna() & s.astype(str).str.match(r"\d{4}-\d{2}-\d{2}")
    if mask_iso.any():
        vals = s[mask_iso].astype(str).str.strip()
        dt_iso = pd.to_datetime(vals, format='%Y-%m-%d', errors='coerce')
        dt.loc[mask_iso] = dt_iso

    return dt




# ---------- PRZYGOTOWANIE DANYCH ----------

def _base_frame_for_chart(df, col_person, col_created, col_completed):
    created   = smart_parse_date(df[col_created])
    completed = smart_parse_date(df[col_completed])

    person = (df[col_person].astype(str)
              .replace({'nan': '', 'None': ''})
              .str.strip()
              .replace('', '—'))
    # filtr: pomiń zasobniki niebrane pod uwagę
    keep = ~person.str.upper().isin(["TO DO", "INSTRUKCJA", "HOLD"])
    return person[keep], created[keep], completed[keep]


def make_chart_df(df, col_person, col_created, col_completed):
    """
    Dane pod wykres: Osoba/Rok/Kwartał + 2 serie:
      'Liczba UTWORZONYCH' (wg daty utworzenia)
      'Liczba ZAKOŃCZONYCH' (wg daty zakończenia)
    """
    person, created, completed = _base_frame_for_chart(df, col_person, col_created, col_completed)

    # UTWORZONE (po dacie utworzenia)
    created_df = (
        pd.DataFrame({
            'osoba': person,
            'rok': created.dt.year,
            'kwartał': created.dt.quarter
        })
        .dropna(subset=['osoba', 'rok', 'kwartał'])
        .groupby(['osoba', 'rok', 'kwartał']).size()
        .rename('Liczba UTWORZONYCH').reset_index()
    )

    # ZAKOŃCZONE (po dacie zakończenia)
    done_df = (
        pd.DataFrame({
            'osoba': person,
            'rok': completed.dt.year,
            'kwartał': completed.dt.quarter
        })
        .dropna(subset=['rok', 'kwartał'])
        .groupby(['osoba', 'rok', 'kwartał']).size()
        .rename('Liczba ZAKOŃCZONYCH').reset_index()
    )

    # merge i porządki
    pivot = pd.merge(created_df, done_df, on=['osoba', 'rok', 'kwartał'], how='outer')
    pivot['Liczba UTWORZONYCH'] = pivot['Liczba UTWORZONYCH'].fillna(0).astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].fillna(0).astype(int)

    pivot = pivot.sort_values(
        by=['osoba', 'rok', 'kwartał'],
        key=lambda s: s.str.casefold() if s.name == 'osoba' else s
    ).reset_index(drop=True)

    return pivot


def _sort_key_person(series):
    return series.astype(str).fillna('').str.casefold()


def aggregate_and_details(df, col_person, col_created, col_completed):
    """
    Zwraca:
      - out (agregat): rok | kwartał | zasobnik | zadania_ukończone | zadania_nieukończone
      - details (szczegóły zadań): osoba | data_utworzenia | data_ukonczenia | status | rok | kwartał
    """
    df = df.copy()

    created = smart_parse_date(df[col_created])
    completed = smart_parse_date(df[col_completed])

    person_raw = df[col_person].astype(str).replace({'nan': '', 'None': ''}).str.strip()
    person = person_raw.replace('', '—')

    # filtr: pomiń TO DO/INSTRUKCJA/HOLD
    mask = ~person.str.upper().isin(["TO DO", "INSTRUKCJA", "HOLD"])
    df = df.loc[mask].copy()
    created = created.loc[mask]
    completed = completed.loc[mask]
    person = person.loc[mask]

    # Ukończone wg daty ukończenia
    done = (
        pd.DataFrame({'rok': completed.dt.year, 'kwartał': completed.dt.quarter, 'zasobnik': person})
        .dropna(subset=['rok', 'kwartał'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False)
        .size().rename('zadania_ukończone').reset_index()
    )

    # Nieukończone wg daty utworzenia
    not_done_mask = completed.isna()
    not_done_created = created.where(not_done_mask)
    not_done = (
        pd.DataFrame({'rok': not_done_created.dt.year, 'kwartał': not_done_created.dt.quarter,
                      'zasobnik': person.where(not_done_mask)})
        .dropna(subset=['rok', 'kwartał', 'zasobnik'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False)
        .size().rename('zadania_nieukończone').reset_index()
    )

    out = pd.merge(done, not_done, on=['rok', 'kwartał', 'zasobnik'], how='outer')
    out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
    out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)

    out.sort_values(
        by=['zasobnik', 'rok', 'kwartał'],
        key=lambda col: _sort_key_person(col) if col.name == 'zasobnik' else col,
        inplace=True
    )
    out.reset_index(drop=True, inplace=True)

    # Szczegóły
    det_done = pd.DataFrame({
        'osoba': person, 'data_utworzenia': created, 'data_ukonczenia': completed,
        'status': pd.Series(['ukończone'] * len(df)), 'rok': completed.dt.year, 'kwartał': completed.dt.quarter
    }).dropna(subset=['rok', 'kwartał'])

    det_not_done = pd.DataFrame({
        'osoba': person.where(not_done_mask), 'data_utworzenia': created.where(not_done_mask),
        'data_ukonczenia': completed.where(not_done_mask),
        'status': pd.Series(['nieukończone'] * len(df)).where(not_done_mask),
        'rok': not_done_created.dt.year, 'kwartał': not_done_created.dt.quarter
    }).dropna(subset=['osoba', 'rok', 'kwartał'])

    details = pd.concat([det_done, det_not_done], ignore_index=True)
    details.sort_values(
        by=['osoba', 'rok', 'kwartał', 'data_utworzenia', 'data_ukonczenia'],
        key=lambda col: _sort_key_person(col) if col.name == 'osoba' else col,
        inplace=True
    )
    details.reset_index(drop=True, inplace=True)

    return out, details


# ---------- ZAPIS XLSX + WYKRES ----------

def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None, title="ZADANIE UTWORZONE DO ZAKOŃCZONE"):
    """Tworzy arkusze: Agregat, Szczegóły, Pivot oraz Wykres z 3-poziomową osią."""
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        # Tabele dodatkowe (opcjonalnie)
        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        # Pivot pod wykres
        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot

        # sanity dla XML: usuń znaki kontrolne
        def clean(s):
            if s is None: return ""
            s = str(s).replace("\r", " ").replace("\n", " ").strip()
            return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", "", s)

        pivot_df = pivot_df.copy()
        pivot_df["osoba"] = pivot_df["osoba"].map(clean)
        pivot_df["rok"] = pivot_df["rok"].astype(int)
        pivot_df["kwartał"] = pivot_df["kwartał"].astype(int)
        pivot_df["Liczba UTWORZONYCH"] = pivot_df["Liczba UTWORZONYCH"].fillna(0).astype(int)
        pivot_df["Liczba ZAKOŃCZONYCH"] = pivot_df["Liczba ZAKOŃCZONYCH"].fillna(0).astype(int)

        cat_person = pivot_df['osoba'].tolist()
        cat_year   = pivot_df['rok'].tolist()
        cat_quart  = pivot_df['kwartał'].map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)",   *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        last_col = len(pivot_df)
        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart

        if last_col >= 1:
            chart = book.add_chart({'type': 'column', 'subtype': 'stacked'})

            cat_range = ['Pivot', 0, 1, 2, last_col]  # 3-poziomowa oś

            chart.add_series({
                'name':       "='Pivot'!$A$5",   # UTWORZONE
                'categories': cat_range,
                'values':     ['Pivot', 4, 1, 4, last_col],
                'data_labels': {'value': True},
            })
            chart.add_series({
                'name':       "='Pivot'!$A$6",   # ZAKOŃCZONE
                'categories': cat_range,
                'values':     ['Pivot', 5, 1, 5, last_col],
                'data_labels': {'value': True},
            })

            chart.set_title({'name': title})
            chart.set_legend({'position': 'top'})
            chart.set_y_axis({'major_gridlines': {'visible': True}})

            ws_chart.insert_chart('B2', chart, {'x_scale': 2.0, 'y_scale': 1.6})
        else:
            ws_chart.write(1, 1, "Brak punktów do wykresu – nie dodano obiektu wykresu.")


# ---------- GUI ----------

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self.create_widgets()

    def create_widgets(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self)
        frm.pack(fill='both', expand=True)

        # Wybór pliku
        row = ttk.Frame(frm); row.pack(fill='x', **pad)
        ttk.Label(row, text="Plik Excel:").pack(side='left')
        ttk.Entry(row, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row, text="Wybierz...", command=self.pick_file).pack(side='left')

        # Arkusz
        row2 = ttk.Frame(frm); row2.pack(fill='x', **pad)
        ttk.Label(row2, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(row2, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row2, text="Wczytaj", command=self.load_sheet).pack(side='left')

        # Mapowanie kolumn
        box = ttk.LabelFrame(frm, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        r1 = ttk.Frame(box); r1.pack(fill='x', **pad)
        ttk.Label(r1, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r1, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r2 = ttk.Frame(box); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r2, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r3 = ttk.Frame(box); r3.pack(fill='x', **pad)
        ttk.Label(r3, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r3, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # Akcje
        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz...", command=self.convert_and_save).pack(side='right')

        # Status/log
        self.status = tk.Text(frm, height=10)
        self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')

        self.log("Wybierz plik i arkusz, potem sprawdź mapowanie kolumn. Sort: osoba → rok → kwartał (1–4).")

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path:
            return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_var.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self.log(f"Błąd odczytu pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self.log(f"Wczytano arkusz. Kolumny: {self.columns}")

            # comboboksy
            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns

            # auto-detekcja
            person_guess = best_guess_column(self.columns, [
                'zasobnik', 'osoba', 'assignee', 'owner', 'wykonawca', 'przypisane do', 'assigned to', 'user'
            ])
            created_guess = best_guess_column(self.columns, [
                'data utworzenia', 'utworzenia', 'created', 'creation date', 'created at', 'created time', 'start date', 'start'
            ])
            completed_guess = best_guess_column(self.columns, [
                'data ukończenia', 'zakonczenia', 'ukończenia', 'completed', 'done', 'closed', 'finished', 'end date', 'resolution date'
            ])

            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self.log("Sprawdź automatyczne dopasowanie kolumn i w razie potrzeby zmień ręcznie.")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self.log(f"Błąd wczytywania arkusza: {e}")

    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            out, details = aggregate_and_details(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            top = out.head(50)
            self.log(f"AGREGAT (pierwsze {len(top)} wierszy):\n{top.to_string(index=False)}")
            self.log("Szczegóły zadań (posortowane): osoba → rok → kwartał → daty (podgląd 10 pierwszych):")
            self.log(details.head(10).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się przygotować podglądu:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            # Zestawienia
            out, details = aggregate_and_details(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            # Dane pod wykres: UTWORZONE vs ZAKOŃCZONE
            pivot = make_chart_df(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )

            in_path = self.path_var.get().strip()
            base, _ = os.path.splitext(in_path)
            default_out = base + "_WYNIK_z_wykresem.xlsx"

            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not path:
                self.log("Anulowano zapis.")
                return

            write_excel_with_chart(
                path,
                pivot_df=pivot,
                aggr_df=out,
                details_df=details,
                # base_df=diag_df,  # jeśli masz ramkę z diagnostyką; w przeciwnym razie pomiń
                title="ZADANIE UTWORZONE DO ZAKOŃCZONE"
)

            self.log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
            self.log(f"Błąd zapisu: {e}")


# ---------- ENTRYPOINT ----------

def main():
    try:
        # DPI-aware na Windows (jeśli dostępne)
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass

    app = App()
    app.mainloop()


if __name__ == "__main__":
    main()


In [5]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import numpy as np

APP_TITLE = "Konwerter zadań: Excel → Excel (rok/kwartał/zasobnik)"
APP_GEOMETRY = "780x560"

# ======== BARDZO ODPORNY PARSER DAT ========
def smart_parse_date(s: pd.Series) -> pd.Series:
    """
    Parsuje daty z mieszanych źródeł:
    - liczby Excela (dni od 1899-12-30)
    - dd.mm.rrrr , dd.mm.rrrr HH:MM
    - dd/mm/rrrr , dd/mm/rrrr HH:MM
    - yyyy-mm-dd , yyyy-mm-dd HH:MM[:SS]
    - z czyszczeniem NBSP i zero-width
    """
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)

    # 0) czyszczenie śmieciowych znaków
    def _clean(x):
        if pd.isna(x):
            return x
        x = str(x)
        x = x.replace("\u200b", "").replace("\u200c", "").replace("\u200d", "")  # zero-width
        x = x.replace("\xa0", " ")  # NBSP
        return x.strip()

    s_clean = s.map(_clean)

    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    # 1) liczby Excela (sensowny zakres)
    as_num = pd.to_numeric(s_clean, errors="coerce")
    m_num = as_num.notna() & (as_num >= 60) & (as_num < 80000)
    if m_num.any():
        dt_num = pd.to_datetime(as_num[m_num], unit="D", origin="1899-12-30", errors="coerce")
        out.loc[m_num] = dt_num

    # 2) szybka próba auto + dayfirst
    m = out.isna()
    if m.any():
        dt_auto = pd.to_datetime(s_clean[m], errors="coerce", dayfirst=True, infer_datetime_format=True, utc=False)
        out.loc[m & dt_auto.notna()] = dt_auto

    # 3) zestaw formatów jawnych
    formats = [
        "%d.%m.%Y %H:%M",
        "%d.%m.%Y",
        "%d/%m/%Y %H:%M",
        "%d/%m/%Y",
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %H:%M",
        "%Y-%m-%d",
    ]
    for fmt in formats:
        m = out.isna()
        if not m.any():
            break
        dt_try = pd.to_datetime(s_clean[m], format=fmt, errors="coerce")
        out.loc[m & dt_try.notna()] = dt_try

    return out


def best_guess_column(columns, candidates):
    """Najlepsze dopasowanie nazwy kolumny po uproszczeniu znaków."""
    import unicodedata
    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s
    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]
    # pełne dopasowanie
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col
    # najdłuższy substring
    best = None; best_len = 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best, best_len = col, len(c)
    return best or (columns[0] if columns else None)


def detect_sheet(path):
    """Wybór arkusza: preferuj 'data/dane/tasks/zadania'."""
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names if str(s).strip().lower() in {
            'data', 'dane', 'tasks', 'zadania', 'sheet1', 'arkusz1'
        }]
        return preferred[0] if preferred else xls.sheet_names[0]
    except Exception:
        return None


def _base_frame_for_chart(df, col_person, col_created, col_completed):
    # solidne parsowanie
    created   = smart_parse_date(df[col_created])
    completed = smart_parse_date(df[col_completed])
    person = (df[col_person].astype(str)
              .replace({'nan': '', 'None': ''})
              .str.strip()
              .replace('', '—'))
    # wytnij niechciane zasobniki
    keep = ~person.str.upper().isin(["TO DO", "INSTRUKCJA", "HOLD"])
    return person[keep], created[keep], completed[keep]


def make_chart_df(df, col_person, col_created, col_completed):
    """
    Dane pod wykres: Osoba/Rok/Kwartał + 2 serie:
      'Liczba UTWORZONYCH' (po dacie utworzenia)
      'Liczba ZAKOŃCZONYCH' (po dacie zakończenia)
    """
    person, created, completed = _base_frame_for_chart(df, col_person, col_created, col_completed)

    created_df = (
        pd.DataFrame({
            'osoba': person,
            'rok':   created.dt.year.astype('Int64'),
            'kwartał': created.dt.quarter.astype('Int64')
        })
        .dropna(subset=['osoba', 'rok', 'kwartał'])
        .groupby(['osoba', 'rok', 'kwartał'], dropna=False).size()
        .rename('Liczba UTWORZONYCH').reset_index()
    )

    done_df = (
        pd.DataFrame({
            'osoba': person,
            'rok':   completed.dt.year.astype('Int64'),
            'kwartał': completed.dt.quarter.astype('Int64')
        })
        .dropna(subset=['rok', 'kwartał'])
        .groupby(['osoba', 'rok', 'kwartał'], dropna=False).size()
        .rename('Liczba ZAKOŃCZONYCH').reset_index()
    )

    pivot = pd.merge(created_df, done_df, on=['osoba', 'rok', 'kwartał'], how='outer')
    pivot['Liczba UTWORZONYCH']  = pivot['Liczba UTWORZONYCH'].fillna(0).astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].fillna(0).astype(int)

    pivot = pivot.sort_values(by=['osoba', 'rok', 'kwartał'],
                              key=lambda s: s.str.casefold() if s.name == 'osoba' else s).reset_index(drop=True)
    return pivot


def _sort_key_person(series):
    return series.astype(str).fillna('').str.casefold()


def aggregate_and_details(df, col_person, col_created, col_completed):
    """(Tak jak miałeś) – zwraca agregat i szczegóły, ale już na solidnie sparsowanych datach."""
    df = df.copy()
    created   = smart_parse_date(df[col_created])
    completed = smart_parse_date(df[col_completed])
    person_raw = df[col_person].astype(str).replace({'nan': '', 'None': ''}).str.strip()
    person = person_raw.replace('', '—')

    mask = ~person.str.upper().isin(["TO DO", "INSTRUKCJA", "HOLD"])
    created, completed, person = created[mask], completed[mask], person[mask]

    done = (
        pd.DataFrame({'rok': completed.dt.year, 'kwartał': completed.dt.quarter, 'zasobnik': person})
        .dropna(subset=['rok', 'kwartał'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False).size()
        .rename('zadania_ukończone').reset_index()
    )
    not_done_mask = completed.isna()
    not_done_created = created.where(not_done_mask)
    not_done = (
        pd.DataFrame({'rok': not_done_created.dt.year, 'kwartał': not_done_created.dt.quarter, 'zasobnik': person.where(not_done_mask)})
        .dropna(subset=['rok', 'kwartał', 'zasobnik'])
        .groupby(['rok', 'kwartał', 'zasobnik'], dropna=False).size()
        .rename('zadania_nieukończone').reset_index()
    )

    out = pd.merge(done, not_done, on=['rok', 'kwartał', 'zasobnik'], how='outer')
    out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
    out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)
    out.sort_values(by=['zasobnik', 'rok', 'kwartał'],
                    key=lambda col: _sort_key_person(col) if col.name == 'zasobnik' else col,
                    inplace=True)
    out.reset_index(drop=True, inplace=True)

    det_done = pd.DataFrame({
        'osoba': person, 'data_utworzenia': created, 'data_ukonczenia': completed,
        'status': pd.Series(['ukończone'] * len(person)),
        'rok': completed.dt.year, 'kwartał': completed.dt.quarter,
    }).dropna(subset=['rok', 'kwartał'])

    det_not_done = pd.DataFrame({
        'osoba': person.where(not_done_mask),
        'data_utworzenia': created.where(not_done_mask),
        'data_ukonczenia': completed.where(not_done_mask),
        'status': pd.Series(['nieukończone'] * len(person)).where(not_done_mask),
        'rok': not_done_created.dt.year, 'kwartał': not_done_created.dt.quarter,
    }).dropna(subset=['osoba', 'rok', 'kwartał'])

    details = pd.concat([det_done, det_not_done], ignore_index=True)
    details.sort_values(by=['osoba', 'rok', 'kwartał', 'data_utworzenia', 'data_ukonczenia'],
                        key=lambda col: _sort_key_person(col) if col.name == 'osoba' else col,
                        inplace=True)
    details.reset_index(drop=True, inplace=True)
    return out, details


def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None, title="ZADANIE UTWORZONE DO ZAKOŃCZONE"):
    """Arkusze: Agregat (opc.), Szczegóły (opc.), Pivot, Wykres + arkusz Diag (kontrola dat)."""
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        # Kontrolny arkusz diagnostyczny (pierwsze 200 wierszy pivotu)
        diag = pivot_df.copy()
        diag['kategoria'] = diag['rok'].astype(int).astype(str) + " | KWARTAŁ" + diag['kwartał'].astype(int).astype(str)
        diag[['osoba', 'kategoria', 'Liczba UTWORZONYCH', 'Liczba ZAKOŃCZONYCH']].head(200).to_excel(
            writer, sheet_name="Diag", index=False
        )

        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot
        cat_person = pivot_df['osoba'].tolist()
        cat_year   = pivot_df['rok'].astype(int).tolist()
        cat_quart  = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)",   *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column', 'subtype': 'stacked'})

        # zakres kategorii wielopoziomowej: rzędy 0..2, kolumny 1..N
        last_col = len(pivot_df)  # N kategorii => kolumny 1..N (0-based)
        chart.add_series({
            'name':       "='Pivot'!$A$5",  # UTWORZONE
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#43A047'},
            'border': {'color': '#43A047'},
        })
        chart.add_series({
            'name':       "='Pivot'!$A$6",  # ZAKOŃCZONE
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#1E88E5'},
            'border': {'color': '#1E88E5'},
        })
        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})

        ws_chart.insert_chart('B2', chart, {'x_scale': 2.0, 'y_scale': 1.6})


# ===================== GUI =====================
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self.create_widgets()

    def create_widgets(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self); frm.pack(fill='both', expand=True)

        row = ttk.Frame(frm); row.pack(fill='x', **pad)
        ttk.Label(row, text="Plik Excel:").pack(side='left')
        ttk.Entry(row, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row, text="Wybierz...", command=self.pick_file).pack(side='left')

        row2 = ttk.Frame(frm); row2.pack(fill='x', **pad)
        ttk.Label(row2, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(row2, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row2, text="Wczytaj", command=self.load_sheet).pack(side='left')

        box = ttk.LabelFrame(frm, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        r1 = ttk.Frame(box); r1.pack(fill='x', **pad)
        ttk.Label(r1, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r1, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r2 = ttk.Frame(box); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r2, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r3 = ttk.Frame(box); r3.pack(fill='x', **pad)
        ttk.Label(r3, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r3, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz...", command=self.convert_and_save).pack(side='right')

        self.status = tk.Text(frm, height=10)
        self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')

        self.log("Wybierz plik i arkusz, sprawdź mapowanie. Liczymy: UTWORZONE (po dacie utworzenia) vs ZAKOŃCZONE (po dacie zakończenia).")

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path:
            return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_var.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self.log(f"Błąd odczytu pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self.log(f"Wczytano arkusz. Kolumny: {self.columns}")

            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns

            person_guess = best_guess_column(self.columns, [
                'zasobnik', 'osoba', 'assignee', 'owner', 'wykonawca', 'przypisane do', 'assigned to', 'user'
            ])
            created_guess = best_guess_column(self.columns, [
                'data utworzenia', 'utworzenia', 'created', 'creation date', 'created at', 'created time', 'start date', 'start'
            ])
            completed_guess = best_guess_column(self.columns, [
                'data ukończenia', 'zakonczenia', 'ukończenia', 'completed', 'done', 'closed', 'finished', 'end date', 'resolution date'
            ])
            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self.log("Sprawdź automatyczne dopasowanie kolumn i w razie potrzeby zmień ręcznie.")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self.log(f"Błąd wczytywania arkusza: {e}")

    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            out, details = aggregate_and_details(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            top = out.head(50)
            self.log(f"AGREGAT (pierwsze {len(top)} wierszy):\n{top.to_string(index=False)}")
            self.log("Szczegóły (top 10):")
            self.log(details.head(10).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się przygotować podglądu:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            out, details = aggregate_and_details(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            pivot = make_chart_df(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            in_path = self.path_var.get().strip()
            base, _ = os.path.splitext(in_path)
            default_out = base + "_WYNIK_z_wykresem.xlsx"
            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not path:
                self.log("Anulowano zapis.")
                return

            write_excel_with_chart(
                path,
                pivot_df=pivot,
                aggr_df=out,
                details_df=details,
                title="ZADANIE UTWORZONE DO ZAKOŃCZONE"
            )
            self.log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
            self.log(f"Błąd zapisu: {e}")


def main():
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()


if __name__ == "__main__":
    main()


In [8]:
import os
import re
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import numpy as np

APP_TITLE = "Konwerter zadań: Excel → Excel (rok/kwartał/zasobnik)"
APP_GEOMETRY = "820x600"

# ===================== UTIL =====================

SEPARATORS_REGEX = r"(?:\s+i\s+|,|;|/|\||&|\+)"  # rozdzielacz osób: „, ; / | & +” oraz słowo „ i ”

def clean_text(x):
    if pd.isna(x):
        return x
    s = str(x)
    # usuń NBSP i zero-width
    s = s.replace("\xa0", " ").replace("\u200b", "").replace("\u200c", "").replace("\u200d", "")
    return s.strip()

def explode_people(series: pd.Series) -> pd.Series:
    return (series.astype(str)
                 .map(clean_text)
                 .str.replace(r"\s+", " ", regex=True)
                 .str.split(SEPARATORS_REGEX)
                 .explode()
                 .map(lambda v: (v or "").strip())
                 .replace("", np.nan))

# Bardzo odporny parser dat
def smart_parse_date(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(pd.NaT, index=[])
    if not isinstance(s, pd.Series):
        s = pd.Series(s)

    raw = s.map(clean_text)

    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    # 1) liczby Excela (rozsądny zakres dni)
    as_num = pd.to_numeric(raw, errors="coerce")
    mask_num = as_num.notna() & (as_num >= 60) & (as_num < 100000)
    if mask_num.any():
        out.loc[mask_num] = pd.to_datetime(as_num[mask_num], unit="D", origin="1899-12-30", errors="coerce")

    # 2) auto dayfirst
    mask = out.isna()
    if mask.any():
        auto_dt = pd.to_datetime(raw[mask], errors="coerce", dayfirst=True, infer_datetime_format=True, utc=False)
        out.loc[mask & auto_dt.notna()] = auto_dt

    # 3) jawne formaty (kolejno)
    explicit_formats = [
        "%d.%m.%Y %H:%M:%S",
        "%d.%m.%Y %H:%M",
        "%d.%m.%Y",
        "%d/%m/%Y %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%d/%m/%Y",
        "%d-%m-%Y %H:%M:%S",
        "%d-%m-%Y %H:%M",
        "%d-%m-%Y",
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %H:%M",
        "%Y-%m-%d",
    ]
    for fmt in explicit_formats:
        m = out.isna()
        if not m.any():
            break
        try_dt = pd.to_datetime(raw[m], format=fmt, errors="coerce")
        out.loc[m & try_dt.notna()] = try_dt

    return out

def quarter_label(year: int, q: int) -> str:
    return f"{int(year)} | KWARTAŁ{int(q)}"

def best_guess_column(columns, candidates):
    import unicodedata
    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s
    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col
    best = None; best_len = 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best, best_len = col, len(c)
    return best or (columns[0] if columns else None)

def detect_sheet(path):
    try:
        xls = pd.ExcelFile(path)
        pref = [s for s in xls.sheet_names if str(s).strip().lower() in
                {'data','dane','tasks','zadania','sheet1','arkusz1'}]
        return pref[0] if pref else xls.sheet_names[0]
    except Exception:
        return None

# ===================== CORE =====================

def _base_frame(df, col_person, col_created, col_completed):
    people = explode_people(df[col_person])
    created = smart_parse_date(df[col_created])
    completed = smart_parse_date(df[col_completed])

    out = pd.DataFrame({
        "osoba": people,
        "created": created,
        "completed": completed,
        "raw_created": df[col_created].astype(str),
        "raw_completed": df[col_completed].astype(str),
    }).dropna(subset=["osoba"])

    # filtruj niebrane zasobniki
    mask_keep = ~out["osoba"].str.upper().isin(["TO DO","INSTRUKCJA","HOLD"])
    return out.loc[mask_keep].reset_index(drop=True)

def make_chart_df(df, col_person, col_created, col_completed):
    base = _base_frame(df, col_person, col_created, col_completed)

    # Utworzone → po dacie utworzenia
    c_ok = base.dropna(subset=["created"]).copy()
    c_ok["rok"] = c_ok["created"].dt.year.astype("Int64")
    c_ok["kwartał"] = c_ok["created"].dt.quarter.astype("Int64")

    created_df = (c_ok.groupby(["osoba","rok","kwartał"], dropna=False)
                     .size().rename("Liczba UTWORZONYCH").reset_index())

    # Zakończone → po dacie zakończenia
    d_ok = base.dropna(subset=["completed"]).copy()
    d_ok["rok"] = d_ok["completed"].dt.year.astype("Int64")
    d_ok["kwartał"] = d_ok["completed"].dt.quarter.astype("Int64")

    done_df = (d_ok.groupby(["osoba","rok","kwartał"], dropna=False)
                    .size().rename("Liczba ZAKOŃCZONYCH").reset_index())

    pivot = pd.merge(created_df, done_df, on=["osoba","rok","kwartał"], how="outer").fillna(0)
    pivot["Liczba UTWORZONYCH"] = pivot["Liczba UTWORZONYCH"].astype(int)
    pivot["Liczba ZAKOŃCZONYCH"] = pivot["Liczba ZAKOŃCZONYCH"].astype(int)

    pivot = pivot.sort_values(
        by=["osoba","rok","kwartał"],
        key=lambda s: s.str.casefold() if s.name=="osoba" else s
    ).reset_index(drop=True)

    return pivot, base  # base wykorzystamy do diagnostyki

def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None, title="ZADANIE UTWORZONE DO ZAKOŃCZONE"):
    """
    Zapisuje:
      - (opcjonalnie) arkusz 'Agregat' i 'Szczegóły'
      - arkusz 'Pivot' (dla podglądu)
      - arkusz 'WykresDane' (etykiety i serie)
      - arkusz 'Wykres' (sam wykres kolumnowy 'stacked')

    Wykres korzysta z JEDNOPOZIOMOWYCH etykiet (flat labels), zapisanych w kolumnie na arkuszu 'WykresDane',
    dzięki czemu Excel nic nie „naprawia” i słupki trafiają do właściwych ćwiartek czasu.
    """
    import pandas as pd

    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        # (1) Opcjonalne: Twoje tabele
        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        # (2) Pivot – tylko do wglądu/debugu (nie używamy go wprost do wykresu)
        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot
        ws_pivot.write_row(0, 0, ["osoba", "rok", "kwartał", "Liczba UTWORZONYCH", "Liczba ZAKOŃCZONYCH"])
        for i, row in enumerate(pivot_df.itertuples(index=False), start=1):
            ws_pivot.write(i, 0, row.osoba)
            ws_pivot.write(i, 1, int(row.rok))
            ws_pivot.write(i, 2, int(row.kwartał))
            ws_pivot.write(i, 3, int(row._3 if hasattr(row, '_3') else row._4) if False else int(row[3]))  # safety
            ws_pivot.write(i, 4, int(row[4]))

        # (3) Dane do wykresu – płaskie etykiety + dwie serie
        ws_data = book.add_worksheet('WykresDane'); writer.sheets['WykresDane'] = ws_data
        text_fmt = book.add_format({'num_format': '@'})  # wymuś "tekst", żeby Excel nic nie konwertował

        # Nagłówki
        ws_data.write(0, 0, "Etykieta (rok|kwartał + osoba)")
        ws_data.write(0, 1, "Liczba UTWORZONYCH")
        ws_data.write(0, 2, "Liczba ZAKOŃCZONYCH")

        # Wiersze
        n = len(pivot_df)
        for i, (osoba, rok, kwartal, utw, zak) in enumerate(
            zip(pivot_df["osoba"], pivot_df["rok"], pivot_df["kwartał"],
                pivot_df["Liczba UTWORZONYCH"], pivot_df["Liczba ZAKOŃCZONYCH"]),
            start=1
        ):
            label = f"{int(rok)} | KWARTAŁ{int(kwartal)}\n{osoba}"
            ws_data.write(i, 0, label, text_fmt)
            ws_data.write_number(i, 1, int(utw))
            ws_data.write_number(i, 2, int(zak))

        # Zakresy dla wykresu
        # (Excel 1-indexed w etykietach – dane od wiersza 2 do 1+n)
        cat_range = ['WykresDane', 1, 0, n, 0]  # kolumna A (etykiety)
        utw_range = ['WykresDane', 1, 1, n, 1]  # kolumna B
        zak_range = ['WykresDane', 1, 2, n, 2]  # kolumna C

        # (4) Wykres
        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column', 'subtype': 'stacked'})

        chart.add_series({
            'name':       "Liczba UTWORZONYCH",
            'categories': cat_range,
            'values':     utw_range,
            'data_labels': {'value': True},
            'fill':   {'color': '#43A047'},
            'border': {'color': '#43A047'},
        })

        chart.add_series({
            'name':       "Liczba ZAKOŃCZONYCH",
            'categories': cat_range,
            'values':     zak_range,
            'data_labels': {'value': True},
            'fill':   {'color': '#1E88E5'},
            'border': {'color': '#1E88E5'},
        })

        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})
        chart.set_x_axis({'num_font': {'size': 9}})

        # Wstaw wykres (skalowanie możesz dostroić)
        ws_chart.insert_chart('B2', chart, {'x_scale': 2.0, 'y_scale': 1.6})


# ===================== AGREGAT / SZCZEGÓŁY (jak wcześniej) =====================

def _sort_key_person(series):
    return series.astype(str).fillna('').str.casefold()

def aggregate_and_details(df, col_person, col_created, col_completed):
    base = _base_frame(df, col_person, col_created, col_completed)

    # ukończone wg completed
    d_ok = base.dropna(subset=["completed"]).copy()
    done = (d_ok.assign(rok=d_ok["completed"].dt.year, kwartał=d_ok["completed"].dt.quarter)
                .groupby(["rok","kwartał","osoba"], dropna=False)
                .size().rename("zadania_ukończone").reset_index())

    # nieukończone wg created
    nd = base[base["completed"].isna()].copy()
    nd = nd.dropna(subset=["created"])
    not_done = (nd.assign(rok=nd["created"].dt.year, kwartał=nd["created"].dt.quarter)
                  .groupby(["rok","kwartał","osoba"], dropna=False)
                  .size().rename("zadania_nieukończone").reset_index())

    out = pd.merge(done, not_done, on=["rok","kwartał","osoba"], how="outer").fillna(0)
    out["zadania_ukończone"] = out["zadania_ukończone"].astype(int)
    out["zadania_nieukończone"] = out["zadania_nieukończone"].astype(int)

    out.sort_values(by=["osoba","rok","kwartał"],
                    key=lambda col: _sort_key_person(col) if col.name=="osoba" else col,
                    inplace=True)
    out.reset_index(drop=True, inplace=True)

    # szczegóły
    det_done = (d_ok.assign(status="ukończone",
                            rok=d_ok["completed"].dt.year,
                            kwartał=d_ok["completed"].dt.quarter)
                   [["osoba","created","completed","status","rok","kwartał"]])
    det_nd = (nd.assign(status="nieukończone",
                        rok=nd["created"].dt.year,
                        kwartał=nd["created"].dt.quarter)
                 [["osoba","created","completed","status","rok","kwartał"]])

    details = pd.concat([det_done, det_nd], ignore_index=True)
    details.rename(columns={"created":"data_utworzenia","completed":"data_ukonczenia"}, inplace=True)
    details.sort_values(by=["osoba","rok","kwartał","data_utworzenia","data_ukonczenia"],
                        key=lambda col: _sort_key_person(col) if col.name=="osoba" else col,
                        inplace=True)
    details.reset_index(drop=True, inplace=True)

    return out, details

# ===================== GUI =====================

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()
        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self.create_widgets()

    def create_widgets(self):
        pad={'padx':8,'pady':6}
        frm = ttk.Frame(self); frm.pack(fill='both', expand=True)

        r = ttk.Frame(frm); r.pack(fill='x', **pad)
        ttk.Label(r, text="Plik Excel:").pack(side='left')
        ttk.Entry(r, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r, text="Wybierz...", command=self.pick_file).pack(side='left')

        r2 = ttk.Frame(frm); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(r2, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r2, text="Wczytaj", command=self.load_sheet).pack(side='left')

        box = ttk.LabelFrame(frm, text="Mapowanie kolumn"); box.pack(fill='x', **pad)
        rr1 = ttk.Frame(box); rr1.pack(fill='x', **pad)
        ttk.Label(rr1, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(rr1, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        rr2 = ttk.Frame(box); rr2.pack(fill='x', **pad)
        ttk.Label(rr2, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(rr2, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        rr3 = ttk.Frame(box); rr3.pack(fill='x', **pad)
        ttk.Label(rr3, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(rr3, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz...", command=self.convert_and_save).pack(side='right')

        self.status = tk.Text(frm, height=10); self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')
        self.log("Liczę: UTWORZONE po dacie utworzenia, ZAKOŃCZONE po dacie zakończenia. "
                 "Arkusz 'Diag' pokaże dokładnie, jakie daty i kwartały wyszły po parsowaniu.")

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files","*.xlsx *.xls *.xlsm"),("All files","*.*")]
        )
        if not path: return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            self.sheet_var.set(detect_sheet(path) or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self.log(f"Błąd odczytu pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga","Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self.log(f"Wczytano arkusz. Kolumny: {self.columns}")

            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns

            # autodopasowanie
            person_guess = best_guess_column(self.columns, ['zasobnik','osoba','assignee','owner','wykonawca','przypisane do','assigned to','user'])
            created_guess = best_guess_column(self.columns, ['data utworzenia','utworzenia','created','creation date','created at','created time','start date','start'])
            completed_guess = best_guess_column(self.columns, ['data ukończenia','zakonczenia','ukończenia','completed','done','closed','finished','end date','resolution date'])
            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self.log("Sprawdź, czy mapowanie jest poprawne (zwłaszcza 'Data utworzenia').")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self.log(f"Błąd wczytywania arkusza: {e}")

    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga","Najpierw wczytaj arkusz.")
            return
        try:
            pivot, base = make_chart_df(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            self.log("PIVOT (pierwsze 30):")
            self.log(pivot.head(30).to_string(index=False))
            # szybki sanity-check: ile NaT po parsowaniu
            created_nat = base["created"].isna().sum()
            completed_nat = base["completed"].isna().sum()
            self.log(f"NaT po parsowaniu → created: {created_nat}, completed: {completed_nat}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Podgląd nieudany:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga","Najpierw wczytaj arkusz.")
            return
        try:
            pivot, base = make_chart_df(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            aggr, details = aggregate_and_details(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())

            in_path = self.path_var.get().strip()
            base_name, _ = os.path.splitext(in_path)
            default_out = base_name + "_WYNIK_z_wykresem.xlsx"
            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files","*.xlsx")]
            )
            if not path:
                self.log("Anulowano zapis.")
                return

            write_excel_with_chart(path, pivot_df=pivot, base_df=base, aggr_df=aggr, details_df=details,
                                   title="ZADANIE UTWORZONE DO ZAKOŃCZONE")
            self.log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
            self.log(f"Błąd zapisu: {e}")

def main():
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()

if __name__ == "__main__":
    main()


In [12]:
# -*- coding: utf-8 -*-
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd


APP_TITLE = "Konwerter zadań: Excel → Excel (Utworzone vs Zakończone)"
APP_GEOMETRY = "820x560"


# ---------------------------- Pomocnicze ----------------------------

def best_guess_column(columns, candidates):
    """Heurystyczny wybór kolumny na podstawie listy słów-kluczy."""
    import unicodedata
    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s

    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]

    # pełne dopasowanie
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col

    # dopasowanie cząstkowe (najdłuższe trafienie)
    best = None
    best_len = 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best = col
                best_len = len(c)
    return best or (columns[0] if columns else None)


def detect_sheet(path):
    """Preferuje arkusze o nazwach: data/dane/zadania/sheet1/arkusz1; inaczej pierwszy."""
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names
                     if str(s).strip().lower() in {'data', 'dane', 'tasks', 'zadania', 'sheet1', 'arkusz1'}]
        return preferred[0] if preferred else xls.sheet_names[0]
    except Exception:
        return None


# ---------------------------- Logika danych ----------------------------

def build_details_and_agregat(df, col_person, col_created, col_completed):
    """
    Zwraca:
      details: osoba | data_utworzenia | data_ukonczenia | status | rok | kwartał
               (rok/kwartał wg daty ukończenia dla ukończonych, a wg daty utworzenia dla nieukończonych)
      agregat: rok | kwartał | zasobnik | zadania_ukończone | zadania_nieukończone
    """
    person = df[col_person].astype(str).replace({'nan': '', 'None': ''}).str.strip().replace('', '—')
    created = pd.to_datetime(df[col_created], errors='coerce', dayfirst=True)
    completed = pd.to_datetime(df[col_completed], errors='coerce', dayfirst=True)

    rows = []
    for p, c, d in zip(person, created, completed):
        if pd.notna(d):  # UKOŃCZONE
            rows.append({
                'osoba': p,
                'data_utworzenia': c,
                'data_ukonczenia': d,
                'status': 'ukończone',
                'rok': int(d.year),
                'kwartał': int(d.quarter)
            })
        elif pd.notna(c):  # NIEUKOŃCZONE
            rows.append({
                'osoba': p,
                'data_utworzenia': c,
                'data_ukonczenia': pd.NaT,
                'status': 'nieukończone',
                'rok': int(c.year),
                'kwartał': int(c.quarter)
            })
    details = pd.DataFrame(rows)

    # Agregacja
    done = (details[details['status'] == 'ukończone']
            .groupby(['rok', 'kwartał', 'osoba']).size()
            .rename('zadania_ukończone'))
    not_done = (details[details['status'] == 'nieukończone']
                .groupby(['rok', 'kwartał', 'osoba']).size()
                .rename('zadania_nieukończone'))

    agregat = (pd.merge(done, not_done, on=['rok', 'kwartał', 'osoba'], how='outer')
               .fillna(0).reset_index())
    agregat['zadania_ukończone'] = agregat['zadania_ukończone'].astype(int)
    agregat['zadania_nieukończone'] = agregat['zadania_nieukończone'].astype(int)

    # sort: osoba/rok/kwartał
    agregat.sort_values(by=['osoba', 'rok', 'kwartał'], inplace=True, ignore_index=True)
    details.sort_values(by=['osoba', 'rok', 'kwartał', 'data_utworzenia', 'data_ukonczenia'],
                        inplace=True, ignore_index=True)
    return details, agregat


def make_pivot_created_done(df, col_person, col_created, col_completed):
    """
    Pivot do wykresu: Osoba/Rok/Kwartał + 2 serie:
    - 'Liczba UTWORZONYCH' (wg daty utworzenia)
    - 'Liczba ZAKOŃCZONYCH' (wg daty ukończenia)
    """
    person = df[col_person].astype(str).replace({'nan': '', 'None': ''}).str.strip().replace('', '—')
    created = pd.to_datetime(df[col_created], errors='coerce', dayfirst=True)
    completed = pd.to_datetime(df[col_completed], errors='coerce', dayfirst=True)

    created_df = (pd.DataFrame({'osoba': person, 'rok': created.dt.year, 'kwartał': created.dt.quarter})
                  .dropna(subset=['osoba', 'rok', 'kwartał'])
                  .groupby(['osoba', 'rok', 'kwartał']).size()
                  .rename('Liczba UTWORZONYCH').reset_index())

    done_df = (pd.DataFrame({'osoba': person, 'rok': completed.dt.year, 'kwartał': completed.dt.quarter})
               .dropna(subset=['rok', 'kwartał'])
               .groupby(['osoba', 'rok', 'kwartał']).size()
               .rename('Liczba ZAKOŃCZONYCH').reset_index())

    pivot = pd.merge(created_df, done_df, on=['osoba', 'rok', 'kwartał'], how='outer').fillna(0)
    pivot['Liczba UTWORZONYCH'] = pivot['Liczba UTWORZONYCH'].astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].astype(int)
    pivot.sort_values(by=['osoba', 'rok', 'kwartał'], inplace=True, ignore_index=True)
    return pivot


def write_excel_with_chart(path, pivot_df, agregat_df=None, details_df=None,
                           title="ZADANIA: UTWORZONE vs ZAKOŃCZONE"):
    """
    Zapisuje:
      - 'Agregat' (opcjonalnie),
      - 'Szczegóły' (opcjonalnie),
      - 'Pivot' (oś 3-poziomowa + wartości),
      - 'Wykres' (stacked column: Utworzone vs Zakończone).
    """
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        if agregat_df is not None:
            agregat_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        # --- Pivot (dla wykresu, multi-level categories) ---
        ws_pivot = book.add_worksheet('Pivot')
        writer.sheets['Pivot'] = ws_pivot

        if len(pivot_df) == 0:
            ws_pivot.write(0, 0, "Brak danych do wykresu")
            return

        cat_person = pivot_df['osoba'].tolist()
        cat_year = pivot_df['rok'].astype(int).tolist()
        cat_quart = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)", *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        # --- Wykres ---
        ws_chart = book.add_worksheet('Wykres')
        writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column', 'subtype': 'stacked'})

        last_col = len(pivot_df)  # kategorie: kolumny 1..last_col
        # UTWORZONE
        chart.add_series({
            'name': "='Pivot'!$A$5",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values': ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True}
        })
        # ZAKOŃCZONE
        chart.add_series({
            'name': "='Pivot'!$A$6",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values': ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True}
        })

        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})

        ws_chart.insert_chart('B2', chart, {'x_scale': 2.1, 'y_scale': 1.65})


# ---------------------------- GUI (tkinter) ----------------------------

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self._build_gui()

    def _build_gui(self):
        pad = {'padx': 10, 'pady': 6}
        root = ttk.Frame(self); root.pack(fill='both', expand=True)

        # Plik
        r = ttk.Frame(root); r.pack(fill='x', **pad)
        ttk.Label(r, text="Plik Excel:").pack(side='left')
        ttk.Entry(r, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r, text="Wybierz…", command=self.pick_file).pack(side='left')

        # Arkusz
        r = ttk.Frame(root); r.pack(fill='x', **pad)
        ttk.Label(r, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(r, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r, text="Wczytaj", command=self.load_sheet).pack(side='left')

        # Mapowanie kolumn
        box = ttk.LabelFrame(root, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        rr = ttk.Frame(box); rr.pack(fill='x', **pad)
        ttk.Label(rr, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(rr, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        rr = ttk.Frame(box); rr.pack(fill='x', **pad)
        ttk.Label(rr, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(rr, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        rr = ttk.Frame(box); rr.pack(fill='x', **pad)
        ttk.Label(rr, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(rr, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # Akcje
        actions = ttk.Frame(root); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd (log)", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz…", command=self.convert_and_save).pack(side='right')

        # Log
        self.log_txt = tk.Text(root, height=12)
        self.log_txt.pack(fill='both', expand=True, **pad)
        self.log_txt.configure(state='disabled')

        self._log("Wybierz plik i arkusz, potem sprawdź mapowanie kolumn. "
                  "Wynik: Agregat, Szczegóły, Pivot, Wykres (Utworzone vs Zakończone).")

    # --- log helper
    def _log(self, msg):
        self.log_txt.configure(state='normal')
        self.log_txt.insert('end', msg + "\n")
        self.log_txt.see('end')
        self.log_txt.configure(state='disabled')

    # --- plik/arkusz
    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path: return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_var.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self._log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self._log(f"[BŁĄD] Odczyt pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self._log(f"Wczytano arkusz. Kolumny: {self.columns}")

            # Comboboksy
            for combo in (self.person_combo, self.created_combo, self.completed_combo):
                combo['values'] = self.columns

            # Auto-detekcja
            person_guess = best_guess_column(self.columns,
                                             ['zasobnik', 'osoba', 'assignee', 'owner', 'wykonawca',
                                              'przypisane do', 'assigned to', 'user'])
            created_guess = best_guess_column(self.columns,
                                              ['data utworzenia', 'utworzenia', 'created', 'creation date',
                                               'created at', 'created time', 'start date', 'start'])
            completed_guess = best_guess_column(self.columns,
                                                ['data ukończenia', 'zakonczenia', 'ukończenia', 'completed',
                                                 'done', 'closed', 'finished', 'end date', 'resolution date'])

            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self._log("Sprawdź automatyczne dopasowanie kolumn i w razie potrzeby zmień ręcznie.")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self._log(f"[BŁĄD] Wczytywanie arkusza: {e}")

    # --- podgląd
    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            details, agregat = build_details_and_agregat(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            self._log("AGREGAT (top 30):")
            self._log(agregat.head(30).to_string(index=False))
            self._log("SZCZEGÓŁY (top 15):")
            self._log(details.head(15).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się przygotować podglądu:\n{e}")
            self._log(f"[BŁĄD] Podgląd: {e}")

    # --- zapis
    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            details, agregat = build_details_and_agregat(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            pivot = make_pivot_created_done(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )

            in_path = self.path_var.get().strip()
            base, _ = os.path.splitext(in_path)
            default_out = base + "_WYNIK_z_wykresem.xlsx"

            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not path:
                self._log("Anulowano zapis.")
                return

            write_excel_with_chart(
                path,
                pivot_df=pivot,
                agregat_df=agregat,
                details_df=details,
                title="ZADANIA: UTWORZONE vs ZAKOŃCZONE"
            )
            self._log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
            self._log(f"[BŁĄD] Zapis: {e}")


def main():
    # DPI-aware (Windows) – opcjonalnie
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass

    app = App()
    app.mainloop()


if __name__ == "__main__":
    main()


In [13]:
# -*- coding: utf-8 -*-
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd

APP_TITLE = "Konwerter zadań: Excel → Excel (Utworzone vs Zakończone)"
APP_GEOMETRY = "820x560"

# ---------------------------- Pomocnicze ----------------------------

EXCLUDED_BUCKETS = {"TO DO", "INSTRUKCJA", "HOLD"}

def best_guess_column(columns, candidates):
    """Heurystyczny wybór kolumny na podstawie listy słów-kluczy."""
    import unicodedata
    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s

    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]

    # pełne dopasowanie
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col

    # dopasowanie cząstkowe (najdłuższe trafienie)
    best = None
    best_len = 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best = col
                best_len = len(c)
    return best or (columns[0] if columns else None)


def detect_sheet(path):
    """Preferuje arkusze o nazwach: data/dane/zadania/sheet1/arkusz1; inaczej pierwszy."""
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names
                     if str(s).strip().lower() in {'data', 'dane', 'tasks', 'zadania', 'sheet1', 'arkusz1'}]
        return preferred[0] if preferred else xls.sheet_names[0]
    except Exception:
        return None


def normalize_person(series: pd.Series) -> pd.Series:
    """Czyści nazwy 'zasobników' (osób), zamienia NaN/None/puste na '—'."""
    ser = (series.astype(str)
                 .replace({'nan': '', 'None': ''})
                 .str.strip())
    ser = ser.where(ser != '', other='—')
    return ser

def filter_excluded(series: pd.Series) -> pd.Series:
    """Usuwa wiersze z zasobnikami: TO DO, INSTRUKCJA, HOLD (case-insensitive, po strip)."""
    up = series.str.upper().str.strip()
    mask = ~up.isin(EXCLUDED_BUCKETS)
    return mask

# ---------------------------- Logika danych ----------------------------

def build_details_and_agregat(df, col_person, col_created, col_completed):
    """
    Zwraca:
      details: osoba | data_utworzenia | data_ukonczenia | status | rok | kwartał
               (DWA wiersze jeśli są dwie daty: 'utworzone' wg daty utworzenia i 'ukończone' wg daty ukończenia)
      agregat: rok | kwartał | zasobnik | zadania_utworzone | zadania_ukończone
    """
    person = normalize_person(df[col_person])
    keep_mask = filter_excluded(person)
    person = person[keep_mask]
    created = pd.to_datetime(df[col_created][keep_mask], errors='coerce', dayfirst=True)
    completed = pd.to_datetime(df[col_completed][keep_mask], errors='coerce', dayfirst=True)

    rows = []
    for p, c, d in zip(person, created, completed):
        # UTWORZONE – każde nie-NaT w 'Data utworzenia'
        if pd.notna(c):
            rows.append({
                'osoba': p,
                'data_utworzenia': c,
                'data_ukonczenia': d if pd.notna(d) else pd.NaT,
                'status': 'utworzone',
                'rok': int(c.year),
                'kwartał': int(c.quarter)
            })
        # UKOŃCZONE – każde nie-NaT w 'Data ukończenia'
        if pd.notna(d):
            rows.append({
                'osoba': p,
                'data_utworzenia': c if pd.notna(c) else pd.NaT,
                'data_ukonczenia': d,
                'status': 'ukończone',
                'rok': int(d.year),
                'kwartał': int(d.quarter)
            })

    details = pd.DataFrame(rows, columns=['osoba','data_utworzenia','data_ukonczenia','status','rok','kwartał'])

    # Agregacja: licz UTWORZONE i UKOŃCZONE niezależnie
    utw = (details[details['status'] == 'utworzone']
           .groupby(['rok', 'kwartał', 'osoba']).size()
           .rename('zadania_utworzone'))
    uko = (details[details['status'] == 'ukończone']
           .groupby(['rok', 'kwartał', 'osoba']).size()
           .rename('zadania_ukończone'))

    agregat = (pd.merge(utw, uko, on=['rok', 'kwartał', 'osoba'], how='outer')
               .fillna(0).reset_index())
    agregat['zadania_utworzone'] = agregat['zadania_utworzone'].astype(int)
    agregat['zadania_ukończone'] = agregat['zadania_ukończone'].astype(int)

    # sort: osoba/rok/kwartał
    agregat.sort_values(by=['osoba', 'rok', 'kwartał'], inplace=True, ignore_index=True)
    details.sort_values(by=['osoba', 'rok', 'kwartał', 'status',
                            'data_utworzenia', 'data_ukonczenia'],
                        inplace=True, ignore_index=True)
    return details, agregat


def make_pivot_created_done(df, col_person, col_created, col_completed):
    """
    Pivot do wykresu (oś 3-poziomowa):
    - 'Liczba UTWORZONYCH' (wg daty utworzenia – zlicza wszystkie nie-NaT),
    - 'Liczba ZAKOŃCZONYCH' (wg daty ukończenia – zlicza wszystkie nie-NaT).
    """
    person = normalize_person(df[col_person])
    keep_mask = filter_excluded(person)
    person = person[keep_mask]
    created = pd.to_datetime(df[col_created][keep_mask], errors='coerce', dayfirst=True)
    completed = pd.to_datetime(df[col_completed][keep_mask], errors='coerce', dayfirst=True)

    created_df = (pd.DataFrame({'osoba': person, 'rok': created.dt.year, 'kwartał': created.dt.quarter})
                  .dropna(subset=['osoba', 'rok', 'kwartał'])
                  .groupby(['osoba', 'rok', 'kwartał']).size()
                  .rename('Liczba UTWORZONYCH').reset_index())

    done_df = (pd.DataFrame({'osoba': person, 'rok': completed.dt.year, 'kwartał': completed.dt.quarter})
               .dropna(subset=['rok', 'kwartał'])
               .groupby(['osoba', 'rok', 'kwartał']).size()
               .rename('Liczba ZAKOŃCZONYCH').reset_index())

    pivot = pd.merge(created_df, done_df, on=['osoba', 'rok', 'kwartał'], how='outer').fillna(0)
    pivot['Liczba UTWORZONYCH'] = pivot['Liczba UTWORZONYCH'].astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].astype(int)
    pivot.sort_values(by=['osoba', 'rok', 'kwartał'], inplace=True, ignore_index=True)
    return pivot


def write_excel_with_chart(path, pivot_df, agregat_df=None, details_df=None,
                           title="ZADANIA: UTWORZONE vs ZAKOŃCZONE"):
    """
    Zapisuje:
      - 'Agregat' (opcjonalnie),
      - 'Szczegóły' (opcjonalnie),
      - 'Pivot' (oś 3-poziomowa + wartości),
      - 'Wykres' (stacked column: Utworzone vs Zakończone).
    """
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        if agregat_df is not None:
            agregat_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        # --- Pivot (dla wykresu, multi-level categories) ---
        ws_pivot = book.add_worksheet('Pivot')
        writer.sheets['Pivot'] = ws_pivot

        if len(pivot_df) == 0:
            ws_pivot.write(0, 0, "Brak danych do wykresu")
            return

        cat_person = pivot_df['osoba'].tolist()
        cat_year = pivot_df['rok'].astype(int).tolist()
        cat_quart = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)", *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        # --- Wykres ---
        ws_chart = book.add_worksheet('Wykres')
        writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column', 'subtype': 'stacked'})

        last_col = len(pivot_df)  # kategorie: kolumny 1..last_col
        # UTWORZONE
        chart.add_series({
            'name': "='Pivot'!$A$5",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values': ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True}
        })
        # ZAKOŃCZONE
        chart.add_series({
            'name': "='Pivot'!$A$6",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values': ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True}
        })

        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})

        ws_chart.insert_chart('B2', chart, {'x_scale': 2.1, 'y_scale': 1.65})

# ---------------------------- GUI (tkinter) ----------------------------

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self._build_gui()

    def _build_gui(self):
        pad = {'padx': 10, 'pady': 6}
        root = ttk.Frame(self); root.pack(fill='both', expand=True)

        # Plik
        r = ttk.Frame(root); r.pack(fill='x', **pad)
        ttk.Label(r, text="Plik Excel:").pack(side='left')
        ttk.Entry(r, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r, text="Wybierz…", command=self.pick_file).pack(side='left')

        # Arkusz
        r = ttk.Frame(root); r.pack(fill='x', **pad)
        ttk.Label(r, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(r, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r, text="Wczytaj", command=self.load_sheet).pack(side='left')

        # Mapowanie kolumn
        box = ttk.LabelFrame(root, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        rr = ttk.Frame(box); rr.pack(fill='x', **pad)
        ttk.Label(rr, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(rr, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        rr = ttk.Frame(box); rr.pack(fill='x', **pad)
        ttk.Label(rr, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(rr, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        rr = ttk.Frame(box); rr.pack(fill='x', **pad)
        ttk.Label(rr, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(rr, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # Akcje
        actions = ttk.Frame(root); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd (log)", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz…", command=self.convert_and_save).pack(side='right')

        # Log
        self.log_txt = tk.Text(root, height=12)
        self.log_txt.pack(fill='both', expand=True, **pad)
        self.log_txt.configure(state='disabled')

        self._log("Wybierz plik i arkusz, potem sprawdź mapowanie kolumn. "
                  "Ignoruję zasobniki: TO DO, INSTRUKCJA, HOLD. "
                  "Utworzone i Ukończone liczone niezależnie.")

    # --- log helper
    def _log(self, msg):
        self.log_txt.configure(state='normal')
        self.log_txt.insert('end', msg + "\n")
        self.log_txt.see('end')
        self.log_txt.configure(state='disabled')

    # --- plik/arkusz
    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path: return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_var.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self._log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self._log(f"[BŁĄD] Odczyt pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self._log(f"Wczytano arkusz. Kolumny: {self.columns}")

            # Comboboksy
            for combo in (self.person_combo, self.created_combo, self.completed_combo):
                combo['values'] = self.columns

            # Auto-detekcja
            person_guess = best_guess_column(self.columns,
                                             ['zasobnik', 'osoba', 'assignee', 'owner', 'wykonawca',
                                              'przypisane do', 'assigned to', 'user'])
            created_guess = best_guess_column(self.columns,
                                              ['data utworzenia', 'utworzenia', 'created', 'creation date',
                                               'created at', 'created time', 'start date', 'start'])
            completed_guess = best_guess_column(self.columns,
                                                ['data ukończenia', 'zakonczenia', 'ukończenia', 'completed',
                                                 'done', 'closed', 'finished', 'end date', 'resolution date'])

            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self._log("Sprawdź automatyczne dopasowanie kolumn i w razie potrzeby zmień ręcznie.")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self._log(f"[BŁĄD] Wczytywanie arkusza: {e}")

    # --- podgląd
    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            details, agregat = build_details_and_agregat(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            self._log("AGREGAT (top 30):")
            self._log(agregat.head(30).to_string(index=False))
            self._log("SZCZEGÓŁY (top 20):")
            self._log(details.head(20).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się przygotować podglądu:\n{e}")
            self._log(f"[BŁĄD] Podgląd: {e}")

    # --- zapis
    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            details, agregat = build_details_and_agregat(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )
            pivot = make_pivot_created_done(
                self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get()
            )

            in_path = self.path_var.get().strip()
            base, _ = os.path.splitext(in_path)
            default_out = base + "_WYNIK_z_wykresem.xlsx"

            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not path:
                self._log("Anulowano zapis.")
                return

            write_excel_with_chart(
                path,
                pivot_df=pivot,
                agregat_df=agregat,
                details_df=details,
                title="ZADANIA: UTWORZONE vs ZAKOŃCZONE"
            )
            self._log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
            self._log(f"[BŁĄD] Zapis: {e}")


def main():
    # DPI-aware (Windows) – opcjonalnie
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass

    app = App()
    app.mainloop()


if __name__ == "__main__":
    main()


In [14]:
import os
import re
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import numpy as np
import pandas as pd

APP_TITLE = "Konwerter zadań: Excel → Excel (Utworzone/Zakończone – rok/kwartał/osoba)"
APP_GEOMETRY = "820x560"

# ====== Parser dat: PL + ISO + seriale Excela ======

EMPTY_MARKERS = {'', '-', 'pusta', 'brak', 'none', 'nan', 'nat'}

def _clean_text_date(x: str) -> str:
    s = str(x).strip()
    s = s.replace('\u00a0', ' ')  # NBSP
    s = s.replace(',', '.')      # czasem 21,08,2025
    s = s.replace('/', '.').replace('-', '.')
    s = re.sub(r'\s+', ' ', s)
    return s

def smart_parse_date_pl(series: pd.Series) -> pd.Series:
    """
    Solidny parser dat:
    - rozpoznaje serial Excela (int/float 10000..60000),
    - 'pusta', '-', itp. → NaT,
    - PL: dd.mm.rrrr, dd.mm.rr (+ opcjonalnie HH:MM),
    - fallback: to_datetime(dayfirst=True).
    """
    s = series.copy()

    # 1) serial Excela
    is_num = pd.to_numeric(s, errors='coerce')
    serial_mask = is_num.between(10000, 60000)  # daty ~1938–2064
    out = pd.Series(pd.NaT, index=s.index, dtype='datetime64[ns]')
    if serial_mask.any():
        out.loc[serial_mask] = pd.to_datetime(
            is_num[serial_mask], unit='D', origin='1899-12-30', errors='coerce'
        )

    # 2) tekst
    text_mask = ~serial_mask
    if text_mask.any():
        txt = s[text_mask].astype(str).map(_clean_text_date)
        empt = txt.str.lower().isin(EMPTY_MARKERS)
        txt = txt.mask(empt, np.nan)

        tried = pd.to_datetime(txt, format='%d.%m.%Y', errors='coerce')
        still = tried.isna()

        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%y', errors='coerce')
            still = tried.isna()

        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%Y %H:%M', errors='coerce')
            still = tried.isna()

        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%y %H:%M', errors='coerce')
            still = tried.isna()

        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], errors='coerce', dayfirst=True)

        out.loc[text_mask] = tried

    return out

# ====== Pomocnicze ======

def best_guess_column(columns, candidates):
    """Najlepsze dopasowanie nazwy kolumny (bez polskich znaków, spacji, itp.)."""
    import unicodedata
    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s
    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col
    best, best_len = None, 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best, best_len = col, len(c)
    return best or (columns[0] if columns else None)

def detect_sheet(path):
    """Wybierz najbardziej sensowny arkusz."""
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names
                     if str(s).strip().lower() in {'data','dane','tasks','zadania','sheet1','arkusz1'}]
        return preferred[0] if preferred else xls.sheet_names[0]
    except Exception:
        return None

# ====== Logika liczenia ======

EXCLUDE_BUCKETS = {"TO DO", "INSTRUKCJA", "HOLD"}

def _normalize_person_col(s: pd.Series) -> pd.Series:
    """Czyści kolumnę osoby/zasobnika i odrzuca 'TO DO/INSTRUKCJA/HOLD'."""
    person = (s.astype(str)
                .replace({'nan':'', 'None':''})
                .str.strip()
                .replace('', '—'))
    keep = ~person.str.upper().isin(EXCLUDE_BUCKETS)
    return person.where(keep).dropna()

def make_chart_df(df, col_person, col_created, col_completed):
    """
    Pivot pod wykres:
      osoba | rok | kwartał | Liczba UTWORZONYCH | Liczba ZAKOŃCZONYCH
    (utworzone wg daty utworzenia; zakończone wg daty ukończenia)
    """
    # Filtr osób (bez 'TO DO', 'INSTRUKCJA', 'HOLD')
    person_raw = (df[col_person].astype(str).replace({'nan':'','None':''}).str.strip().replace('', '—'))
    keep = ~person_raw.str.upper().isin(EXCLUDE_BUCKETS)
    person = person_raw[keep]

    created   = smart_parse_date_pl(df[col_created][keep])
    completed = smart_parse_date_pl(df[col_completed][keep])

    # UTWORZONE
    created_df = (pd.DataFrame({
                        'osoba': person,
                        'rok': created.dt.year,
                        'kwartał': created.dt.quarter
                    })
                    .dropna(subset=['osoba','rok','kwartał'])
                    .groupby(['osoba','rok','kwartał']).size()
                    .rename('Liczba UTWORZONYCH').reset_index())

    # ZAKOŃCZONE
    done_df = (pd.DataFrame({
                        'osoba': person,
                        'rok': completed.dt.year,
                        'kwartał': completed.dt.quarter
                    })
                    .dropna(subset=['rok','kwartał'])
                    .groupby(['osoba','rok','kwartał']).size()
                    .rename('Liczba ZAKOŃCZONYCH').reset_index())

    pivot = pd.merge(created_df, done_df, on=['osoba','rok','kwartał'], how='outer')
    pivot['Liczba UTWORZONYCH']  = pivot['Liczba UTWORZONYCH'].fillna(0).astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].fillna(0).astype(int)

    pivot = pivot.sort_values(by=['osoba','rok','kwartał'],
                              key=lambda s: s.str.casefold() if s.name=='osoba' else s).reset_index(drop=True)
    return pivot

def _sort_key_person(series):
    return series.astype(str).fillna('').str.casefold()

def aggregate_and_details(df, col_person, col_created, col_completed):
    """
    Zwraca:
      - out (agregat): rok | kwartał | zasobnik | zadania_ukończone | zadania_nieukończone
      - details: osoba | data_utworzenia | data_ukonczenia | status | rok | kwartał
        (dla 'ukończone' rok/kwartał z daty ukończenia; dla 'nieukończone' z daty utworzenia)
    """
    person_raw = (df[col_person].astype(str).replace({'nan':'','None':''}).str.strip().replace('', '—'))
    keep = ~person_raw.str.upper().isin(EXCLUDE_BUCKETS)
    person = person_raw[keep]

    created   = smart_parse_date_pl(df[col_created][keep])
    completed = smart_parse_date_pl(df[col_completed][keep])

    # --- AGREGAT ---
    done = (pd.DataFrame({
                'rok': completed.dt.year,
                'kwartał': completed.dt.quarter,
                'zasobnik': person,
            })
            .dropna(subset=['rok','kwartał'])
            .groupby(['rok','kwartał','zasobnik'], dropna=False).size()
            .rename('zadania_ukończone').reset_index())

    not_done_mask = completed.isna()
    not_done_created = created.where(not_done_mask)
    not_done = (pd.DataFrame({
                    'rok': not_done_created.dt.year,
                    'kwartał': not_done_created.dt.quarter,
                    'zasobnik': person.where(not_done_mask),
                })
                .dropna(subset=['rok','kwartał','zasobnik'])
                .groupby(['rok','kwartał','zasobnik'], dropna=False).size()
                .rename('zadania_nieukończone').reset_index())

    out = pd.merge(done, not_done, on=['rok','kwartał','zasobnik'], how='outer')
    out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
    out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)
    out.sort_values(by=['zasobnik','rok','kwartał'],
                    key=lambda c: _sort_key_person(c) if c.name=='zasobnik' else c, inplace=True)
    out.reset_index(drop=True, inplace=True)

    # --- SZCZEGÓŁY ---
    det_done = (pd.DataFrame({
                    'osoba': person,
                    'data_utworzenia': created,
                    'data_ukonczenia': completed,
                    'status': pd.Series(['ukończone']*len(person), index=person.index),
                    'rok': completed.dt.year,
                    'kwartał': completed.dt.quarter,
                }).dropna(subset=['rok','kwartał']))

    det_not_done = (pd.DataFrame({
                        'osoba': person.where(not_done_mask),
                        'data_utworzenia': created.where(not_done_mask),
                        'data_ukonczenia': completed.where(not_done_mask),
                        'status': pd.Series(['nieukończone']*len(person), index=person.index).where(not_done_mask),
                        'rok': not_done_created.dt.year,
                        'kwartał': not_done_created.dt.quarter,
                    }).dropna(subset=['osoba','rok','kwartał']))

    details = pd.concat([det_done, det_not_done], ignore_index=True)
    details.sort_values(by=['osoba','rok','kwartał','data_utworzenia','data_ukonczenia'],
                        key=lambda c: _sort_key_person(c) if c.name=='osoba' else c, inplace=True)
    details.reset_index(drop=True, inplace=True)

    return out, details

# ====== Excel + wykres (xlsxwriter) ======

def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None,
                           title="ZADANIA: UTWORZONE vs ZAKOŃCZONE"):
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot
        cat_person = pivot_df['osoba'].tolist()
        cat_year   = pivot_df['rok'].astype(int).tolist()
        cat_quart  = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)",   *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column', 'subtype': 'stacked'})

        last_col = len(pivot_df)
        chart.add_series({
            'name':       "='Pivot'!$A$5",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#43A047'},
            'border': {'color': '#43A047'},
        })
        chart.add_series({
            'name':       "='Pivot'!$A$6",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#1E88E5'},
            'border': {'color': '#1E88E5'},
        })
        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})

        ws_chart.insert_chart('B2', chart, {'x_scale': 2.0, 'y_scale': 1.6})

# ====== GUI (Tkinter) ======

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()
        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self.create_widgets()

    def create_widgets(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self)
        frm.pack(fill='both', expand=True)

        row = ttk.Frame(frm); row.pack(fill='x', **pad)
        ttk.Label(row, text="Plik Excel:").pack(side='left')
        ttk.Entry(row, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row, text="Wybierz...", command=self.pick_file).pack(side='left')

        row2 = ttk.Frame(frm); row2.pack(fill='x', **pad)
        ttk.Label(row2, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(row2, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(row2, text="Wczytaj", command=self.load_sheet).pack(side='left')

        box = ttk.LabelFrame(frm, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        r1 = ttk.Frame(box); r1.pack(fill='x', **pad)
        ttk.Label(r1, text="Nazwa zasobnika (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r1, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r2 = ttk.Frame(box); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r2, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r3 = ttk.Frame(box); r3.pack(fill='x', **pad)
        ttk.Label(r3, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r3, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz...", command=self.convert_and_save).pack(side='right')

        self.status = tk.Text(frm, height=12)
        self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')

        self.log("Wybierz plik i arkusz, sprawdź mapowanie. Liczę: Utworzone wg daty utworzenia, "
                 "Zakończone wg daty ukończenia. Filtr: TO DO / INSTRUKCJA / HOLD → pomijane.")

    # --- helpers ---

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path:
            return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_var.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.log(f"Znalezione arkusze: {xls.sheet_names}. Wybrano: {self.sheet_var.get()}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się odczytać pliku:\n{e}")
            self.log(f"Błąd odczytu pliku: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self.log(f"Wczytano arkusz. Kolumny: {self.columns}")

            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns

            person_guess = best_guess_column(self.columns, [
                'nazwa zasobnika', 'osoba', 'assignee', 'owner', 'wykonawca', 'przypisane do', 'assigned to', 'user'
            ])
            created_guess = best_guess_column(self.columns, [
                'data utworzenia','utworzenia','created','creation date','created at','start date','start'
            ])
            completed_guess = best_guess_column(self.columns, [
                'data ukończenia','data zakonczenia','zakonczenia','completed','done','closed','end date','resolution date'
            ])

            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)

            self.log("Sprawdź automatyczne dopasowanie kolumn i popraw w razie potrzeby.")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self.log(f"Błąd wczytywania arkusza: {e}")

    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            # diagnostyka parsowania
            cr = smart_parse_date_pl(self.df[self.created_var.get()])
            cm = smart_parse_date_pl(self.df[self.completed_var.get()])
            self.log(f"Diagnostyka dat → utworzenia NaT={int(cr.isna().sum())}/{len(cr)}, "
                     f"ukończenia NaT={int(cm.isna().sum())}/{len(cm)}")

            out, details = aggregate_and_details(self.df,
                                                 self.person_var.get(),
                                                 self.created_var.get(),
                                                 self.completed_var.get())
            top = out.head(50)
            self.log(f"AGREGAT (pierwsze {len(top)} wierszy):\n{top.to_string(index=False)}")
            self.log("Szczegóły (10 pierwszych):")
            self.log(details.head(10).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się przygotować podglądu:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            out, details = aggregate_and_details(self.df,
                                                 self.person_var.get(),
                                                 self.created_var.get(),
                                                 self.completed_var.get())
            pivot = make_chart_df(self.df,
                                  self.person_var.get(),
                                  self.created_var.get(),
                                  self.completed_var.get())

            in_path = self.path_var.get().strip()
            base, _ = os.path.splitext(in_path)
            default_out = base + "_WYNIK_z_wykresem.xlsx"

            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not path:
                self.log("Anulowano zapis.")
                return

            write_excel_with_chart(path, pivot_df=pivot, aggr_df=out, details_df=details,
                                   title="ZADANIA: UTWORZONE vs ZAKOŃCZONE")
            self.log(f"Zapisano: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się zapisać pliku:\n{e}")
            self.log(f"Błąd zapisu: {e}")

# ====== MAIN ======

def main():
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()

if __name__ == "__main__":
    main()


In [1]:
import os
import re
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import numpy as np
import pandas as pd

APP_TITLE = "Konwerter: Utworzone vs Zakończone (rok/kwartał/osoba)"
APP_GEOMETRY = "860x580"

EXCLUDE_BUCKETS = {"TO DO", "INSTRUKCJA", "HOLD"}
EMPTY_MARKERS = {'', '-', 'pusta', 'brak', 'none', 'nan', 'nat'}

# ---------- PARSER DAT: PL + ISO + SERIAL EXCEL ----------
def _clean_text_date(x: str) -> str:
    s = str(x).strip().replace('\u00a0', ' ')
    s = re.sub(r'\s+', ' ', s)
    return s

def smart_parse_date(series: pd.Series) -> pd.Series:
    """Czyta: seriale Excela, dd.mm.rrrr[ HH:MM[:SS]], dd-mm-rrrr, ISO, itp."""
    s = series.copy()

    out = pd.Series(pd.NaT, index=s.index, dtype='datetime64[ns]')

    # Serial Excela (dni od 1899-12-30)
    as_num = pd.to_numeric(s, errors='coerce')
    mask_serial = as_num.between(10000, 60000)  # ~1938–2064
    if mask_serial.any():
        out.loc[mask_serial] = pd.to_datetime(as_num[mask_serial], unit='D',
                                              origin='1899-12-30', errors='coerce')

    # Teksty
    mask_text = ~mask_serial
    if mask_text.any():
        txt = s[mask_text].astype(str).map(_clean_text_date)
        low = txt.str.lower()
        txt = txt.mask(low.isin(EMPTY_MARKERS), np.nan)

        tried = pd.to_datetime(txt, format='%d.%m.%Y', errors='coerce')
        still = tried.isna()

        # dd.mm.YYYY HH:MM i HH:MM:SS
        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%Y %H:%M', errors='coerce')
            still = tried.isna()
        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%Y %H:%M:%S', errors='coerce')
            still = tried.isna()

        # dd.mm.yy warianty
        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%y', errors='coerce')
            still = tried.isna()
        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%y %H:%M', errors='coerce')
            still = tried.isna()
        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], format='%d.%m.%y %H:%M:%S', errors='coerce')
            still = tried.isna()

        # ISO i inne (najpierw bez dayfirst, potem z dayfirst)
        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], errors='coerce', dayfirst=False)
            still = tried.isna()
        if still.any():
            tried.loc[still] = pd.to_datetime(txt[still], errors='coerce', dayfirst=True)

        out.loc[mask_text] = tried

    return out

# ---------- WSPÓLNE PRZYGOTOWANIE DANYCH ----------
SPLIT_REGEX = re.compile(r'[;,/|&+]|(?<!\S)i(?!\S)', flags=re.IGNORECASE)

def preprocess(df: pd.DataFrame, col_person: str, col_created: str, col_completed: str) -> pd.DataFrame:
    """
    Zwraca ramkę: ['osoba','created','completed']
    - rozbicie wielu osób na wiersze,
    - filtruje TO DO/INSTRUKCJA/HOLD,
    - parsuje daty.
    """
    tmp = pd.DataFrame({
        'raw_person': df[col_person],
        'raw_created': df[col_created],
        'raw_completed': df[col_completed],
    }).copy()

    # split osób
    tmp['osoba'] = (tmp['raw_person'].astype(str)
                    .str.replace('\u00a0', ' ')
                    .str.replace(r'\s+', ' ', regex=True)
                    .str.strip()
                    .str.split(SPLIT_REGEX))
    tmp = tmp.explode('osoba')
    tmp['osoba'] = tmp['osoba'].astype(str).str.strip()
    tmp = tmp[ tmp['osoba'].ne('') ]

    # odrzuć bucket-y pomocnicze
    tmp = tmp[ ~tmp['osoba'].str.upper().isin(EXCLUDE_BUCKETS) ]

    # parse daty
    tmp['created']   = smart_parse_date(tmp['raw_created'])
    tmp['completed'] = smart_parse_date(tmp['raw_completed'])

    return tmp[['osoba','created','completed']].reset_index(drop=True)

# ---------- TABELKI LOGICZNE ----------
def make_pivot(df, col_person, col_created, col_completed) -> pd.DataFrame:
    base = preprocess(df, col_person, col_created, col_completed)

    created_df = (base.assign(rok = base['created'].dt.year,
                              kwartał = base['created'].dt.quarter)
                       .dropna(subset=['rok','kwartał'])
                       .groupby(['osoba','rok','kwartał']).size()
                       .rename('Liczba UTWORZONYCH').reset_index())

    done_df = (base.assign(rok = base['completed'].dt.year,
                           kwartał = base['completed'].dt.quarter)
                    .dropna(subset=['rok','kwartał'])
                    .groupby(['osoba','rok','kwartał']).size()
                    .rename('Liczba ZAKOŃCZONYCH').reset_index())

    pivot = pd.merge(created_df, done_df, on=['osoba','rok','kwartał'], how='outer')
    pivot['Liczba UTWORZONYCH']  = pivot['Liczba UTWORZONYCH'].fillna(0).astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].fillna(0).astype(int)

    # usuwamy ewentualne rządki 0/0 (nie powinny się pojawić, ale na wszelki wypadek)
    pivot = pivot[(pivot['Liczba UTWORZONYCH'] > 0) | (pivot['Liczba ZAKOŃCZONYCH'] > 0)]

    pivot = pivot.sort_values(
        by=['osoba','rok','kwartał'],
        key=lambda s: s.str.casefold() if s.name=='osoba' else s
    ).reset_index(drop=True)
    return pivot

def aggregate_and_details(df, col_person, col_created, col_completed):
    base = preprocess(df, col_person, col_created, col_completed)

    # AGREGAT
    done = (base.assign(rok = base['completed'].dt.year,
                        kwartał = base['completed'].dt.quarter,
                        zasobnik = base['osoba'])
                 .dropna(subset=['rok','kwartał'])
                 .groupby(['rok','kwartał','zasobnik']).size()
                 .rename('zadania_ukończone').reset_index())

    nd_mask = base['completed'].isna()
    not_done = (base.loc[nd_mask].assign(
                    rok = base.loc[nd_mask, 'created'].dt.year,
                    kwartał = base.loc[nd_mask, 'created'].dt.quarter,
                    zasobnik = base.loc[nd_mask, 'osoba'])
                .dropna(subset=['rok','kwartał','zasobnik'])
                .groupby(['rok','kwartał','zasobnik']).size()
                .rename('zadania_nieukończone').reset_index())

    out = pd.merge(done, not_done, on=['rok','kwartał','zasobnik'], how='outer')
    out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
    out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)
    out = out.sort_values(by=['zasobnik','rok','kwartał'],
                          key=lambda c: c.str.casefold() if c.name=='zasobnik' else c)\
             .reset_index(drop=True)

    # SZCZEGÓŁY
    det_done = (base.assign(status='ukończone',
                            rok = base['completed'].dt.year,
                            kwartał = base['completed'].dt.quarter)
                     .dropna(subset=['rok','kwartał']))

    det_not = (base.loc[nd_mask].assign(
                    status='nieukończone',
                    rok = base.loc[nd_mask, 'created'].dt.year,
                    kwartał = base.loc[nd_mask, 'created'].dt.quarter)
               .dropna(subset=['rok','kwartał']))

    details = pd.concat([det_done, det_not], ignore_index=True)[
        ['osoba','created','completed','status','rok','kwartał']
    ].rename(columns={'created':'data_utworzenia','completed':'data_ukonczenia'})\
     .sort_values(by=['osoba','rok','kwartał','data_utworzenia','data_ukonczenia'],
                  key=lambda c: c.str.casefold() if c.name=='osoba' else c)\
     .reset_index(drop=True)

    return out, details

# ---------- ZAPIS EXCEL + WYKRES ----------
def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None,
                           title="ZADANIA: UTWORZONE vs ZAKOŃCZONE"):
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book

        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot

        cat_person = pivot_df['osoba'].tolist()
        cat_year   = pivot_df['rok'].astype(int).tolist()
        cat_quart  = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)",   *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column', 'subtype': 'stacked'})

        last_col = len(pivot_df)
        chart.add_series({
            'name':       "='Pivot'!$A$5",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#43A047'},
            'border': {'color': '#43A047'},
        })
        chart.add_series({
            'name':       "='Pivot'!$A$6",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#1E88E5'},
            'border': {'color': '#1E88E5'},
        })
        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})

        ws_chart.insert_chart('B2', chart, {'x_scale': 2.0, 'y_scale': 1.6})

# ---------- GUI ----------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_var = tk.StringVar()
        self.sheet_var = tk.StringVar()
        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()

        self.df = None
        self.columns = []

        self._build_ui()

    def _build_ui(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self); frm.pack(fill='both', expand=True)

        # plik
        r = ttk.Frame(frm); r.pack(fill='x', **pad)
        ttk.Label(r, text="Plik Excel:").pack(side='left')
        ttk.Entry(r, textvariable=self.path_var).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r, text="Wybierz…", command=self.pick_file).pack(side='left')

        # arkusz
        r = ttk.Frame(frm); r.pack(fill='x', **pad)
        ttk.Label(r, text="Arkusz:").pack(side='left')
        self.sheet_combo = ttk.Combobox(r, textvariable=self.sheet_var, state='readonly')
        self.sheet_combo.pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r, text="Wczytaj", command=self.load_sheet).pack(side='left')

        # mapowanie
        box = ttk.LabelFrame(frm, text="Mapowanie kolumn"); box.pack(fill='x', **pad)

        r = ttk.Frame(box); r.pack(fill='x', **pad)
        ttk.Label(r, text="Nazwa zasobnika (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r, textvariable=self.person_var, state='readonly')
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r = ttk.Frame(box); r.pack(fill='x', **pad)
        ttk.Label(r, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r, textvariable=self.created_var, state='readonly')
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r = ttk.Frame(box); r.pack(fill='x', **pad)
        ttk.Label(r, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r, textvariable=self.completed_var, state='readonly')
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # akcje
        act = ttk.Frame(frm); act.pack(fill='x', **pad)
        ttk.Button(act, text="Podgląd", command=self.preview).pack(side='right', padx=6)
        ttk.Button(act, text="Konwertuj i zapisz…", command=self.convert_and_save).pack(side='right')

        # log
        self.logbox = tk.Text(frm, height=13); self.logbox.pack(fill='both', expand=True, **pad)
        self.logbox.configure(state='disabled')
        self.log("Liczę: UTWORZONE wg daty utworzenia, ZAKOŃCZONE wg daty ukończenia. "
                 "Rozbijam wiele osób. Filtruję: TO DO / INSTRUKCJA / HOLD.")

    def log(self, msg):
        self.logbox.configure(state='normal')
        self.logbox.insert('end', msg + "\n")
        self.logbox.see('end')
        self.logbox.configure(state='disabled')

    def pick_file(self):
        path = filedialog.askopenfilename(
            title="Wybierz plik Excel",
            filetypes=[("Excel files", "*.xlsx *.xls *.xlsm"), ("All files", "*.*")]
        )
        if not path: return
        self.path_var.set(path)
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo['values'] = xls.sheet_names
            self.sheet_var.set(xls.sheet_names[0] if xls.sheet_names else "")
            self.log(f"Arkusze: {xls.sheet_names}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie można odczytać pliku:\n{e}")
            self.log(f"Błąd odczytu: {e}")

    def load_sheet(self):
        path = self.path_var.get().strip()
        if not path:
            messagebox.showwarning("Uwaga", "Najpierw wybierz plik.")
            return
        try:
            self.df = pd.read_excel(path, sheet_name=self.sheet_var.get() or 0)
            self.columns = list(self.df.columns)
            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns

            # auto-sugestie
            def guess(cols, cands):
                import unicodedata
                def norm(s):
                    s = str(s)
                    s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
                    s = s.lower()
                    for ch in ['-','_',' ','/','\\','.','(',')','[',']',':',';','|']:
                        s = s.replace(ch,'')
                    return s
                cn = {c:norm(c) for c in cols}
                cand = [norm(c) for c in cands]
                for k,v in cn.items():
                    if v in cand: return k
                best,bl=None,0
                for k,v in cn.items():
                    for c in cand:
                        if c in v and len(c)>bl:
                            best,bl=k,len(c)
                return best or (cols[0] if cols else '')
            p = guess(self.columns, ['nazwa zasobnika','osoba','assignee','owner','przypisane do'])
            c = guess(self.columns, ['data utworzenia','utworzenia','created','created at','start date'])
            d = guess(self.columns, ['data ukończenia','zakonczenia','ukończenia','completed','end date','resolution date'])
            if p: self.person_var.set(p)
            if c: self.created_var.set(c)
            if d: self.completed_var.set(d)

            self.log(f"Wczytano. Kolumny: {self.columns}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać arkusza:\n{e}")
            self.log(f"Błąd: {e}")

    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            piv = make_pivot(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            out, det = aggregate_and_details(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            self.log(f"Pivot (pierwsze 10):\n{piv.head(10).to_string(index=False)}")
            self.log(f"Agregat (pierwsze 10):\n{out.head(10).to_string(index=False)}")
            self.log(f"Szczegóły (pierwsze 10):\n{det.head(10).to_string(index=False)}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Podgląd nieudany:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj arkusz.")
            return
        try:
            piv = make_pivot(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            out, det = aggregate_and_details(self.df, self.person_var.get(), self.created_var.get(), self.completed_var.get())
            in_path = self.path_var.get().strip()
            base, _ = os.path.splitext(in_path)
            default_out = base + "_WYNIK_z_wykresem.xlsx"
            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako", defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files", "*.xlsx")]
            )
            if not path:
                self.log("Anulowano.")
                return
            write_excel_with_chart(path, piv, out, det, "ZADANIA: UTWORZONE vs ZAKOŃCZONE")
            self.log(f"Zapisano: {path}")
            messagebox.showinfo("OK", f"Zapisano do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Zapis nieudany:\n{e}")
            self.log(f"Błąd zapisu: {e}")

def main():
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()

if __name__ == "__main__":
    main()


In [7]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from datetime import datetime
from typing import Optional  # ← ważne

APP_TITLE = "Konwerter zadań: Excel → Excel (rok/kwartał/zasobnik)"
APP_GEOMETRY = "820x560"

# ←← USTAW ŚCIEŻKĘ DO DOMYŚLNEGO PLIKU (Excel #1)
DEFAULT_MASTER_PATH = r"C:\Sciezka\do\Excel1_master.xlsx"

# ==== KONFIG – data „fałszywa” z nowego systemu (Panasonic) ==================
SENTINEL_CREATED = datetime(2025, 4, 10).date()   # 10.04.2025

# ====== Pomocnicze ============================================================

def best_guess_column(columns, candidates):
    import unicodedata
    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s
    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col
    best, best_len = None, 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best, best_len = col, len(c)
    return best or (columns[0] if columns else None)

def detect_sheet(path):
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names
                     if str(s).strip().lower() in {'data','dane','tasks','zadania','sheet1','arkusz1'}]
        return preferred[0] if preferred else xls.sheet_names[0]
    except Exception:
        return None

def norm_text(s: pd.Series) -> pd.Series:
    return (s.astype(str)
              .str.strip()
              .str.replace(r'\s+', ' ', regex=True)
              .str.replace('\u200b', '', regex=False)  # zero-width space
              .replace({'nan':'', 'None':''}))

def smart_datetime(s: pd.Series) -> pd.Series:
    """Solidne parsowanie dat (PL dd.mm.rrrr i ISO). Zwraca datetime64[ns]."""
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    # dopalacz dla  dd.mm.yyyy HH:MM  /  dd.mm.yyyy
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        vals = s[mask].astype(str).str.strip()
        dt2 = pd.to_datetime(vals, format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(vals[still], format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt

def norm_date_string(s: pd.Series) -> pd.Series:
    dt = smart_datetime(s)
    return dt.dt.strftime('%Y-%m-%d').fillna('')

def filter_bins(person: pd.Series) -> pd.Series:
    up = person.str.upper()
    return ~(up.isin(["TO DO", "INSTRUKCJA", "HOLD"]))

def make_chart_df(df, col_person, col_created, col_completed):
    # Osoba/rok/kwartał + 2 serie: utworzone (wg Data utworzenia), zakończone (wg Data ukończenia)
    person = norm_text(df[col_person]).replace('', '—')
    keep = filter_bins(person)
    person = person[keep]
    created = smart_datetime(df[col_created])[keep]
    completed = smart_datetime(df[col_completed])[keep]

    created_df = (
        pd.DataFrame({'osoba': person, 'rok': created.dt.year, 'kwartał': created.dt.quarter})
        .dropna(subset=['osoba','rok','kwartał'])
        .groupby(['osoba','rok','kwartał']).size()
        .rename('Liczba UTWORZONYCH').reset_index()
    )
    done_df = (
        pd.DataFrame({'osoba': person, 'rok': completed.dt.year, 'kwartał': completed.dt.quarter})
        .dropna(subset=['rok','kwartał'])
        .groupby(['osoba','rok','kwartał']).size()
        .rename('Liczba ZAKOŃCZONYCH').reset_index()
    )
    pivot = pd.merge(created_df, done_df, on=['osoba','rok','kwartał'], how='outer')
    pivot['Liczba UTWORZONYCH']  = pivot['Liczba UTWORZONYCH'].fillna(0).astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].fillna(0).astype(int)
    pivot = pivot.sort_values(by=['osoba','rok','kwartał'],
                              key=lambda s: s.str.casefold() if s.name=='osoba' else s).reset_index(drop=True)
    return pivot

def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None, title="ZADANIE UTWORZONE DO ZAKOŃCZONE"):
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book
        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        # PIVOT (oś 3-poziomowa)
        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot
        cat_person = pivot_df['osoba'].tolist()
        cat_year   = pivot_df['rok'].astype(int).tolist()
        cat_quart  = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()
        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)",   *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])
        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        # Wykres
        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart
        chart = book.add_chart({'type': 'column','subtype': 'stacked'})
        last_col = len(pivot_df)
        chart.add_series({
            'name': "='Pivot'!$A$5",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#43A047'}, 'border': {'color': '#43A047'},
        })
        chart.add_series({
            'name': "='Pivot'!$A$6",
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#1E88E5'}, 'border': {'color': '#1E88E5'},
        })
        chart.set_title({'name': title})
        chart.set_legend({'position': 'top'})
        chart.set_y_axis({'major_gridlines': {'visible': True}})
        ws_chart.insert_chart('B2', chart, {'x_scale': 2.0, 'y_scale': 1.6})

# ====== Budowa klucza unikalności (anty-duplikacja) ===========================

def build_unique_key(df: pd.DataFrame,
                     col_person: str,
                     col_task: Optional[str],
                     col_created: str,
                     col_completed: str,
                     id_col: str = "Identyfikator zadania") -> pd.Series:
    """
    Klucz unikalności:
      1) Jeśli istnieje kolumna z ID – użyj jej.
      2) W przeciwnym razie:
         - bazowo: (osoba, nazwa_zadania, data_utworzenia)
         - jeśli data_utworzenia == 12.04.2025 (SENTINEL) → POMIŃ ją w kluczu
           i dołóż data_ukończenia (jeśli jest), aby rozróżnić różne zadania.
         - jeśli brak kolumny nazwy, użyj (osoba, [data_utworzenia?], [data_ukończenia?]) z powyższą regułą.
    """
    cols = list(df.columns)
    if id_col in cols:
        return norm_text(df[id_col])

    person = norm_text(df.get(col_person, pd.Series(['']*len(df)))).str.lower()
    task   = norm_text(df.get(col_task,   pd.Series(['']*len(df)))).str.lower() if col_task else pd.Series(['']*len(df))
    created_dt   = smart_datetime(df.get(col_created,   pd.Series(['']*len(df))))
    completed_dt = smart_datetime(df.get(col_completed, pd.Series(['']*len(df))))

    created_date   = created_dt.dt.date
    completed_date = completed_dt.dt.date

    created_str   = created_date.astype(str).where(pd.notna(created_date), "")
    completed_str = completed_date.astype(str).where(pd.notna(completed_date), "")

    use_created = ~created_date.eq(SENTINEL_CREATED)

    key = (
        person + "||" +
        task.fillna("") + "||" +
        created_str.where(use_created, "") + "||" +
        completed_str.where(use_created, completed_str)
    )
    return key

# ====== GUI ===================================================================

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_master = tk.StringVar(value=DEFAULT_MASTER_PATH)
        self.path_user   = tk.StringVar()

        self.sheet_master = tk.StringVar()
        self.sheet_user   = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()
        self.task_var = tk.StringVar()  # (opcjonalnie) nazwa zadania do klucza

        self.df_master = None   # Excel #1
        self.df = None          # scalony master + user
        self.columns = []

        self.create_widgets()
        self.load_master_on_start()

    # --- UI ---
    def create_widgets(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self); frm.pack(fill='both', expand=True)

        # MASTER (auto)
        row0 = ttk.LabelFrame(frm, text="Excel #1 (domyślny – ładowany automatycznie)")
        row0.pack(fill='x', **pad)
        r0 = ttk.Frame(row0); r0.pack(fill='x', **pad)
        ttk.Entry(r0, textvariable=self.path_master).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r0, text="Zmień…", command=self.pick_master).pack(side='left')
        r0b = ttk.Frame(row0); r0b.pack(fill='x', **pad)
        ttk.Label(r0b, text="Arkusz:").pack(side='left')
        self.sheet_combo_master = ttk.Combobox(r0b, textvariable=self.sheet_master, state='readonly', width=30)
        self.sheet_combo_master.pack(side='left', padx=6)
        ttk.Button(r0b, text="Przeładuj Excel #1", command=self.load_master_on_start).pack(side='left')

        # USER (scalanie)
        row1 = ttk.LabelFrame(frm, text="Excel #2 (od użytkownika – nowe wiersze zostaną dopisane)")
        row1.pack(fill='x', **pad)
        r1 = ttk.Frame(row1); r1.pack(fill='x', **pad)
        ttk.Entry(r1, textvariable=self.path_user).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r1, text="➕ Wczytaj plik 2 i scal", command=self.load_and_merge_user).pack(side='left')

        # Mapowanie
        box = ttk.LabelFrame(frm, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        r_task = ttk.Frame(box); r_task.pack(fill='x', **pad)
        ttk.Label(r_task, text="Nazwa zadania (opcjonalnie, do klucza):").pack(side='left')
        self.task_combo = ttk.Combobox(r_task, textvariable=self.task_var, state='readonly', width=40)
        self.task_combo.pack(side='left', fill='x', expand=True, padx=6)

        r2 = ttk.Frame(box); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r2, textvariable=self.person_var, state='readonly', width=40)
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r3 = ttk.Frame(box); r3.pack(fill='x', **pad)
        ttk.Label(r3, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r3, textvariable=self.created_var, state='readonly', width=40)
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r4 = ttk.Frame(box); r4.pack(fill='x', **pad)
        ttk.Label(r4, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r4, textvariable=self.completed_var, state='readonly', width=40)
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # Akcje
        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz…", command=self.convert_and_save).pack(side='right')

        # Log
        self.status = tk.Text(frm, height=10); self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')

        self.log("Start: wczytam Excel #1 (master). Potem doładuj Excel #2 – dopiszę tylko nowe wiersze.")

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    # --- Master load ---
    def pick_master(self):
        path = filedialog.askopenfilename(title="Wybierz plik Excel #1 (master)",
                                          filetypes=[("Excel files","*.xlsx *.xls *.xlsm")])
        if path:
            self.path_master.set(path)
            self.load_master_on_start()

    def load_master_on_start(self):
        path = self.path_master.get().strip()
        if not path or not os.path.isfile(path):
            self.log("Brak/nieprawidłowa ścieżka Excel #1 – pomiń wczytanie.")
            return
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo_master['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_master.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.df_master = pd.read_excel(path, sheet_name=self.sheet_master.get() or 0)
            self.df = self.df_master.copy()  # na starcie zestaw = master
            self.columns = list(self.df_master.columns)

            # mapowanie
            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns
            self.task_combo['values'] = self.columns

            person_guess = best_guess_column(self.columns,
                ['zasobnik','osoba','assignee','owner','wykonawca','przypisane do','assigned to','user'])
            created_guess = best_guess_column(self.columns,
                ['data utworzenia','utworzenia','created','creation date','created at','start date'])
            completed_guess = best_guess_column(self.columns,
                ['data ukończenia','ukonczenia','completed','done','closed','end date','resolution date'])
            task_guess = best_guess_column(self.columns,
                ['nazwa zadania','tytuł','title','task','nazwa'])
            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)
            if task_guess: self.task_var.set(task_guess)

            self.log(f"Załadowano Excel #1: {os.path.basename(path)} | Arkusz: {self.sheet_master.get()} | Wierszy: {len(self.df_master)}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać Excel #1:\n{e}")
            self.log(f"Błąd wczytywania Excel #1: {e}")

    # --- Merge user file ---
    def load_and_merge_user(self):
        if self.df_master is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj lub ustaw Excel #1 (master).")
            return
        path = filedialog.askopenfilename(title="Wybierz plik Excel #2",
                                        filetypes=[("Excel files","*.xlsx *.xls *.xlsm")])
        if not path:
            return
        try:
            df2 = pd.read_excel(path, sheet_name=detect_sheet(path) or 0)

            # filtr dat (tylko po dacie utworzenia)
            created_col = self.created_var.get()
            created_dt = smart_datetime(df2[created_col]).dt.date
            mask_keep = (created_dt > datetime(2025, 4, 12).date())  # tylko > 12.04.2025
            # UWAGA: wszystkie <= 09.04.2025 i == 12.04.2025 odrzucamy
            df2 = df2.loc[mask_keep].copy()

            # zbuduj klucze
            key_master = build_unique_key(
                self.df_master,
                self.person_var.get(),
                self.task_var.get() if self.task_var.get() else None,
                self.created_var.get(),
                self.completed_var.get()
            )
            key_new = build_unique_key(
                df2,
                self.person_var.get(),
                self.task_var.get() if self.task_var.get() else None,
                self.created_var.get(),
                self.completed_var.get()
            )

            df2['__key__'] = key_new
            master_keys = set(key_master.dropna().tolist())
            before = len(df2)
            df2_new = df2[~df2['__key__'].isin(master_keys)].drop(columns='__key__', errors='ignore')
            added = len(df2_new)

            self.df = pd.concat([self.df_master, df2_new], ignore_index=True)
            self.log(f"Excel #2: {os.path.basename(path)} | wierszy po filtrze: {before} | nowych dopisano: {added} | razem: {len(self.df)}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Scalanie nie powiodło się:\n{e}")
            self.log(f"Błąd scalania: {e}")

    # --- Podgląd / zapis ---
    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Brak danych. Wczytaj Excel #1 lub scal z #2.")
            return
        try:
            out, details = self._aggregate_and_details(self.df,
                                                       self.person_var.get(),
                                                       self.created_var.get(),
                                                       self.completed_var.get())
            top = out.head(50)
            self.log(f"AGREGAT (pierwsze {len(top)}):\n{top.to_string(index=False)}")
            self.log("Szczegóły (podgląd 10):")
            self.log(details.head(10).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Podgląd nie powiódł się:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Brak danych. Wczytaj Excel #1 lub scal z #2.")
            return
        try:
            out, details = self._aggregate_and_details(self.df,
                                                       self.person_var.get(),
                                                       self.created_var.get(),
                                                       self.completed_var.get())
            pivot = make_chart_df(self.df,
                                  self.person_var.get(),
                                  self.created_var.get(),
                                  self.completed_var.get())

            base = os.path.splitext(self.path_master.get().strip() or "wynik")[0]
            default_out = base + "_WYNIK_z_wykresem.xlsx"
            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files","*.xlsx")]
            )
            if not path:
                self.log("Anulowano zapis.")
                return

            write_excel_with_chart(path, pivot_df=pivot, aggr_df=out, details_df=details,
                                   title="ZADANIE UTWORZONE DO ZAKOŃCZONE")
            self.log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Zapis nie powiódł się:\n{e}")
            self.log(f"Błąd zapisu: {e}")

    # --- Agregaty / szczegóły ---
    def _aggregate_and_details(self, df, col_person, col_created, col_completed):
        person_raw = norm_text(df[col_person])
        person = person_raw.replace('', '—')
        keep = filter_bins(person)
        person = person[keep]
        created = smart_datetime(df[col_created])[keep]
        completed = smart_datetime(df[col_completed])[keep]

        # Ukończone wg daty ukończenia
        done = (pd.DataFrame({'rok': completed.dt.year, 'kwartał': completed.dt.quarter, 'zasobnik': person})
                .dropna(subset=['rok','kwartał'])
                .groupby(['rok','kwartał','zasobnik'], dropna=False).size()
                .rename('zadania_ukończone').reset_index())

        # Nieukończone wg braku ukończenia – liczone po dacie utworzenia
        not_done_mask = completed.isna()
        not_done_created = created.where(not_done_mask)
        not_done = (pd.DataFrame({'rok': not_done_created.dt.year,
                                  'kwartał': not_done_created.dt.quarter,
                                  'zasobnik': person.where(not_done_mask)})
                    .dropna(subset=['rok','kwartał','zasobnik'])
                    .groupby(['rok','kwartał','zasobnik'], dropna=False).size()
                    .rename('zadania_nieukończone').reset_index())

        out = pd.merge(done, not_done, on=['rok','kwartał','zasobnik'], how='outer')
        out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
        out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)
        out.sort_values(by=['zasobnik','rok','kwartał'],
                        key=lambda col: col.str.casefold() if col.name=='zasobnik' else col,
                        inplace=True)
        out.reset_index(drop=True, inplace=True)

        # Szczegóły – oba zbiory (ukończone i nieukończone)
        det_done = pd.DataFrame({
            'osoba': person,
            'data_utworzenia': created,
            'data_ukonczenia': completed,
            'status': pd.Series(['ukończone']*len(person)),
            'rok': completed.dt.year,
            'kwartał': completed.dt.quarter,
        }).dropna(subset=['rok','kwartał'])

        det_not_done = pd.DataFrame({
            'osoba': person.where(not_done_mask),
            'data_utworzenia': created.where(not_done_mask),
            'data_ukonczenia': completed.where(not_done_mask),
            'status': pd.Series(['nieukończone']*len(person)).where(not_done_mask),
            'rok': not_done_created.dt.year,
            'kwartał': not_done_created.dt.quarter,
        }).dropna(subset=['osoba','rok','kwartał'])

        details = pd.concat([det_done, det_not_done], ignore_index=True).sort_values(
            by=['osoba','rok','kwartał','data_utworzenia','data_ukonczenia'],
            key=lambda col: col.str.casefold() if col.name=='osoba' else col
        ).reset_index(drop=True)

        return out, details

# ---- run ---------------------------------------------------------------------

def main():
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()

if __name__ == "__main__":
    main()


In [8]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from datetime import datetime
from typing import Optional  # ← ważne

APP_TITLE = "Konwerter zadań: Excel → Excel (rok/kwartał/zasobnik)"
APP_GEOMETRY = "820x560"

# ←← USTAW ŚCIEŻKĘ DO DOMYŚLNEGO PLIKU (Excel #1)
DEFAULT_MASTER_PATH = r"C:\Sciezka\do\Excel1_master.xlsx"

# ==== KONFIG – data „fałszywa” z nowego systemu (Panasonic) ==================
SENTINEL_CREATED = datetime(2025, 4, 10).date()   # 10.04.2025

# ====== Pomocnicze ============================================================

def best_guess_column(columns, candidates):
    import unicodedata
    def norm(s):
        s = str(s)
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
        s = s.lower()
        for ch in ['-', '_', ' ', '/', '\\', '.', '(', ')', '[', ']', ':', ';', '|']:
            s = s.replace(ch, '')
        return s
    cols_norm = {col: norm(col) for col in columns}
    cand_norm = [norm(c) for c in candidates]
    for col, ncol in cols_norm.items():
        if ncol in cand_norm:
            return col
    best, best_len = None, 0
    for col, ncol in cols_norm.items():
        for c in cand_norm:
            if c in ncol and len(c) > best_len:
                best, best_len = col, len(c)
    return best or (columns[0] if columns else None)

def detect_sheet(path):
    try:
        xls = pd.ExcelFile(path)
        preferred = [s for s in xls.sheet_names
                     if str(s).strip().lower() in {'data','dane','tasks','zadania','sheet1','arkusz1'}]
        return preferred[0] if preferred else xls.sheet_names[0]
    except Exception:
        return None

def norm_text(s: pd.Series) -> pd.Series:
    return (s.astype(str)
              .str.strip()
              .str.replace(r'\s+', ' ', regex=True)
              .str.replace('\u200b', '', regex=False)  # zero-width space
              .replace({'nan':'', 'None':''}))

def smart_datetime(s: pd.Series) -> pd.Series:
    """Solidne parsowanie dat (PL dd.mm.rrrr i ISO). Zwraca datetime64[ns]."""
    if not isinstance(s, pd.Series):
        s = pd.Series(s)
    dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
    # dopalacz dla  dd.mm.yyyy HH:MM  /  dd.mm.yyyy
    mask = dt.isna() & s.astype(str).str.contains(r"\d{1,2}\.\d{1,2}\.\d{2,4}")
    if mask.any():
        vals = s[mask].astype(str).str.strip()
        dt2 = pd.to_datetime(vals, format='%d.%m.%Y', errors='coerce')
        still = dt2.isna()
        if still.any():
            dt2.loc[still] = pd.to_datetime(vals[still], format='%d.%m.%Y %H:%M', errors='coerce')
        dt.loc[mask] = dt2
    return dt

def norm_date_string(s: pd.Series) -> pd.Series:
    dt = smart_datetime(s)
    return dt.dt.strftime('%Y-%m-%d').fillna('')

def filter_bins(person: pd.Series) -> pd.Series:
    up = person.str.upper()
    return ~(up.isin(["TO DO", "INSTRUKCJA", "HOLD"]))

def make_chart_df(df, col_person, col_created, col_completed):
    # Osoba/rok/kwartał + 2 serie: utworzone (wg Data utworzenia), zakończone (wg Data ukończenia)
    person = norm_text(df[col_person]).replace('', '—')
    keep = filter_bins(person)
    person = person[keep]
    created = smart_datetime(df[col_created])[keep]
    completed = smart_datetime(df[col_completed])[keep]

    created_df = (
        pd.DataFrame({'osoba': person, 'rok': created.dt.year, 'kwartał': created.dt.quarter})
        .dropna(subset=['osoba','rok','kwartał'])
        .groupby(['osoba','rok','kwartał']).size()
        .rename('Liczba UTWORZONYCH').reset_index()
    )
    done_df = (
        pd.DataFrame({'osoba': person, 'rok': completed.dt.year, 'kwartał': completed.dt.quarter})
        .dropna(subset=['rok','kwartał'])
        .groupby(['osoba','rok','kwartał']).size()
        .rename('Liczba ZAKOŃCZONYCH').reset_index()
    )
    pivot = pd.merge(created_df, done_df, on=['osoba','rok','kwartał'], how='outer')
    pivot['Liczba UTWORZONYCH']  = pivot['Liczba UTWORZONYCH'].fillna(0).astype(int)
    pivot['Liczba ZAKOŃCZONYCH'] = pivot['Liczba ZAKOŃCZONYCH'].fillna(0).astype(int)
    pivot = pivot.sort_values(by=['osoba','rok','kwartał'],
                              key=lambda s: s.str.casefold() if s.name=='osoba' else s).reset_index(drop=True)
    return pivot

def write_excel_with_chart(path, pivot_df, aggr_df=None, details_df=None, title="ZADANIE UTWORZONE DO ZAKOŃCZONE"):
    with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
        book = writer.book
        if aggr_df is not None:
            aggr_df.to_excel(writer, sheet_name="Agregat", index=False)
        if details_df is not None:
            details_df.to_excel(writer, sheet_name="Szczegóły", index=False)

        # === PIVOT (oś 3-poziomowa: Osoba / Rok / Kwartał) ===
        ws_pivot = book.add_worksheet('Pivot'); writer.sheets['Pivot'] = ws_pivot
        cat_person = pivot_df['osoba'].tolist()
        cat_year   = pivot_df['rok'].astype(int).tolist()
        cat_quart  = pivot_df['kwartał'].astype(int).map(lambda q: f"KWARTAŁ{q}").tolist()

        ws_pivot.write_row(0, 0, ["Osoba (oś)", *cat_person])
        ws_pivot.write_row(1, 0, ["Rok (oś)",   *cat_year])
        ws_pivot.write_row(2, 0, ["Kwartał (oś)", *cat_quart])

        ws_pivot.write(4, 0, "Liczba UTWORZONYCH")
        ws_pivot.write_row(4, 1, pivot_df["Liczba UTWORZONYCH"].tolist())
        ws_pivot.write(5, 0, "Liczba ZAKOŃCZONYCH")
        ws_pivot.write_row(5, 1, pivot_df["Liczba ZAKOŃCZONYCH"].tolist())

        # === DODATKOWA SEKCJA: SUMA PER KWARTAŁ (bez podziału na osoby) ===
        if not pivot_df.empty:
            per_q = (pivot_df
                     .groupby(['rok', 'kwartał'], as_index=False)[['Liczba UTWORZONYCH','Liczba ZAKOŃCZONYCH']]
                     .sum()
                     .sort_values(['rok','kwartał']))
            q_labels = [f"{int(r)} | KWARTAŁ{int(k)}" for r, k in zip(per_q['rok'], per_q['kwartał'])]
            ws_pivot.write_row(8, 0, ["Kwartał (oś)", *q_labels])
            ws_pivot.write(10, 0, "UTWORZONE (suma)")
            ws_pivot.write_row(10, 1, per_q['Liczba UTWORZONYCH'].tolist())
            ws_pivot.write(11, 0, "ZAKOŃCZONE (suma)")
            ws_pivot.write_row(11, 1, per_q['Liczba ZAKOŃCZONYCH'].tolist())
        else:
            q_labels = []

        # === Arkusz z wykresami ===
        ws_chart = book.add_worksheet('Wykres'); writer.sheets['Wykres'] = ws_chart

        # Wykres 1: OSOBY × KWARTAŁY (stacked)
        chart1 = book.add_chart({'type': 'column','subtype': 'stacked'})
        last_col = len(pivot_df)
        chart1.add_series({
            'name':       "='Pivot'!$A$5",   # UTWORZONE
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 4, 1, 4, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#43A047'},  # zielony
            'border': {'color': '#43A047'},
        })
        chart1.add_series({
            'name':       "='Pivot'!$A$6",   # ZAKOŃCZONE
            'categories': ['Pivot', 0, 1, 2, last_col],
            'values':     ['Pivot', 5, 1, 5, last_col],
            'data_labels': {'value': True},
            'fill': {'color': '#1E88E5'},  # niebieski
            'border': {'color': '#1E88E5'},
        })
        chart1.set_title({'name': title})
        chart1.set_legend({'position': 'top'})
        chart1.set_y_axis({'major_gridlines': {'visible': True}})
        ws_chart.insert_chart('B2', chart1, {'x_scale': 2.0, 'y_scale': 1.6})

        # Wykres 2: SUMA PER KWARTAŁ (CLUSTERED, bez podziału na osoby)
        if q_labels:
            chart2 = book.add_chart({'type': 'column'})  # clustered
            last_col_q = len(q_labels)
            chart2.add_series({
                'name':       "='Pivot'!$A$11",  # UTWORZONE (suma)
                'categories': ['Pivot', 8, 1, 8, last_col_q],
                'values':     ['Pivot', 10, 1, 10, last_col_q],
                'data_labels': {'value': True},
                'fill': {'color': '#43A047'},
                'border': {'color': '#43A047'},
            })
            chart2.add_series({
                'name':       "='Pivot'!$A$12",  # ZAKOŃCZONE (suma)
                'categories': ['Pivot', 8, 1, 8, last_col_q],
                'values':     ['Pivot', 11, 1, 11, last_col_q],
                'data_labels': {'value': True},
                'fill': {'color': '#1E88E5'},
                'border': {'color': '#1E88E5'},
            })
            chart2.set_title({'name': 'Suma kwartalna — Utworzone vs Zakończone'})
            chart2.set_legend({'position': 'top'})
            chart2.set_y_axis({'major_gridlines': {'visible': True}})
            # "obok" pierwszego wykresu:
            ws_chart.insert_chart('N2', chart2, {'x_scale': 1.6, 'y_scale': 1.6})


# ====== Budowa klucza unikalności (anty-duplikacja) ===========================

def build_unique_key(df: pd.DataFrame,
                     col_person: str,
                     col_task: Optional[str],
                     col_created: str,
                     col_completed: str,
                     id_col: str = "Identyfikator zadania") -> pd.Series:
    """
    Klucz unikalności:
      1) Jeśli istnieje kolumna z ID – użyj jej.
      2) W przeciwnym razie:
         - bazowo: (osoba, nazwa_zadania, data_utworzenia)
         - jeśli data_utworzenia == 12.04.2025 (SENTINEL) → POMIŃ ją w kluczu
           i dołóż data_ukończenia (jeśli jest), aby rozróżnić różne zadania.
         - jeśli brak kolumny nazwy, użyj (osoba, [data_utworzenia?], [data_ukończenia?]) z powyższą regułą.
    """
    cols = list(df.columns)
    if id_col in cols:
        return norm_text(df[id_col])

    person = norm_text(df.get(col_person, pd.Series(['']*len(df)))).str.lower()
    task   = norm_text(df.get(col_task,   pd.Series(['']*len(df)))).str.lower() if col_task else pd.Series(['']*len(df))
    created_dt   = smart_datetime(df.get(col_created,   pd.Series(['']*len(df))))
    completed_dt = smart_datetime(df.get(col_completed, pd.Series(['']*len(df))))

    created_date   = created_dt.dt.date
    completed_date = completed_dt.dt.date

    created_str   = created_date.astype(str).where(pd.notna(created_date), "")
    completed_str = completed_date.astype(str).where(pd.notna(completed_date), "")

    use_created = ~created_date.eq(SENTINEL_CREATED)

    key = (
        person + "||" +
        task.fillna("") + "||" +
        created_str.where(use_created, "") + "||" +
        completed_str.where(use_created, completed_str)
    )
    return key

# ====== GUI ===================================================================

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        self.path_master = tk.StringVar(value=DEFAULT_MASTER_PATH)
        self.path_user   = tk.StringVar()

        self.sheet_master = tk.StringVar()
        self.sheet_user   = tk.StringVar()

        self.person_var = tk.StringVar()
        self.created_var = tk.StringVar()
        self.completed_var = tk.StringVar()
        self.task_var = tk.StringVar()  # (opcjonalnie) nazwa zadania do klucza

        self.df_master = None   # Excel #1
        self.df = None          # scalony master + user
        self.columns = []

        self.create_widgets()
        self.load_master_on_start()

    # --- UI ---
    def create_widgets(self):
        pad = {'padx': 8, 'pady': 6}
        frm = ttk.Frame(self); frm.pack(fill='both', expand=True)

        # MASTER (auto)
        row0 = ttk.LabelFrame(frm, text="Excel #1 (domyślny – ładowany automatycznie)")
        row0.pack(fill='x', **pad)
        r0 = ttk.Frame(row0); r0.pack(fill='x', **pad)
        ttk.Entry(r0, textvariable=self.path_master).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r0, text="Zmień…", command=self.pick_master).pack(side='left')
        r0b = ttk.Frame(row0); r0b.pack(fill='x', **pad)
        ttk.Label(r0b, text="Arkusz:").pack(side='left')
        self.sheet_combo_master = ttk.Combobox(r0b, textvariable=self.sheet_master, state='readonly', width=30)
        self.sheet_combo_master.pack(side='left', padx=6)
        ttk.Button(r0b, text="Przeładuj Excel #1", command=self.load_master_on_start).pack(side='left')

        # USER (scalanie)
        row1 = ttk.LabelFrame(frm, text="Excel #2 (od użytkownika – nowe wiersze zostaną dopisane)")
        row1.pack(fill='x', **pad)
        r1 = ttk.Frame(row1); r1.pack(fill='x', **pad)
        ttk.Entry(r1, textvariable=self.path_user).pack(side='left', fill='x', expand=True, padx=6)
        ttk.Button(r1, text="➕ Wczytaj plik 2 i scal", command=self.load_and_merge_user).pack(side='left')

        # Mapowanie
        box = ttk.LabelFrame(frm, text="Mapowanie kolumn")
        box.pack(fill='x', **pad)

        r_task = ttk.Frame(box); r_task.pack(fill='x', **pad)
        ttk.Label(r_task, text="Nazwa zadania (opcjonalnie, do klucza):").pack(side='left')
        self.task_combo = ttk.Combobox(r_task, textvariable=self.task_var, state='readonly', width=40)
        self.task_combo.pack(side='left', fill='x', expand=True, padx=6)

        r2 = ttk.Frame(box); r2.pack(fill='x', **pad)
        ttk.Label(r2, text="Zasobnik (osoba):").pack(side='left')
        self.person_combo = ttk.Combobox(r2, textvariable=self.person_var, state='readonly', width=40)
        self.person_combo.pack(side='left', fill='x', expand=True, padx=6)

        r3 = ttk.Frame(box); r3.pack(fill='x', **pad)
        ttk.Label(r3, text="Data utworzenia:").pack(side='left')
        self.created_combo = ttk.Combobox(r3, textvariable=self.created_var, state='readonly', width=40)
        self.created_combo.pack(side='left', fill='x', expand=True, padx=6)

        r4 = ttk.Frame(box); r4.pack(fill='x', **pad)
        ttk.Label(r4, text="Data ukończenia:").pack(side='left')
        self.completed_combo = ttk.Combobox(r4, textvariable=self.completed_var, state='readonly', width=40)
        self.completed_combo.pack(side='left', fill='x', expand=True, padx=6)

        # Akcje
        actions = ttk.Frame(frm); actions.pack(fill='x', **pad)
        ttk.Button(actions, text="Podgląd wyniku", command=self.preview).pack(side='right', padx=6)
        ttk.Button(actions, text="Konwertuj i zapisz…", command=self.convert_and_save).pack(side='right')

        # Log
        self.status = tk.Text(frm, height=10); self.status.pack(fill='both', expand=True, **pad)
        self.status.configure(state='disabled')

        self.log("Start: wczytam Excel #1 (master). Potem doładuj Excel #2 – dopiszę tylko nowe wiersze.")

    def log(self, msg):
        self.status.configure(state='normal')
        self.status.insert('end', f"{msg}\n")
        self.status.see('end')
        self.status.configure(state='disabled')

    # --- Master load ---
    def pick_master(self):
        path = filedialog.askopenfilename(title="Wybierz plik Excel #1 (master)",
                                          filetypes=[("Excel files","*.xlsx *.xls *.xlsm")])
        if path:
            self.path_master.set(path)
            self.load_master_on_start()

    def load_master_on_start(self):
        path = self.path_master.get().strip()
        if not path or not os.path.isfile(path):
            self.log("Brak/nieprawidłowa ścieżka Excel #1 – pomiń wczytanie.")
            return
        try:
            xls = pd.ExcelFile(path)
            self.sheet_combo_master['values'] = xls.sheet_names
            preferred = detect_sheet(path)
            self.sheet_master.set(preferred or (xls.sheet_names[0] if xls.sheet_names else ""))
            self.df_master = pd.read_excel(path, sheet_name=self.sheet_master.get() or 0)
            self.df = self.df_master.copy()  # na starcie zestaw = master
            self.columns = list(self.df_master.columns)

            # mapowanie
            self.person_combo['values'] = self.columns
            self.created_combo['values'] = self.columns
            self.completed_combo['values'] = self.columns
            self.task_combo['values'] = self.columns

            person_guess = best_guess_column(self.columns,
                ['zasobnik','osoba','assignee','owner','wykonawca','przypisane do','assigned to','user'])
            created_guess = best_guess_column(self.columns,
                ['data utworzenia','utworzenia','created','creation date','created at','start date'])
            completed_guess = best_guess_column(self.columns,
                ['data ukończenia','ukonczenia','completed','done','closed','end date','resolution date'])
            task_guess = best_guess_column(self.columns,
                ['nazwa zadania','tytuł','title','task','nazwa'])
            if person_guess: self.person_var.set(person_guess)
            if created_guess: self.created_var.set(created_guess)
            if completed_guess: self.completed_var.set(completed_guess)
            if task_guess: self.task_var.set(task_guess)

            self.log(f"Załadowano Excel #1: {os.path.basename(path)} | Arkusz: {self.sheet_master.get()} | Wierszy: {len(self.df_master)}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się wczytać Excel #1:\n{e}")
            self.log(f"Błąd wczytywania Excel #1: {e}")

    # --- Merge user file ---
    def load_and_merge_user(self):
        if self.df_master is None:
            messagebox.showwarning("Uwaga", "Najpierw wczytaj lub ustaw Excel #1 (master).")
            return
        path = filedialog.askopenfilename(title="Wybierz plik Excel #2",
                                        filetypes=[("Excel files","*.xlsx *.xls *.xlsm")])
        if not path:
            return
        try:
            df2 = pd.read_excel(path, sheet_name=detect_sheet(path) or 0)

            # filtr dat (tylko po dacie utworzenia)
            created_col = self.created_var.get()
            created_dt = smart_datetime(df2[created_col]).dt.date
            mask_keep = (created_dt > datetime(2025, 4, 12).date())  # tylko > 12.04.2025
            # UWAGA: wszystkie <= 09.04.2025 i == 12.04.2025 odrzucamy
            df2 = df2.loc[mask_keep].copy()

            # zbuduj klucze
            key_master = build_unique_key(
                self.df_master,
                self.person_var.get(),
                self.task_var.get() if self.task_var.get() else None,
                self.created_var.get(),
                self.completed_var.get()
            )
            key_new = build_unique_key(
                df2,
                self.person_var.get(),
                self.task_var.get() if self.task_var.get() else None,
                self.created_var.get(),
                self.completed_var.get()
            )

            df2['__key__'] = key_new
            master_keys = set(key_master.dropna().tolist())
            before = len(df2)
            df2_new = df2[~df2['__key__'].isin(master_keys)].drop(columns='__key__', errors='ignore')
            added = len(df2_new)

            self.df = pd.concat([self.df_master, df2_new], ignore_index=True)
            self.log(f"Excel #2: {os.path.basename(path)} | wierszy po filtrze: {before} | nowych dopisano: {added} | razem: {len(self.df)}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Scalanie nie powiodło się:\n{e}")
            self.log(f"Błąd scalania: {e}")

    # --- Podgląd / zapis ---
    def preview(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Brak danych. Wczytaj Excel #1 lub scal z #2.")
            return
        try:
            out, details = self._aggregate_and_details(self.df,
                                                       self.person_var.get(),
                                                       self.created_var.get(),
                                                       self.completed_var.get())
            top = out.head(50)
            self.log(f"AGREGAT (pierwsze {len(top)}):\n{top.to_string(index=False)}")
            self.log("Szczegóły (podgląd 10):")
            self.log(details.head(10).to_string(index=False))
        except Exception as e:
            messagebox.showerror("Błąd", f"Podgląd nie powiódł się:\n{e}")
            self.log(f"Błąd podglądu: {e}")

    def convert_and_save(self):
        if self.df is None:
            messagebox.showwarning("Uwaga", "Brak danych. Wczytaj Excel #1 lub scal z #2.")
            return
        try:
            out, details = self._aggregate_and_details(self.df,
                                                       self.person_var.get(),
                                                       self.created_var.get(),
                                                       self.completed_var.get())
            pivot = make_chart_df(self.df,
                                  self.person_var.get(),
                                  self.created_var.get(),
                                  self.completed_var.get())

            base = os.path.splitext(self.path_master.get().strip() or "wynik")[0]
            default_out = base + "_WYNIK_z_wykresem.xlsx"
            path = filedialog.asksaveasfilename(
                title="Zapisz wynik jako",
                defaultextension=".xlsx",
                initialfile=os.path.basename(default_out),
                filetypes=[("Excel files","*.xlsx")]
            )
            if not path:
                self.log("Anulowano zapis.")
                return

            write_excel_with_chart(path, pivot_df=pivot, aggr_df=out, details_df=details,
                                   title="ZADANIE UTWORZONE DO ZAKOŃCZONE")
            self.log(f"Zapisano wynik do: {path}")
            messagebox.showinfo("Sukces", f"Zapisano wynik do:\n{path}")
        except Exception as e:
            messagebox.showerror("Błąd", f"Zapis nie powiódł się:\n{e}")
            self.log(f"Błąd zapisu: {e}")

    # --- Agregaty / szczegóły ---
    def _aggregate_and_details(self, df, col_person, col_created, col_completed):
        person_raw = norm_text(df[col_person])
        person = person_raw.replace('', '—')
        keep = filter_bins(person)
        person = person[keep]
        created = smart_datetime(df[col_created])[keep]
        completed = smart_datetime(df[col_completed])[keep]

        # Ukończone wg daty ukończenia
        done = (pd.DataFrame({'rok': completed.dt.year, 'kwartał': completed.dt.quarter, 'zasobnik': person})
                .dropna(subset=['rok','kwartał'])
                .groupby(['rok','kwartał','zasobnik'], dropna=False).size()
                .rename('zadania_ukończone').reset_index())

        # Nieukończone wg braku ukończenia – liczone po dacie utworzenia
        not_done_mask = completed.isna()
        not_done_created = created.where(not_done_mask)
        not_done = (pd.DataFrame({'rok': not_done_created.dt.year,
                                  'kwartał': not_done_created.dt.quarter,
                                  'zasobnik': person.where(not_done_mask)})
                    .dropna(subset=['rok','kwartał','zasobnik'])
                    .groupby(['rok','kwartał','zasobnik'], dropna=False).size()
                    .rename('zadania_nieukończone').reset_index())

        out = pd.merge(done, not_done, on=['rok','kwartał','zasobnik'], how='outer')
        out['zadania_ukończone'] = out['zadania_ukończone'].fillna(0).astype(int)
        out['zadania_nieukończone'] = out['zadania_nieukończone'].fillna(0).astype(int)
        out.sort_values(by=['zasobnik','rok','kwartał'],
                        key=lambda col: col.str.casefold() if col.name=='zasobnik' else col,
                        inplace=True)
        out.reset_index(drop=True, inplace=True)

        # Szczegóły – oba zbiory (ukończone i nieukończone)
        det_done = pd.DataFrame({
            'osoba': person,
            'data_utworzenia': created,
            'data_ukonczenia': completed,
            'status': pd.Series(['ukończone']*len(person)),
            'rok': completed.dt.year,
            'kwartał': completed.dt.quarter,
        }).dropna(subset=['rok','kwartał'])

        det_not_done = pd.DataFrame({
            'osoba': person.where(not_done_mask),
            'data_utworzenia': created.where(not_done_mask),
            'data_ukonczenia': completed.where(not_done_mask),
            'status': pd.Series(['nieukończone']*len(person)).where(not_done_mask),
            'rok': not_done_created.dt.year,
            'kwartał': not_done_created.dt.quarter,
        }).dropna(subset=['osoba','rok','kwartał'])

        details = pd.concat([det_done, det_not_done], ignore_index=True).sort_values(
            by=['osoba','rok','kwartał','data_utworzenia','data_ukonczenia'],
            key=lambda col: col.str.casefold() if col.name=='osoba' else col
        ).reset_index(drop=True)

        return out, details

# ---- run ---------------------------------------------------------------------

def main():
    try:
        import ctypes
        ctypes.windll.shcore.SetProcessDpiAwareness(1)
    except Exception:
        pass
    app = App()
    app.mainloop()

if __name__ == "__main__":
    main()
